<a href="https://colab.research.google.com/github/Rohanrathod7/my-ml-labs/blob/main/05_Linear_Classifiers_in_Python/dataset/04_Support_Vector_Machines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Support Vector Machine(SVM)

In [15]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import datetime as dt
# Import confusion matrix and train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, LogisticRegression, LinearRegression
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier




url = "https://raw.githubusercontent.com/Rohanrathod7/my-ds-labs/main/34_Unsupervised_Learning_in_Python/dataset/Grains/seeds-width-vs-length.csv"
# Read the CSV file
# The original code tried to read a feather file as a CSV, and had a UnicodeDecodeError.
# The file extension is feather, so it should be read using pd.read_feather.
# Also, the variable name was confusing, it should be spotify_population.
grains = pd.read_csv(url, header=None)
display(grains.head())

,0,1
0,3.312,5.763
1,3.333,5.554
2,3.337,5.291
3,3.379,5.324
4,3.562,5.658


**Effect of removing examples**  
Support vectors are defined as training examples that influence the decision boundary. In this exercise, you'll observe this behavior by removing non support vectors from the training set.

The wine quality dataset is already loaded into X and y (first two features only). (Note: we specify lims in plot_classifier() so that the two plots are forced to use the same axis limits and can be compared directly.)

In [ ]:
# Train a linear SVM
svm = SVC(kernel="linear")
svm.fit(X, y)
plot_classifier(X, y, svm, lims=(11,15,0,6))

# Make a new data set keeping only the support vectors
print("Number of original examples", len(X))
print("Number of support vectors", len(svm.support_))
X_small = X[svm.support_]
y_small = y[svm.support_]

# Train a new SVM using only the support vectors
svm_small = SVC(kernel="linear")
svm_small.fit(X_small, y_small)
plot_classifier(X_small, y_small, svm_small, lims=(11,15,0,6))

# Compare the decision boundaries of the two trained models: are they the same? By the definition of support vectors, they should be!

**GridSearchCV warm-up**   
In the video we saw that increasing the RBF kernel hyperparameter gamma increases training accuracy. In this exercise we'll search for the gamma that maximizes cross-validation accuracy using scikit-learn's GridSearchCV. A binary version of the handwritten digits dataset, in which you're just trying to predict whether or not an image is a "2", is already loaded into the variables X and y.

In [3]:
X = [[0.0, 1.0, 10.0, 15.0, 11.0, 1.0, 0.0, 0.0, 0.0, 3.0, 8.0, 8.0, 11.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 10.0, 0.0, 0.0, 0.0, 0.0, 3.0, 4.0, 10.0, 16.0, 1.0, 0.0, 0.0, 0.0, 13.0, 16.0, 15.0, 10.0, 0.0, 0.0], [0.0, 2.0, 9.0, 15.0, 14.0, 9.0, 3.0, 0.0, 0.0, 4.0, 13.0, 8.0, 9.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 11.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 4.0, 0.0, 0.0, 1.0, 5.0, 6.0, 13.0, 16.0, 6.0, 0.0, 0.0, 2.0, 12.0, 12.0, 13.0, 11.0, 0.0, 0.0], [0.0, 0.0, 4.0, 14.0, 15.0, 6.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 16.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 2.0, 8.0, 13.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 8.0, 0.0, 0.0, 0.0, 1.0, 6.0, 13.0, 16.0, 6.0, 0.0, 0.0, 0.0, 4.0, 13.0, 15.0, 9.0, 0.0, 0.0], [0.0, 3.0, 12.0, 12.0, 14.0, 4.0, 0.0, 0.0, 0.0, 1.0, 13.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 12.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 8.0, 0.0, 0.0, 0.0, 6.0, 2.0, 0.0, 8.0, 8.0, 0.0, 0.0, 2.0, 13.0, 16.0, 16.0, 16.0, 2.0, 0.0], [0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 6.0, 0.0, 0.0, 0.0, 3.0, 11.0, 16.0, 16.0, 5.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 4.0, 10.0, 9.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 14.0, 5.0, 0.0], [0.0, 0.0, 9.0, 15.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 13.0, 9.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 2.0, 6.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 14.0, 16.0, 14.0, 8.0, 7.0, 0.0, 0.0, 0.0, 3.0, 14.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 9.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 8.0, 13.0, 7.0, 0.0, 0.0, 0.0, 0.0, 12.0, 0.0, 8.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 15.0, 10.0, 9.0, 1.0, 0.0, 0.0, 12.0, 14.0, 13.0, 16.0, 16.0, 5.0], [0.0, 0.0, 1.0, 13.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 13.0, 2.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 16.0, 16.0, 6.0, 0.0, 0.0, 1.0, 14.0, 16.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 3.0, 12.0, 15.0, 14.0, 7.0, 0.0], [0.0, 0.0, 9.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 7.0, 16.0, 12.0, 7.0, 11.0, 2.0, 0.0, 0.0, 13.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 10.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 12.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 3.0, 0.0, 0.0, 0.0, 1.0, 0.0, 11.0, 16.0, 1.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 6.0, 0.0, 0.0], [0.0, 0.0, 2.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 14.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 13.0, 9.0, 9.0, 16.0, 3.0, 0.0, 0.0, 6.0, 15.0, 6.0, 16.0, 3.0, 0.0, 0.0, 0.0, 9.0, 13.0, 12.0, 15.0, 12.0, 8.0, 0.0, 0.0, 9.0, 16.0, 16.0, 14.0, 7.0, 2.0, 0.0, 0.0, 1.0, 7.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 7.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 6.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 10.0, 12.0, 9.0, 2.0, 0.0, 0.0, 0.0, 13.0, 8.0, 2.0, 5.0, 13.0, 0.0, 0.0, 0.0, 6.0, 11.0, 1.0, 2.0, 16.0, 3.0, 0.0, 0.0, 0.0, 8.0, 11.0, 14.0, 11.0, 2.0], [0.0, 0.0, 2.0, 13.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 9.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 10.0, 9.0, 9.0, 2.0, 0.0, 0.0, 0.0, 16.0, 11.0, 8.0, 11.0, 12.0, 0.0, 0.0, 1.0, 14.0, 11.0, 1.0, 4.0, 13.0, 0.0, 0.0, 0.0, 3.0, 11.0, 16.0, 15.0, 4.0, 0.0], [0.0, 0.0, 0.0, 0.0, 8.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 3.0, 10.0, 10.0, 0.0, 0.0, 1.0, 14.0, 15.0, 10.0, 16.0, 6.0, 0.0, 0.0, 14.0, 14.0, 12.0, 15.0, 16.0, 2.0, 0.0, 0.0, 3.0, 0.0, 0.0, 8.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 10.0, 0.0, 0.0], [0.0, 0.0, 5.0, 15.0, 14.0, 13.0, 2.0, 0.0, 0.0, 0.0, 12.0, 15.0, 9.0, 7.0, 1.0, 0.0, 0.0, 5.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 6.0, 12.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 12.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 14.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 12.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 10.0, 0.0, 0.0, 0.0], [0.0, 0.0, 8.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 6.0, 15.0, 1.0, 0.0, 0.0, 0.0, 4.0, 10.0, 0.0, 12.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 6.0, 0.0, 0.0, 0.0, 12.0, 9.0, 9.0, 16.0, 2.0, 0.0, 0.0, 0.0, 8.0, 16.0, 15.0, 7.0, 0.0, 0.0], [0.0, 0.0, 6.0, 12.0, 13.0, 2.0, 0.0, 0.0, 0.0, 3.0, 16.0, 6.0, 1.0, 15.0, 0.0, 0.0, 0.0, 5.0, 16.0, 13.0, 12.0, 16.0, 2.0, 0.0, 0.0, 2.0, 13.0, 16.0, 12.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 3.0, 16.0, 2.0, 0.0, 10.0, 7.0, 0.0, 0.0, 0.0, 5.0, 11.0, 16.0, 13.0, 1.0, 0.0], [0.0, 0.0, 4.0, 15.0, 12.0, 2.0, 0.0, 0.0, 0.0, 2.0, 15.0, 7.0, 11.0, 10.0, 0.0, 0.0, 0.0, 4.0, 16.0, 0.0, 0.0, 15.0, 1.0, 0.0, 0.0, 6.0, 10.0, 0.0, 0.0, 10.0, 8.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 6.0, 8.0, 0.0, 0.0, 5.0, 12.0, 0.0, 0.0, 11.0, 8.0, 0.0, 0.0, 2.0, 16.0, 7.0, 8.0, 16.0, 2.0, 0.0, 0.0, 0.0, 6.0, 15.0, 16.0, 8.0, 0.0, 0.0], [0.0, 0.0, 7.0, 12.0, 16.0, 9.0, 0.0, 0.0, 0.0, 4.0, 16.0, 6.0, 7.0, 3.0, 0.0, 0.0, 0.0, 4.0, 16.0, 2.0, 8.0, 3.0, 0.0, 0.0, 0.0, 7.0, 16.0, 15.0, 13.0, 16.0, 3.0, 0.0, 0.0, 5.0, 11.0, 1.0, 1.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 10.0, 11.0, 4.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 12.0, 16.0, 12.0, 0.0, 0.0, 0.0, 5.0, 16.0, 8.0, 4.0, 12.0, 2.0, 0.0, 0.0, 12.0, 6.0, 0.0, 0.0, 13.0, 4.0, 0.0, 0.0, 6.0, 16.0, 13.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 3.0, 4.0, 1.0, 8.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 12.0, 0.0, 0.0, 0.0, 8.0, 9.0, 2.0, 9.0, 9.0, 0.0, 0.0, 0.0, 2.0, 13.0, 16.0, 15.0, 3.0, 0.0], [0.0, 0.0, 0.0, 6.0, 15.0, 11.0, 2.0, 0.0, 0.0, 0.0, 6.0, 13.0, 4.0, 13.0, 5.0, 0.0, 0.0, 0.0, 7.0, 11.0, 0.0, 13.0, 3.0, 0.0, 0.0, 0.0, 2.0, 15.0, 13.0, 7.0, 0.0, 0.0, 0.0, 3.0, 13.0, 12.0, 16.0, 2.0, 0.0, 0.0, 0.0, 8.0, 15.0, 1.0, 9.0, 8.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 8.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 12.0, 16.0, 2.0, 0.0], [0.0, 0.0, 4.0, 11.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 11.0, 13.0, 7.0, 0.0, 0.0, 3.0, 16.0, 12.0, 0.0, 4.0, 8.0, 0.0, 0.0, 6.0, 16.0, 5.0, 0.0, 4.0, 8.0, 0.0, 0.0, 7.0, 9.0, 0.0, 0.0, 9.0, 7.0, 0.0, 0.0, 4.0, 10.0, 0.0, 2.0, 15.0, 2.0, 0.0, 0.0, 1.0, 16.0, 12.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 14.0, 1.0, 0.0, 0.0], [0.0, 0.0, 3.0, 15.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 3.0, 14.0, 7.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 0.0, 0.0, 0.0, 0.0, 2.0, 9.0, 15.0, 16.0, 14.0, 0.0, 0.0, 0.0, 7.0, 16.0, 14.0, 6.0, 2.0, 0.0, 0.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 12.0, 0.0, 0.0, 0.0, 0.0], [0.0, 3.0, 15.0, 16.0, 14.0, 1.0, 0.0, 0.0, 0.0, 2.0, 12.0, 13.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 11.0, 8.0, 11.0, 3.0, 0.0, 0.0, 3.0, 16.0, 16.0, 16.0, 12.0, 3.0, 0.0], [0.0, 0.0, 0.0, 5.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 6.0, 1.0, 0.0, 0.0, 0.0, 0.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 2.0, 7.0, 4.0, 0.0, 0.0, 0.0, 7.0, 15.0, 16.0, 13.0, 15.0, 3.0, 0.0, 0.0, 3.0, 16.0, 9.0, 0.0, 1.0, 12.0, 0.0, 0.0, 0.0, 10.0, 12.0, 2.0, 6.0, 13.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 16.0, 5.0, 0.0], [0.0, 0.0, 8.0, 13.0, 12.0, 3.0, 0.0, 0.0, 0.0, 6.0, 15.0, 7.0, 9.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 7.0, 0.0, 0.0, 0.0, 11.0, 2.0, 1.0, 15.0, 7.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 12.0, 0.0, 0.0], [0.0, 0.0, 0.0, 7.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 1.0, 13.0, 7.0, 0.0, 0.0, 0.0, 4.0, 16.0, 6.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 14.0, 1.0, 0.0], [0.0, 0.0, 2.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 16.0, 13.0, 1.0, 0.0, 0.0, 2.0, 16.0, 8.0, 4.0, 7.0, 11.0, 0.0, 0.0, 0.0, 12.0, 11.0, 1.0, 8.0, 11.0, 0.0, 0.0, 0.0, 3.0, 12.0, 16.0, 15.0, 4.0, 0.0], [0.0, 0.0, 5.0, 15.0, 15.0, 2.0, 0.0, 0.0, 0.0, 3.0, 16.0, 9.0, 16.0, 5.0, 0.0, 0.0, 0.0, 5.0, 9.0, 1.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 14.0, 7.0, 6.0, 2.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 15.0, 2.0, 0.0], [0.0, 0.0, 1.0, 13.0, 10.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 8.0, 16.0, 13.0, 10.0, 15.0, 0.0, 0.0, 0.0, 8.0, 16.0, 2.0, 2.0, 15.0, 3.0, 0.0, 0.0, 5.0, 15.0, 2.0, 0.0, 12.0, 7.0, 0.0, 0.0, 1.0, 15.0, 6.0, 2.0, 16.0, 3.0, 0.0, 0.0, 0.0, 11.0, 15.0, 13.0, 16.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 14.0, 8.0, 0.0, 0.0], [0.0, 0.0, 0.0, 13.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 4.0, 0.0, 0.0], [0.0, 0.0, 0.0, 7.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 10.0, 8.0, 6.0, 1.0, 0.0, 0.0, 0.0, 15.0, 16.0, 8.0, 10.0, 8.0, 0.0, 0.0, 0.0, 14.0, 7.0, 0.0, 0.0, 12.0, 0.0, 0.0, 0.0, 8.0, 11.0, 0.0, 5.0, 16.0, 2.0, 0.0, 0.0, 0.0, 9.0, 14.0, 14.0, 5.0, 0.0], [0.0, 0.0, 0.0, 4.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 15.0, 14.0, 0.0, 0.0, 0.0, 0.0, 8.0, 13.0, 8.0, 16.0, 0.0, 0.0, 0.0, 0.0, 1.0, 6.0, 15.0, 11.0, 0.0, 0.0, 0.0, 1.0, 8.0, 13.0, 15.0, 1.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 3.0, 13.0, 16.0, 16.0, 11.0, 5.0, 0.0, 0.0, 0.0, 0.0, 3.0, 11.0, 16.0, 9.0, 0.0], [0.0, 0.0, 6.0, 16.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 14.0, 0.0, 0.0, 0.0, 0.0, 8.0, 7.0, 12.0, 16.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 12.0, 1.0, 0.0], [0.0, 0.0, 9.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 2.0, 16.0, 7.0, 6.0, 15.0, 3.0, 0.0, 0.0, 8.0, 14.0, 0.0, 0.0, 8.0, 3.0, 0.0, 0.0, 5.0, 14.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 2.0, 16.0, 13.0, 11.0, 14.0, 4.0, 0.0, 0.0, 3.0, 16.0, 15.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 8.0, 0.0, 0.0, 0.0], [0.0, 1.0, 7.0, 13.0, 16.0, 13.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 1.0, 7.0, 16.0, 10.0, 1.0, 0.0, 0.0, 0.0, 1.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 15.0, 3.0, 0.0, 0.0, 0.0, 1.0, 5.0, 13.0, 16.0, 7.0, 0.0, 0.0, 0.0, 8.0, 15.0, 10.0, 6.0, 0.0, 0.0], [0.0, 0.0, 4.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 11.0, 16.0, 13.0, 4.0, 0.0, 0.0, 7.0, 16.0, 16.0, 11.0, 14.0, 14.0, 0.0, 0.0, 2.0, 16.0, 11.0, 5.0, 15.0, 12.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 14.0, 3.0, 0.0], [0.0, 0.0, 5.0, 12.0, 8.0, 0.0, 1.0, 0.0, 0.0, 0.0, 11.0, 16.0, 5.0, 13.0, 6.0, 0.0, 0.0, 0.0, 2.0, 15.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 6.0, 12.0, 12.0, 3.0, 0.0, 0.0], [0.0, 0.0, 0.0, 7.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 8.0, 0.0, 0.0, 1.0, 0.0, 0.0, 4.0, 16.0, 3.0, 1.0, 10.0, 10.0, 0.0, 0.0, 8.0, 16.0, 12.0, 14.0, 13.0, 3.0, 0.0, 0.0, 2.0, 12.0, 10.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 13.0, 0.0, 0.0, 0.0], [0.0, 0.0, 10.0, 15.0, 15.0, 11.0, 4.0, 0.0, 0.0, 1.0, 10.0, 5.0, 7.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 10.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 7.0, 15.0, 3.0, 0.0, 0.0, 0.0, 6.0, 11.0, 16.0, 8.0, 0.0, 0.0], [0.0, 0.0, 1.0, 13.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 14.0, 6.0, 0.0, 13.0, 3.0, 0.0, 0.0, 6.0, 10.0, 0.0, 0.0, 10.0, 6.0, 0.0, 0.0, 7.0, 13.0, 0.0, 0.0, 9.0, 8.0, 0.0, 0.0, 3.0, 16.0, 1.0, 3.0, 14.0, 7.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 6.0, 0.0, 0.0], [0.0, 0.0, 0.0, 2.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 2.0, 5.0, 0.0, 0.0, 0.0, 5.0, 16.0, 6.0, 6.0, 16.0, 0.0, 0.0, 2.0, 16.0, 10.0, 4.0, 13.0, 13.0, 0.0, 0.0, 13.0, 16.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 6.0, 4.0, 4.0, 11.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 7.0, 0.0, 0.0], [0.0, 0.0, 11.0, 16.0, 12.0, 2.0, 0.0, 0.0, 0.0, 7.0, 16.0, 6.0, 10.0, 13.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 3.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 12.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 15.0, 7.0, 0.0, 0.0, 3.0, 16.0, 7.0, 6.0, 14.0, 8.0, 0.0, 0.0, 1.0, 9.0, 15.0, 16.0, 12.0, 1.0, 0.0], [0.0, 0.0, 11.0, 16.0, 12.0, 12.0, 16.0, 7.0, 0.0, 3.0, 16.0, 9.0, 8.0, 16.0, 10.0, 0.0, 0.0, 1.0, 10.0, 0.0, 6.0, 14.0, 2.0, 0.0, 0.0, 0.0, 1.0, 8.0, 15.0, 15.0, 11.0, 0.0, 0.0, 0.0, 12.0, 16.0, 15.0, 7.0, 2.0, 0.0, 0.0, 0.0, 4.0, 13.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 16.0, 4.0, 0.0, 0.0, 2.0, 13.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 12.0, 2.0, 0.0], [0.0, 0.0, 0.0, 9.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 3.0, 2.0, 0.0, 0.0, 0.0, 0.0, 15.0, 2.0, 1.0, 14.0, 3.0, 0.0, 0.0, 5.0, 13.0, 0.0, 13.0, 8.0, 1.0, 0.0, 0.0, 8.0, 13.0, 3.0, 16.0, 14.0, 6.0, 0.0, 0.0, 6.0, 15.0, 16.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 7.0, 0.0, 0.0, 0.0], [0.0, 0.0, 8.0, 12.0, 13.0, 5.0, 0.0, 0.0, 0.0, 4.0, 13.0, 4.0, 9.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 13.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 2.0, 0.0, 0.0, 7.0, 8.0, 0.0, 0.0, 12.0, 7.0, 0.0, 0.0, 8.0, 9.0, 1.0, 3.0, 16.0, 3.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 6.0, 0.0, 0.0], [0.0, 0.0, 0.0, 3.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 9.0, 0.0, 0.0, 0.0, 3.0, 12.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 13.0, 16.0, 9.0, 16.0, 8.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 9.0, 0.0, 0.0], [0.0, 0.0, 0.0, 4.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 15.0, 1.0, 11.0, 9.0, 0.0, 0.0, 3.0, 14.0, 8.0, 0.0, 14.0, 10.0, 0.0, 0.0, 10.0, 16.0, 12.0, 12.0, 16.0, 8.0, 0.0, 0.0, 13.0, 16.0, 14.0, 15.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 9.0, 0.0, 0.0], [0.0, 1.0, 8.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 4.0, 13.0, 5.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 14.0, 6.0, 2.0, 0.0, 0.0, 0.0, 12.0, 16.0, 14.0, 13.0, 8.0, 0.0, 0.0, 0.0, 8.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 8.0, 14.0, 15.0, 5.0, 0.0, 0.0, 1.0, 14.0, 8.0, 1.0, 14.0, 8.0, 0.0, 0.0, 7.0, 12.0, 0.0, 7.0, 16.0, 8.0, 0.0, 0.0, 4.0, 14.0, 12.0, 12.0, 9.0, 8.0, 0.0, 0.0, 0.0, 1.0, 3.0, 0.0, 9.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 6.0, 0.0, 0.0, 0.0, 12.0, 10.0, 4.0, 16.0, 0.0, 0.0, 0.0, 0.0, 2.0, 8.0, 16.0, 7.0, 0.0, 0.0], [0.0, 2.0, 15.0, 16.0, 16.0, 14.0, 2.0, 0.0, 0.0, 3.0, 16.0, 14.0, 9.0, 10.0, 1.0, 0.0, 0.0, 7.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 13.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 8.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 16.0, 13.0, 10.0, 16.0, 3.0, 0.0, 0.0, 0.0, 12.0, 1.0, 2.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 15.0, 2.0, 0.0, 0.0, 0.0, 1.0, 11.0, 16.0, 12.0, 8.0, 8.0, 1.0, 0.0, 0.0, 11.0, 16.0, 16.0, 16.0, 12.0, 1.0], [0.0, 1.0, 12.0, 15.0, 16.0, 13.0, 1.0, 0.0, 0.0, 4.0, 16.0, 15.0, 7.0, 15.0, 4.0, 0.0, 0.0, 0.0, 16.0, 6.0, 11.0, 15.0, 2.0, 0.0, 0.0, 0.0, 9.0, 16.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 15.0, 11.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 10.0, 12.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 16.0, 10.0, 0.0, 0.0, 0.0], [0.0, 1.0, 14.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 1.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 10.0, 7.0, 10.0, 6.0, 4.0, 0.0, 0.0, 0.0, 1.0, 12.0, 16.0, 14.0, 5.0, 0.0, 0.0, 0.0, 3.0, 14.0, 16.0, 4.0, 0.0, 0.0, 0.0, 1.0, 15.0, 5.0, 6.0, 12.0, 0.0, 0.0, 0.0, 4.0, 10.0, 0.0, 1.0, 15.0, 0.0, 0.0, 0.0, 1.0, 12.0, 12.0, 12.0, 5.0, 0.0, 0.0], [0.0, 0.0, 5.0, 16.0, 12.0, 2.0, 0.0, 0.0, 0.0, 0.0, 13.0, 14.0, 15.0, 11.0, 0.0, 0.0, 0.0, 6.0, 15.0, 1.0, 2.0, 16.0, 4.0, 0.0, 0.0, 6.0, 14.0, 0.0, 0.0, 9.0, 8.0, 0.0, 0.0, 8.0, 10.0, 0.0, 0.0, 13.0, 8.0, 0.0, 0.0, 4.0, 13.0, 0.0, 1.0, 14.0, 8.0, 0.0, 0.0, 0.0, 14.0, 14.0, 15.0, 15.0, 3.0, 0.0, 0.0, 0.0, 5.0, 12.0, 13.0, 8.0, 0.0, 0.0], [0.0, 0.0, 2.0, 9.0, 13.0, 12.0, 2.0, 0.0, 0.0, 1.0, 14.0, 13.0, 7.0, 10.0, 6.0, 0.0, 0.0, 0.0, 13.0, 8.0, 1.0, 7.0, 7.0, 0.0, 0.0, 0.0, 3.0, 13.0, 14.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 9.0, 5.0, 8.0, 5.0, 0.0, 0.0, 0.0, 0.0, 13.0, 4.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 13.0, 15.0, 6.0, 0.0, 0.0], [0.0, 0.0, 12.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 15.0, 10.0, 1.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 7.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 9.0, 0.0, 0.0, 0.0, 5.0, 4.0, 12.0, 16.0, 4.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 10.0, 0.0, 0.0], [0.0, 0.0, 1.0, 9.0, 15.0, 10.0, 1.0, 0.0, 0.0, 0.0, 2.0, 12.0, 8.0, 12.0, 4.0, 0.0, 0.0, 0.0, 0.0, 11.0, 1.0, 11.0, 3.0, 0.0, 0.0, 0.0, 0.0, 8.0, 8.0, 15.0, 4.0, 0.0, 0.0, 2.0, 15.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 2.0, 15.0, 5.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 4.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 9.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 5.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 5.0, 15.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 9.0, 0.0, 0.0], [0.0, 0.0, 8.0, 16.0, 13.0, 2.0, 0.0, 0.0, 0.0, 2.0, 16.0, 8.0, 11.0, 14.0, 1.0, 0.0, 0.0, 3.0, 16.0, 1.0, 12.0, 16.0, 5.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 4.0, 2.0, 9.0, 14.0, 0.0, 0.0, 0.0, 4.0, 5.0, 0.0, 8.0, 13.0, 0.0, 0.0, 1.0, 16.0, 11.0, 1.0, 13.0, 7.0, 0.0, 0.0, 0.0, 8.0, 15.0, 16.0, 15.0, 1.0, 0.0], [0.0, 0.0, 3.0, 12.0, 16.0, 13.0, 0.0, 0.0, 0.0, 1.0, 14.0, 9.0, 10.0, 13.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 3.0, 7.0, 15.0, 16.0, 10.0, 0.0, 0.0, 0.0, 16.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 13.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.0, 13.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 12.0, 7.0, 3.0, 13.0, 0.0, 0.0, 0.0, 0.0, 16.0, 0.0, 5.0, 12.0, 0.0, 0.0, 0.0, 0.0, 10.0, 13.0, 14.0, 16.0, 2.0, 0.0, 0.0, 0.0, 1.0, 7.0, 6.0, 13.0, 4.0, 0.0, 0.0, 1.0, 4.0, 0.0, 0.0, 5.0, 11.0, 0.0, 0.0, 2.0, 14.0, 6.0, 2.0, 9.0, 11.0, 0.0, 0.0, 0.0, 4.0, 10.0, 16.0, 16.0, 4.0, 0.0], [0.0, 0.0, 8.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 9.0, 4.0, 0.0, 0.0, 0.0, 5.0, 16.0, 14.0, 11.0, 16.0, 5.0, 0.0, 0.0, 2.0, 16.0, 16.0, 0.0, 12.0, 8.0, 0.0, 0.0, 0.0, 15.0, 15.0, 1.0, 15.0, 6.0, 0.0, 0.0, 0.0, 7.0, 14.0, 16.0, 13.0, 1.0, 0.0], [0.0, 0.0, 3.0, 10.0, 13.0, 7.0, 0.0, 0.0, 0.0, 1.0, 14.0, 13.0, 15.0, 14.0, 0.0, 0.0, 0.0, 0.0, 15.0, 5.0, 14.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 16.0, 3.0, 0.0, 0.0, 4.0, 5.0, 0.0, 2.0, 16.0, 4.0, 0.0, 0.0, 10.0, 16.0, 10.0, 8.0, 16.0, 3.0, 0.0, 0.0, 0.0, 5.0, 12.0, 14.0, 8.0, 0.0, 0.0], [0.0, 0.0, 2.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 15.0, 5.0, 0.0, 0.0, 0.0, 4.0, 16.0, 5.0, 3.0, 13.0, 7.0, 0.0, 0.0, 1.0, 14.0, 9.0, 0.0, 8.0, 13.0, 0.0, 0.0, 0.0, 2.0, 13.0, 16.0, 16.0, 8.0, 0.0], [0.0, 0.0, 0.0, 0.0, 14.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 12.0, 0.0, 0.0, 0.0, 1.0, 10.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 3.0, 12.0, 14.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 1.0, 0.0], [0.0, 1.0, 11.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 8.0, 16.0, 11.0, 7.0, 16.0, 1.0, 0.0, 0.0, 7.0, 11.0, 0.0, 5.0, 16.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 7.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 16.0, 11.0, 6.0, 0.0, 0.0, 0.0, 14.0, 16.0, 13.0, 13.0, 16.0, 5.0], [0.0, 0.0, 15.0, 13.0, 12.0, 12.0, 2.0, 0.0, 0.0, 4.0, 16.0, 16.0, 12.0, 6.0, 0.0, 0.0, 0.0, 0.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 16.0, 4.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 13.0, 15.0, 9.0, 0.0, 0.0, 0.0, 4.0, 14.0, 6.0, 5.0, 16.0, 0.0, 0.0, 0.0, 7.0, 12.0, 2.0, 2.0, 16.0, 0.0, 0.0, 0.0, 4.0, 16.0, 15.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 10.0, 6.0, 8.0, 15.0, 2.0, 0.0, 0.0, 0.0, 11.0, 9.0, 4.0, 13.0, 11.0, 0.0, 0.0, 0.0, 2.0, 14.0, 16.0, 15.0, 6.0, 0.0], [0.0, 0.0, 7.0, 13.0, 8.0, 4.0, 0.0, 0.0, 0.0, 1.0, 15.0, 11.0, 9.0, 15.0, 2.0, 0.0, 0.0, 4.0, 16.0, 6.0, 0.0, 8.0, 7.0, 0.0, 0.0, 4.0, 10.0, 0.0, 0.0, 7.0, 8.0, 0.0, 0.0, 4.0, 10.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 5.0, 12.0, 0.0, 0.0, 12.0, 5.0, 0.0, 0.0, 3.0, 15.0, 5.0, 9.0, 14.0, 2.0, 0.0, 0.0, 0.0, 8.0, 14.0, 12.0, 3.0, 0.0, 0.0], [0.0, 3.0, 16.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 4.0, 14.0, 10.0, 5.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 10.0, 3.0, 0.0, 0.0, 0.0, 4.0, 15.0, 12.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 1.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 13.0, 0.0, 0.0, 3.0, 16.0, 10.0, 7.0, 9.0, 16.0, 0.0, 0.0, 3.0, 13.0, 15.0, 16.0, 16.0, 8.0, 0.0], [0.0, 0.0, 7.0, 11.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 15.0, 15.0, 4.0, 11.0, 4.0, 0.0, 0.0, 3.0, 11.0, 5.0, 0.0, 2.0, 10.0, 0.0, 0.0, 7.0, 8.0, 0.0, 0.0, 3.0, 8.0, 0.0, 0.0, 6.0, 8.0, 0.0, 0.0, 4.0, 8.0, 0.0, 0.0, 5.0, 8.0, 0.0, 0.0, 8.0, 5.0, 0.0, 0.0, 1.0, 12.0, 2.0, 1.0, 13.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 14.0, 3.0, 0.0, 0.0], [0.0, 0.0, 2.0, 13.0, 16.0, 8.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 10.0, 7.0, 0.0, 0.0, 0.0, 5.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 13.0, 1.0, 0.0, 0.0, 0.0, 5.0, 16.0, 6.0, 14.0, 9.0, 0.0, 0.0, 0.0, 0.0, 13.0, 12.0, 14.0, 15.0, 0.0, 0.0, 0.0, 0.0, 3.0, 12.0, 16.0, 11.0, 0.0, 0.0], [0.0, 0.0, 9.0, 14.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 10.0, 6.0, 12.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 13.0, 5.0, 0.0, 0.0, 2.0, 12.0, 15.0, 16.0, 15.0, 14.0, 0.0, 0.0, 2.0, 12.0, 16.0, 7.0, 0.0, 1.0, 0.0, 0.0, 0.0, 3.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 11.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 8.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 12.0, 0.0, 4.0, 13.0, 1.0, 0.0, 6.0, 16.0, 9.0, 7.0, 15.0, 10.0, 0.0, 0.0, 9.0, 16.0, 16.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 4.0, 1.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 13.0, 0.0, 0.0, 0.0], [0.0, 2.0, 6.0, 10.0, 12.0, 1.0, 0.0, 0.0, 0.0, 14.0, 13.0, 10.0, 5.0, 1.0, 0.0, 0.0, 0.0, 10.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 13.0, 12.0, 12.0, 5.0, 0.0, 0.0, 0.0, 2.0, 8.0, 5.0, 7.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 12.0, 0.0, 0.0, 0.0, 2.0, 2.0, 1.0, 10.0, 10.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 14.0, 1.0, 0.0], [0.0, 0.0, 0.0, 2.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 12.0, 0.0, 9.0, 12.0, 0.0, 0.0, 9.0, 16.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 1.0, 6.0, 10.0, 14.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 7.0, 0.0, 0.0], [0.0, 0.0, 5.0, 11.0, 12.0, 5.0, 0.0, 0.0, 0.0, 0.0, 12.0, 6.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 6.0, 12.0, 11.0, 7.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 14.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 10.0, 11.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 7.0, 0.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 1.0, 0.0, 0.0, 4.0, 8.0, 12.0, 15.0, 4.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 12.0, 12.0, 4.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 5.0, 0.0], [0.0, 0.0, 0.0, 9.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 4.0, 0.0, 3.0, 9.0, 0.0, 0.0, 7.0, 14.0, 0.0, 1.0, 14.0, 12.0, 0.0, 0.0, 9.0, 16.0, 12.0, 14.0, 15.0, 1.0, 0.0, 0.0, 0.0, 6.0, 8.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 13.0, 0.0, 0.0, 0.0], [0.0, 0.0, 5.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 13.0, 14.0, 14.0, 15.0, 0.0, 0.0, 0.0, 0.0, 3.0, 11.0, 14.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 16.0, 6.0, 0.0, 0.0, 0.0, 4.0, 15.0, 16.0, 10.0, 0.0, 0.0], [0.0, 0.0, 0.0, 8.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 10.0, 11.0, 0.0, 0.0, 0.0, 0.0, 6.0, 12.0, 11.0, 13.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 1.0, 16.0, 12.0, 11.0, 5.0, 0.0, 0.0, 0.0, 2.0, 13.0, 0.0, 2.0, 9.0, 0.0, 0.0, 0.0, 0.0, 8.0, 6.0, 2.0, 12.0, 0.0, 0.0, 0.0, 0.0, 1.0, 9.0, 14.0, 9.0, 0.0, 0.0], [0.0, 1.0, 13.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 9.0, 8.0, 10.0, 13.0, 0.0, 0.0, 0.0, 0.0, 5.0, 3.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 15.0, 12.0, 12.0, 11.0, 0.0, 0.0, 1.0, 11.0, 13.0, 16.0, 16.0, 12.0, 0.0], [0.0, 0.0, 5.0, 14.0, 14.0, 8.0, 2.0, 0.0, 0.0, 3.0, 15.0, 3.0, 0.0, 13.0, 8.0, 0.0, 0.0, 5.0, 12.0, 0.0, 2.0, 15.0, 8.0, 0.0, 0.0, 2.0, 15.0, 9.0, 14.0, 14.0, 8.0, 0.0, 0.0, 0.0, 1.0, 3.0, 0.0, 12.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 4.0, 0.0, 0.0, 6.0, 15.0, 2.0, 0.0, 14.0, 1.0, 0.0, 0.0, 1.0, 7.0, 14.0, 12.0, 9.0, 0.0, 0.0], [0.0, 0.0, 3.0, 14.0, 10.0, 3.0, 0.0, 0.0, 0.0, 0.0, 10.0, 14.0, 13.0, 15.0, 1.0, 0.0, 0.0, 0.0, 12.0, 4.0, 4.0, 16.0, 4.0, 0.0, 0.0, 0.0, 6.0, 15.0, 15.0, 16.0, 8.0, 0.0, 0.0, 0.0, 1.0, 8.0, 8.0, 14.0, 8.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 9.0, 11.0, 0.0, 0.0, 0.0, 16.0, 10.0, 8.0, 12.0, 12.0, 0.0, 0.0, 0.0, 7.0, 12.0, 14.0, 14.0, 6.0, 0.0], [0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 15.0, 16.0, 10.0, 8.0, 1.0, 0.0, 0.0, 3.0, 16.0, 12.0, 5.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 8.0, 11.0, 2.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 14.0, 2.0, 0.0, 0.0], [0.0, 0.0, 0.0, 6.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 15.0, 1.0, 0.0, 0.0, 0.0, 1.0, 11.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 10.0, 0.0, 10.0, 6.0, 0.0, 0.0, 12.0, 16.0, 8.0, 9.0, 16.0, 12.0, 0.0, 0.0, 2.0, 15.0, 16.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 3.0, 0.0, 0.0], [0.0, 0.0, 3.0, 15.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 10.0, 12.0, 10.0, 16.0, 6.0, 0.0, 0.0, 2.0, 15.0, 2.0, 3.0, 16.0, 1.0, 0.0, 0.0, 0.0, 2.0, 3.0, 10.0, 13.0, 2.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 7.0, 1.0, 0.0, 0.0, 0.0, 1.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 2.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.0, 12.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 12.0, 9.0, 13.0, 16.0, 9.0, 0.0, 0.0, 3.0, 16.0, 5.0, 0.0, 8.0, 12.0, 0.0, 0.0, 0.0, 9.0, 16.0, 10.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 9.0, 16.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 2.0, 12.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 6.0, 0.0, 0.0], [0.0, 0.0, 9.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 14.0, 16.0, 2.0, 0.0, 0.0, 0.0, 3.0, 6.0, 12.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 6.0, 1.0, 3.0, 0.0, 0.0, 0.0, 9.0, 16.0, 13.0, 15.0, 8.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 8.0, 1.0, 0.0], [0.0, 0.0, 11.0, 16.0, 16.0, 16.0, 16.0, 2.0, 0.0, 5.0, 16.0, 16.0, 14.0, 10.0, 4.0, 0.0, 0.0, 5.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 13.0, 1.0, 0.0, 0.0], [0.0, 0.0, 9.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 2.0, 16.0, 11.0, 4.0, 15.0, 2.0, 0.0, 0.0, 4.0, 16.0, 2.0, 2.0, 16.0, 6.0, 0.0, 0.0, 0.0, 13.0, 11.0, 13.0, 16.0, 10.0, 0.0, 0.0, 0.0, 1.0, 10.0, 13.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 14.0, 1.0, 0.0, 0.0, 0.0, 11.0, 15.0, 15.0, 6.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 5.0, 0.0, 0.0, 1.0, 13.0, 15.0, 12.0, 16.0, 1.0, 0.0, 0.0, 4.0, 12.0, 3.0, 10.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 4.0, 0.0], [0.0, 0.0, 7.0, 15.0, 14.0, 8.0, 0.0, 0.0, 0.0, 1.0, 15.0, 7.0, 5.0, 14.0, 5.0, 0.0, 0.0, 0.0, 15.0, 8.0, 0.0, 10.0, 7.0, 0.0, 0.0, 3.0, 16.0, 6.0, 0.0, 12.0, 8.0, 0.0, 0.0, 5.0, 16.0, 2.0, 0.0, 12.0, 8.0, 0.0, 0.0, 4.0, 16.0, 3.0, 1.0, 16.0, 4.0, 0.0, 0.0, 5.0, 16.0, 10.0, 14.0, 12.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 15.0, 2.0, 0.0, 0.0], [0.0, 0.0, 10.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 8.0, 15.0, 7.0, 6.0, 14.0, 0.0, 0.0, 0.0, 1.0, 3.0, 0.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 12.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 12.0, 10.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 15.0, 3.0, 0.0, 0.0], [0.0, 0.0, 5.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 6.0, 11.0, 6.0, 0.0, 0.0, 0.0, 7.0, 9.0, 0.0, 0.0, 14.0, 0.0, 0.0, 0.0, 5.0, 9.0, 0.0, 0.0, 8.0, 6.0, 0.0, 0.0, 4.0, 13.0, 0.0, 0.0, 4.0, 8.0, 0.0, 0.0, 1.0, 16.0, 0.0, 0.0, 4.0, 11.0, 0.0, 0.0, 0.0, 15.0, 7.0, 5.0, 16.0, 4.0, 0.0, 0.0, 0.0, 2.0, 15.0, 15.0, 5.0, 0.0, 0.0], [0.0, 0.0, 7.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 2.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 14.0, 8.0, 10.0, 0.0, 0.0, 0.0, 7.0, 12.0, 12.0, 12.0, 15.0, 2.0], [0.0, 0.0, 2.0, 15.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 14.0, 3.0, 0.0, 0.0, 0.0, 4.0, 12.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 11.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 12.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 5.0, 15.0, 13.0, 12.0, 4.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 3.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 1.0, 12.0, 16.0, 16.0, 8.0, 0.0, 0.0, 5.0, 14.0, 16.0, 16.0, 16.0, 5.0, 0.0, 0.0, 1.0, 4.0, 7.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 0.0, 0.0], [0.0, 1.0, 12.0, 16.0, 13.0, 7.0, 0.0, 0.0, 0.0, 12.0, 11.0, 4.0, 4.0, 15.0, 0.0, 0.0, 0.0, 8.0, 3.0, 0.0, 6.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 5.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 10.0, 0.0, 0.0, 0.0, 10.0, 10.0, 13.0, 14.0, 1.0, 0.0], [0.0, 0.0, 3.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 2.0, 5.0, 0.0, 0.0, 0.0, 2.0, 16.0, 6.0, 10.0, 15.0, 1.0, 0.0, 0.0, 9.0, 15.0, 3.0, 16.0, 11.0, 7.0, 0.0, 0.0, 12.0, 16.0, 16.0, 15.0, 11.0, 5.0, 0.0, 0.0, 3.0, 9.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 7.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 8.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 15.0, 5.0, 0.0, 0.0, 0.0, 14.0, 11.0, 0.0, 1.0, 15.0, 0.0, 0.0, 0.0, 6.0, 11.0, 1.0, 3.0, 14.0, 2.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 7.0, 0.0], [0.0, 0.0, 0.0, 1.0, 8.0, 13.0, 14.0, 2.0, 0.0, 0.0, 2.0, 13.0, 9.0, 4.0, 14.0, 4.0, 0.0, 0.0, 13.0, 9.0, 0.0, 9.0, 14.0, 1.0, 0.0, 4.0, 16.0, 14.0, 14.0, 16.0, 6.0, 0.0, 0.0, 1.0, 11.0, 10.0, 7.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 3.0, 0.0, 0.0], [0.0, 0.0, 10.0, 16.0, 13.0, 5.0, 0.0, 0.0, 0.0, 7.0, 13.0, 5.0, 14.0, 12.0, 0.0, 0.0, 0.0, 9.0, 10.0, 0.0, 13.0, 14.0, 0.0, 0.0, 0.0, 4.0, 15.0, 13.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 2.0, 6.0, 3.0, 12.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 8.0, 0.0, 0.0, 0.0, 12.0, 6.0, 5.0, 15.0, 4.0, 0.0, 0.0, 0.0, 9.0, 13.0, 16.0, 8.0, 0.0, 0.0], [0.0, 0.0, 6.0, 9.0, 11.0, 9.0, 0.0, 0.0, 0.0, 13.0, 16.0, 15.0, 15.0, 15.0, 0.0, 0.0, 0.0, 4.0, 5.0, 2.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 12.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 16.0, 9.0, 0.0, 0.0, 0.0, 3.0, 13.0, 12.0, 7.0, 1.0, 0.0], [0.0, 0.0, 2.0, 13.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 9.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 14.0, 12.0, 13.0, 4.0, 0.0, 0.0, 0.0, 14.0, 10.0, 0.0, 10.0, 15.0, 0.0, 0.0, 0.0, 2.0, 12.0, 16.0, 13.0, 7.0, 0.0], [0.0, 0.0, 7.0, 14.0, 11.0, 1.0, 0.0, 0.0, 0.0, 6.0, 15.0, 6.0, 7.0, 10.0, 0.0, 0.0, 0.0, 11.0, 7.0, 0.0, 2.0, 12.0, 0.0, 0.0, 0.0, 5.0, 4.0, 0.0, 1.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 13.0, 2.0, 3.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 16.0, 6.0], [0.0, 0.0, 1.0, 15.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 16.0, 7.0, 0.0, 0.0], [0.0, 0.0, 2.0, 11.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 12.0, 6.0, 11.0, 9.0, 0.0, 0.0, 0.0, 4.0, 11.0, 0.0, 7.0, 16.0, 0.0, 0.0, 0.0, 5.0, 6.0, 0.0, 1.0, 16.0, 6.0, 0.0, 0.0, 5.0, 4.0, 0.0, 0.0, 10.0, 7.0, 0.0, 0.0, 0.0, 10.0, 0.0, 0.0, 10.0, 5.0, 0.0, 0.0, 0.0, 13.0, 2.0, 6.0, 12.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 12.0, 1.0, 0.0, 0.0], [0.0, 2.0, 9.0, 16.0, 13.0, 13.0, 2.0, 0.0, 0.0, 11.0, 11.0, 4.0, 2.0, 10.0, 4.0, 0.0, 0.0, 6.0, 12.0, 2.0, 4.0, 12.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 7.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 5.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 14.0, 3.0, 0.0, 0.0, 0.0], [0.0, 1.0, 12.0, 16.0, 14.0, 8.0, 0.0, 0.0, 0.0, 4.0, 16.0, 8.0, 10.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 15.0, 6.0, 10.0, 11.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 14.0, 8.0, 1.0], [0.0, 0.0, 1.0, 9.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 14.0, 11.0, 8.0, 16.0, 8.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 3.0, 0.0, 0.0, 0.0, 6.0, 12.0, 16.0, 16.0, 9.0, 0.0, 0.0, 1.0, 16.0, 14.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 2.0, 8.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 7.0, 0.0, 0.0, 0.0], [0.0, 1.0, 11.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 10.0, 14.0, 11.0, 16.0, 0.0, 0.0, 0.0, 0.0, 14.0, 5.0, 6.0, 15.0, 0.0, 0.0, 0.0, 0.0, 3.0, 1.0, 11.0, 14.0, 3.0, 1.0, 0.0, 0.0, 2.0, 13.0, 16.0, 16.0, 16.0, 9.0, 0.0, 0.0, 2.0, 14.0, 16.0, 5.0, 4.0, 2.0, 0.0, 0.0, 0.0, 11.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 12.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 6.0, 0.0, 0.0, 0.0, 7.0, 15.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 10.0, 0.0, 0.0], [0.0, 1.0, 11.0, 15.0, 16.0, 9.0, 0.0, 0.0, 0.0, 3.0, 16.0, 10.0, 10.0, 16.0, 1.0, 0.0, 0.0, 0.0, 2.0, 1.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 8.0, 0.0, 0.0, 0.0, 6.0, 8.0, 13.0, 15.0, 5.0, 0.0, 0.0, 0.0, 15.0, 16.0, 12.0, 5.0, 0.0, 0.0], [0.0, 1.0, 10.0, 12.0, 12.0, 11.0, 0.0, 0.0, 0.0, 7.0, 14.0, 8.0, 8.0, 6.0, 0.0, 0.0, 0.0, 7.0, 11.0, 7.0, 3.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 13.0, 13.0, 8.0, 0.0, 0.0, 0.0, 1.0, 3.0, 0.0, 1.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 12.0, 0.0, 0.0, 0.0, 11.0, 3.0, 0.0, 10.0, 12.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 14.0, 4.0, 0.0], [0.0, 0.0, 8.0, 14.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 14.0, 13.0, 8.0, 8.0, 0.0, 0.0, 0.0, 2.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 13.0, 16.0, 13.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 12.0, 16.0, 7.0, 0.0, 0.0, 0.0, 4.0, 1.0, 2.0, 14.0, 6.0, 0.0, 0.0, 0.0, 1.0, 6.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 8.0, 1.0, 0.0, 0.0], [0.0, 0.0, 5.0, 14.0, 10.0, 7.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 16.0, 16.0, 3.0, 0.0, 0.0, 3.0, 16.0, 10.0, 2.0, 16.0, 7.0, 0.0, 0.0, 7.0, 16.0, 3.0, 0.0, 12.0, 8.0, 0.0, 0.0, 8.0, 16.0, 1.0, 0.0, 12.0, 8.0, 0.0, 0.0, 7.0, 16.0, 5.0, 2.0, 16.0, 4.0, 0.0, 0.0, 2.0, 16.0, 15.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 13.0, 2.0, 0.0, 0.0], [0.0, 0.0, 9.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 7.0, 15.0, 7.0, 16.0, 7.0, 0.0, 0.0, 0.0, 2.0, 2.0, 0.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 3.0, 3.0, 5.0, 0.0, 0.0, 0.0, 11.0, 14.0, 10.0, 16.0, 6.0, 0.0, 0.0, 0.0, 11.0, 16.0, 13.0, 5.0, 0.0, 0.0], [0.0, 0.0, 5.0, 12.0, 13.0, 4.0, 0.0, 0.0, 0.0, 3.0, 16.0, 10.0, 2.0, 5.0, 9.0, 0.0, 0.0, 0.0, 15.0, 14.0, 11.0, 15.0, 3.0, 0.0, 0.0, 0.0, 7.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 13.0, 12.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 13.0, 11.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 11.0, 2.0, 0.0, 0.0], [0.0, 0.0, 0.0, 5.0, 14.0, 12.0, 2.0, 0.0, 0.0, 0.0, 7.0, 15.0, 8.0, 14.0, 4.0, 0.0, 0.0, 0.0, 6.0, 2.0, 3.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 9.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 16.0, 11.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 12.0, 13.0, 3.0, 0.0], [0.0, 0.0, 2.0, 14.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 12.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 9.0, 15.0, 12.0, 5.0, 0.0, 0.0, 0.0, 12.0, 11.0, 11.0, 12.0, 11.0, 0.0, 0.0, 1.0, 16.0, 1.0, 0.0, 8.0, 11.0, 0.0, 0.0, 3.0, 15.0, 12.0, 10.0, 15.0, 2.0, 0.0, 0.0, 0.0, 1.0, 11.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 13.0, 2.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 4.0, 16.0, 12.0, 16.0, 0.0, 0.0, 0.0, 4.0, 15.0, 6.0, 7.0, 13.0, 0.0, 0.0, 0.0, 11.0, 15.0, 15.0, 16.0, 16.0, 9.0, 0.0, 0.0, 9.0, 13.0, 12.0, 13.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0], [0.0, 2.0, 13.0, 16.0, 16.0, 16.0, 11.0, 0.0, 0.0, 5.0, 16.0, 10.0, 5.0, 4.0, 1.0, 0.0, 0.0, 6.0, 16.0, 7.0, 3.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 3.0, 8.0, 4.0, 11.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 15.0, 0.0, 0.0, 0.0, 0.0, 4.0, 13.0, 16.0, 6.0, 0.0, 0.0, 0.0, 2.0, 16.0, 15.0, 8.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 3.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 2.0, 16.0, 14.0, 16.0, 5.0, 1.0, 0.0, 0.0, 11.0, 16.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 5.0, 8.0, 11.0, 16.0, 4.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 2.0, 0.0, 0.0], [0.0, 0.0, 5.0, 4.0, 9.0, 10.0, 0.0, 0.0, 0.0, 0.0, 10.0, 8.0, 11.0, 16.0, 2.0, 0.0, 0.0, 0.0, 8.0, 12.0, 14.0, 14.0, 1.0, 0.0, 0.0, 0.0, 5.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 5.0, 0.0, 0.0, 0.0], [0.0, 1.0, 12.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 7.0, 11.0, 7.0, 14.0, 1.0, 0.0, 0.0, 0.0, 2.0, 2.0, 3.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 5.0, 15.0, 6.0, 0.0, 0.0, 1.0, 11.0, 4.0, 4.0, 13.0, 8.0, 0.0, 0.0, 2.0, 14.0, 16.0, 16.0, 13.0, 1.0, 0.0], [0.0, 0.0, 5.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 8.0, 15.0, 9.0, 1.0, 0.0, 0.0, 4.0, 16.0, 16.0, 12.0, 15.0, 11.0, 0.0, 0.0, 1.0, 15.0, 14.0, 4.0, 14.0, 11.0, 0.0, 0.0, 0.0, 5.0, 14.0, 14.0, 10.0, 1.0, 0.0], [0.0, 2.0, 8.0, 15.0, 16.0, 10.0, 0.0, 0.0, 0.0, 11.0, 16.0, 10.0, 4.0, 15.0, 1.0, 0.0, 0.0, 12.0, 13.0, 0.0, 0.0, 13.0, 5.0, 0.0, 0.0, 11.0, 13.0, 0.0, 0.0, 15.0, 7.0, 0.0, 0.0, 8.0, 16.0, 0.0, 0.0, 15.0, 6.0, 0.0, 0.0, 6.0, 16.0, 1.0, 5.0, 16.0, 2.0, 0.0, 0.0, 2.0, 16.0, 6.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 14.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 11.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 5.0, 16.0, 10.0, 16.0, 4.0, 0.0, 0.0, 2.0, 15.0, 10.0, 0.0, 8.0, 1.0, 0.0, 0.0, 5.0, 16.0, 9.0, 1.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 10.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 1.0, 8.0, 13.0, 0.0, 0.0, 0.0], [0.0, 0.0, 6.0, 15.0, 8.0, 10.0, 12.0, 2.0, 0.0, 0.0, 10.0, 14.0, 10.0, 12.0, 16.0, 1.0, 0.0, 0.0, 9.0, 10.0, 1.0, 13.0, 7.0, 0.0, 0.0, 0.0, 4.0, 4.0, 8.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 10.0, 12.0, 2.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 13.0, 10.0, 0.0, 0.0, 0.0, 0.0, 9.0, 7.0, 2.0, 15.0, 0.0, 0.0, 0.0, 0.0, 3.0, 2.0, 3.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 2.0, 7.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 16.0, 16.0, 3.0, 0.0, 0.0, 3.0, 13.0, 9.0, 8.0, 4.0, 0.0], [0.0, 0.0, 0.0, 13.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 13.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 14.0, 15.0, 8.0, 0.0, 0.0, 0.0, 1.0, 16.0, 15.0, 8.0, 13.0, 9.0, 0.0, 0.0, 0.0, 11.0, 10.0, 0.0, 11.0, 16.0, 0.0, 0.0, 0.0, 2.0, 14.0, 16.0, 16.0, 13.0, 0.0], [0.0, 5.0, 16.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 4.0, 10.0, 4.0, 4.0, 4.0, 0.0, 0.0, 0.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 13.0, 7.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 8.0, 14.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 7.0, 0.0, 0.0, 3.0, 3.0, 2.0, 11.0, 15.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 12.0, 1.0, 0.0, 0.0], [0.0, 0.0, 13.0, 16.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 9.0, 8.0, 13.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 0.0, 0.0, 0.0, 0.0, 3.0, 6.0, 15.0, 16.0, 7.0, 0.0, 0.0, 0.0, 15.0, 16.0, 16.0, 11.0, 1.0, 0.0], [0.0, 3.0, 16.0, 9.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 13.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 14.0, 10.0, 6.0, 0.0, 0.0, 0.0, 2.0, 15.0, 16.0, 10.0, 2.0, 0.0, 0.0, 0.0, 6.0, 14.0, 15.0, 1.0, 0.0, 0.0, 0.0, 3.0, 15.0, 4.0, 9.0, 7.0, 0.0, 0.0, 0.0, 6.0, 13.0, 1.0, 10.0, 9.0, 0.0, 0.0, 0.0, 2.0, 11.0, 12.0, 14.0, 4.0, 0.0, 0.0], [0.0, 0.0, 0.0, 3.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 5.0, 3.0, 0.0, 0.0, 0.0, 3.0, 15.0, 11.0, 5.0, 16.0, 2.0, 0.0, 0.0, 5.0, 16.0, 11.0, 11.0, 16.0, 6.0, 0.0, 0.0, 0.0, 6.0, 12.0, 16.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 7.0, 0.0, 0.0], [0.0, 0.0, 0.0, 10.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 0.0, 0.0, 9.0, 5.0, 0.0, 0.0, 14.0, 10.0, 0.0, 7.0, 16.0, 4.0, 0.0, 5.0, 16.0, 7.0, 5.0, 16.0, 6.0, 0.0, 0.0, 11.0, 16.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 3.0, 4.0, 11.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 6.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 13.0, 1.0, 2.0, 1.0, 0.0, 0.0, 7.0, 16.0, 5.0, 1.0, 14.0, 10.0, 0.0, 0.0, 12.0, 16.0, 8.0, 12.0, 16.0, 2.0, 0.0, 0.0, 2.0, 12.0, 15.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 0.0, 0.0, 0.0], [0.0, 0.0, 5.0, 14.0, 9.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 13.0, 16.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 10.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 2.0, 4.0, 5.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 7.0, 0.0, 0.0, 0.0, 11.0, 8.0, 8.0, 16.0, 4.0, 0.0, 0.0, 0.0, 8.0, 13.0, 15.0, 10.0, 0.0, 0.0], [0.0, 0.0, 2.0, 13.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 6.0, 2.0, 0.0, 0.0, 0.0, 2.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 9.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 13.0, 2.0, 0.0, 0.0, 0.0, 1.0, 16.0, 6.0, 8.0, 14.0, 0.0, 0.0, 0.0, 0.0, 11.0, 10.0, 1.0, 16.0, 5.0, 0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 16.0, 3.0, 0.0], [0.0, 1.0, 12.0, 15.0, 10.0, 2.0, 0.0, 0.0, 0.0, 4.0, 14.0, 1.0, 6.0, 12.0, 2.0, 0.0, 0.0, 7.0, 15.0, 0.0, 1.0, 14.0, 4.0, 0.0, 0.0, 3.0, 15.0, 12.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 6.0, 0.0, 0.0, 0.0, 0.0, 11.0, 12.0, 13.0, 4.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 4.0, 16.0, 7.0, 8.0, 16.0, 4.0, 0.0, 0.0, 1.0, 4.0, 0.0, 10.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 1.0, 0.0], [0.0, 0.0, 6.0, 15.0, 15.0, 3.0, 0.0, 0.0, 0.0, 3.0, 16.0, 14.0, 14.0, 13.0, 0.0, 0.0, 0.0, 6.0, 15.0, 2.0, 1.0, 14.0, 5.0, 0.0, 0.0, 8.0, 14.0, 2.0, 0.0, 9.0, 8.0, 0.0, 0.0, 8.0, 16.0, 4.0, 0.0, 8.0, 8.0, 0.0, 0.0, 5.0, 16.0, 6.0, 0.0, 11.0, 9.0, 0.0, 0.0, 1.0, 16.0, 16.0, 14.0, 16.0, 9.0, 0.0, 0.0, 0.0, 5.0, 14.0, 15.0, 10.0, 1.0, 0.0], [0.0, 0.0, 0.0, 5.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 2.0, 15.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 4.0, 8.0, 12.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 11.0, 2.0, 0.0], [0.0, 0.0, 2.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 10.0, 16.0, 1.0, 0.0, 0.0, 0.0, 11.0, 8.0, 2.0, 15.0, 0.0, 0.0, 0.0, 0.0, 9.0, 8.0, 1.0, 13.0, 0.0, 0.0, 0.0, 0.0, 1.0, 3.0, 6.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 9.0, 4.0, 7.0, 3.0, 0.0, 0.0, 1.0, 14.0, 16.0, 16.0, 13.0, 8.0], [0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 1.0, 0.0, 0.0, 0.0, 7.0, 15.0, 16.0, 14.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 8.0, 2.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 13.0, 1.0, 0.0], [0.0, 0.0, 3.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 14.0, 16.0, 13.0, 2.0, 0.0, 0.0, 3.0, 16.0, 11.0, 3.0, 7.0, 12.0, 0.0, 0.0, 0.0, 13.0, 6.0, 3.0, 8.0, 14.0, 0.0, 0.0, 0.0, 4.0, 14.0, 16.0, 14.0, 7.0, 0.0], [0.0, 0.0, 10.0, 16.0, 15.0, 8.0, 2.0, 0.0, 0.0, 9.0, 16.0, 12.0, 8.0, 9.0, 3.0, 0.0, 0.0, 13.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 7.0, 13.0, 8.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 2.0, 12.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 6.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 5.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 11.0, 12.0, 0.0, 0.0, 0.0, 3.0, 15.0, 3.0, 11.0, 10.0, 0.0, 0.0, 0.0, 8.0, 11.0, 0.0, 13.0, 10.0, 2.0, 0.0, 0.0, 10.0, 16.0, 16.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 4.0, 10.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 9.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.0, 11.0, 16.0, 15.0, 2.0, 0.0, 0.0, 4.0, 16.0, 10.0, 4.0, 16.0, 4.0, 0.0, 0.0, 7.0, 6.0, 0.0, 5.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 6.0, 1.0, 0.0, 0.0, 0.0, 0.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 2.0, 0.0, 0.0, 0.0], [0.0, 4.0, 15.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 16.0, 14.0, 6.0, 0.0, 0.0, 6.0, 16.0, 16.0, 15.0, 11.0, 3.0, 0.0, 0.0, 7.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 12.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 12.0, 14.0, 0.0, 0.0, 0.0, 0.0, 7.0, 5.0, 1.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 12.0, 8.0, 5.0, 0.0, 0.0, 0.0, 9.0, 8.0, 13.0, 15.0, 7.0, 0.0], [0.0, 0.0, 0.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 3.0, 2.0, 0.0, 0.0, 0.0, 0.0, 13.0, 12.0, 8.0, 12.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 15.0, 16.0, 13.0, 4.0, 0.0, 0.0, 4.0, 9.0, 14.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 13.0, 0.0, 0.0, 0.0], [0.0, 0.0, 13.0, 16.0, 16.0, 16.0, 13.0, 0.0, 0.0, 4.0, 16.0, 9.0, 8.0, 5.0, 4.0, 0.0, 0.0, 9.0, 15.0, 7.0, 8.0, 2.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 14.0, 15.0, 1.0, 0.0, 0.0, 1.0, 3.0, 0.0, 4.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 0.0, 0.0, 0.0, 0.0, 11.0, 8.0, 16.0, 6.0, 0.0, 0.0, 0.0, 1.0, 15.0, 16.0, 10.0, 0.0, 0.0, 0.0], [0.0, 4.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 12.0, 10.0, 5.0, 0.0, 0.0, 3.0, 16.0, 16.0, 16.0, 16.0, 8.0, 0.0], [0.0, 0.0, 0.0, 12.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 13.0, 10.0, 0.0, 0.0, 0.0, 0.0, 8.0, 9.0, 1.0, 12.0, 0.0, 0.0, 0.0, 2.0, 16.0, 9.0, 0.0, 10.0, 5.0, 0.0, 0.0, 4.0, 16.0, 8.0, 0.0, 7.0, 8.0, 0.0, 0.0, 1.0, 16.0, 3.0, 0.0, 10.0, 7.0, 0.0, 0.0, 0.0, 8.0, 13.0, 9.0, 16.0, 6.0, 0.0, 0.0, 0.0, 1.0, 10.0, 16.0, 13.0, 1.0, 0.0], [0.0, 0.0, 1.0, 14.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 13.0, 15.0, 10.0, 0.0, 0.0, 0.0, 8.0, 16.0, 2.0, 3.0, 14.0, 5.0, 0.0, 0.0, 5.0, 16.0, 4.0, 0.0, 12.0, 6.0, 0.0, 0.0, 0.0, 10.0, 13.0, 2.0, 14.0, 6.0, 0.0, 0.0, 0.0, 2.0, 12.0, 16.0, 11.0, 1.0, 0.0], [0.0, 0.0, 0.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 14.0, 15.0, 1.0, 11.0, 0.0, 0.0, 0.0, 0.0, 16.0, 15.0, 0.0, 14.0, 1.0, 0.0, 0.0, 1.0, 16.0, 10.0, 0.0, 14.0, 2.0, 0.0, 0.0, 0.0, 15.0, 13.0, 3.0, 15.0, 3.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 8.0, 0.0, 0.0], [0.0, 0.0, 2.0, 12.0, 16.0, 10.0, 0.0, 0.0, 0.0, 2.0, 15.0, 10.0, 6.0, 15.0, 0.0, 0.0, 0.0, 2.0, 7.0, 1.0, 4.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 15.0, 10.0, 5.0, 0.0, 0.0, 0.0, 1.0, 8.0, 8.0, 11.0, 16.0, 6.0], [0.0, 3.0, 12.0, 15.0, 16.0, 16.0, 3.0, 0.0, 0.0, 6.0, 16.0, 9.0, 9.0, 16.0, 6.0, 0.0, 0.0, 0.0, 3.0, 0.0, 11.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 11.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 15.0, 14.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 15.0, 6.0, 0.0, 0.0], [0.0, 0.0, 0.0, 12.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 1.0, 4.0, 2.0, 0.0, 0.0, 0.0, 4.0, 16.0, 15.0, 12.0, 15.0, 5.0, 0.0, 0.0, 3.0, 16.0, 6.0, 0.0, 5.0, 11.0, 0.0, 0.0, 0.0, 9.0, 11.0, 4.0, 13.0, 5.0, 0.0, 0.0, 0.0, 1.0, 11.0, 16.0, 9.0, 0.0, 0.0], [0.0, 0.0, 9.0, 14.0, 14.0, 3.0, 0.0, 0.0, 0.0, 4.0, 13.0, 1.0, 4.0, 11.0, 0.0, 0.0, 0.0, 1.0, 14.0, 1.0, 11.0, 7.0, 6.0, 0.0, 0.0, 0.0, 15.0, 13.0, 9.0, 12.0, 3.0, 0.0, 0.0, 0.0, 1.0, 14.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 1.0, 9.0, 5.0, 0.0, 0.0, 0.0, 0.0, 11.0, 12.0, 13.0, 3.0, 0.0, 0.0], [0.0, 0.0, 0.0, 2.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 10.0, 0.0, 3.0, 8.0, 0.0, 0.0, 5.0, 16.0, 1.0, 2.0, 15.0, 5.0, 0.0, 0.0, 9.0, 13.0, 7.0, 14.0, 16.0, 2.0, 0.0, 0.0, 5.0, 15.0, 14.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 5.0, 0.0, 0.0], [0.0, 0.0, 0.0, 4.0, 11.0, 15.0, 16.0, 12.0, 0.0, 0.0, 2.0, 16.0, 12.0, 9.0, 11.0, 12.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 14.0, 5.0, 0.0, 0.0, 0.0, 7.0, 12.0, 14.0, 15.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 10.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 11.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 5.0, 11.0, 9.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 6.0, 0.0, 0.0, 0.0, 6.0, 9.0, 12.0, 16.0, 9.0, 0.0, 0.0, 1.0, 16.0, 16.0, 16.0, 14.0, 3.0, 0.0, 0.0, 0.0, 3.0, 5.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 5.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 12.0, 16.0, 6.0, 0.0, 0.0, 0.0, 4.0, 16.0, 10.0, 5.0, 16.0, 4.0, 0.0, 0.0, 8.0, 13.0, 0.0, 5.0, 15.0, 5.0, 0.0, 0.0, 6.0, 12.0, 7.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 11.0, 10.0, 9.0, 11.0, 0.0, 0.0, 0.0, 0.0, 12.0, 6.0, 0.0, 13.0, 3.0, 0.0, 0.0, 0.0, 6.0, 13.0, 13.0, 8.0, 0.0, 0.0], [0.0, 0.0, 7.0, 16.0, 13.0, 4.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 12.0, 14.0, 8.0, 0.0, 0.0], [0.0, 0.0, 5.0, 13.0, 14.0, 5.0, 0.0, 0.0, 0.0, 2.0, 15.0, 6.0, 11.0, 15.0, 1.0, 0.0, 0.0, 1.0, 16.0, 5.0, 8.0, 16.0, 4.0, 0.0, 0.0, 0.0, 4.0, 12.0, 9.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 7.0, 0.0, 0.0, 0.0, 7.0, 7.0, 5.0, 15.0, 2.0, 0.0, 0.0, 0.0, 5.0, 15.0, 16.0, 7.0, 0.0, 0.0], [0.0, 0.0, 0.0, 3.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 2.0, 3.0, 0.0, 0.0, 0.0, 5.0, 16.0, 4.0, 6.0, 16.0, 1.0, 0.0, 0.0, 10.0, 15.0, 4.0, 9.0, 16.0, 2.0, 0.0, 0.0, 12.0, 16.0, 16.0, 16.0, 13.0, 2.0, 0.0, 0.0, 1.0, 4.0, 7.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 5.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 8.0, 4.0, 6.0, 2.0, 0.0, 0.0, 0.0, 11.0, 16.0, 13.0, 12.0, 13.0, 0.0, 0.0, 0.0, 12.0, 14.0, 4.0, 5.0, 16.0, 2.0, 0.0, 0.0, 1.0, 8.0, 16.0, 13.0, 9.0, 1.0], [0.0, 0.0, 0.0, 12.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 13.0, 11.0, 12.0, 2.0, 0.0, 0.0, 12.0, 16.0, 10.0, 15.0, 16.0, 9.0, 0.0, 0.0, 4.0, 14.0, 16.0, 16.0, 12.0, 4.0, 0.0, 0.0, 0.0, 0.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 13.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 12.0, 16.0, 10.0, 0.0, 0.0, 0.0, 3.0, 15.0, 10.0, 7.0, 16.0, 4.0, 0.0, 0.0, 9.0, 8.0, 0.0, 11.0, 10.0, 0.0, 0.0, 0.0, 3.0, 15.0, 11.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 7.0, 13.0, 4.0, 0.0, 0.0, 0.0, 0.0, 9.0, 7.0, 6.0, 10.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 16.0, 5.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 12.0, 0.0, 12.0, 7.0, 0.0, 0.0, 0.0, 12.0, 14.0, 6.0, 16.0, 14.0, 1.0, 0.0, 0.0, 6.0, 16.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 0.0, 0.0, 0.0], [0.0, 0.0, 12.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 5.0, 16.0, 10.0, 15.0, 8.0, 0.0, 0.0, 0.0, 1.0, 7.0, 3.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 13.0, 16.0, 6.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 15.0, 11.0, 0.0, 0.0, 2.0, 16.0, 10.0, 11.0, 16.0, 7.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 10.0, 1.0, 0.0], [0.0, 0.0, 6.0, 10.0, 8.0, 3.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 4.0, 9.0, 12.0, 11.0, 2.0, 0.0], [0.0, 0.0, 7.0, 14.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 14.0, 14.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 12.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 14.0, 9.0, 2.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 16.0, 7.0, 0.0, 0.0, 6.0, 12.0, 16.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 13.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 6.0, 15.0, 12.0, 1.0, 0.0, 0.0, 4.0, 12.0, 16.0, 12.0, 16.0, 3.0, 0.0, 0.0, 15.0, 16.0, 6.0, 4.0, 16.0, 3.0, 0.0, 0.0, 4.0, 5.0, 1.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 10.0, 1.0, 0.0, 0.0, 0.0, 3.0, 2.0, 4.0, 15.0, 7.0, 0.0, 0.0, 0.0, 12.0, 15.0, 8.0, 11.0, 14.0, 0.0, 0.0, 0.0, 1.0, 8.0, 15.0, 16.0, 11.0, 0.0], [0.0, 0.0, 9.0, 9.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 15.0, 14.0, 12.0, 0.0, 0.0, 0.0, 3.0, 10.0, 1.0, 0.0, 12.0, 5.0, 0.0, 0.0, 5.0, 8.0, 0.0, 0.0, 8.0, 6.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 5.0, 8.0, 0.0, 0.0, 10.0, 6.0, 0.0, 0.0, 4.0, 13.0, 4.0, 6.0, 13.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 14.0, 3.0, 0.0, 0.0], [0.0, 1.0, 13.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 3.0, 12.0, 6.0, 11.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 12.0, 0.0, 0.0, 0.0, 1.0, 9.0, 15.0, 16.0, 16.0, 9.0, 0.0, 0.0, 2.0, 12.0, 15.0, 14.0, 5.0, 5.0, 0.0, 0.0, 0.0, 5.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 7.0, 7.0, 13.0, 16.0, 4.0, 0.0, 0.0, 0.0, 13.0, 13.0, 6.0, 12.0, 7.0, 0.0, 0.0, 0.0, 10.0, 4.0, 10.0, 11.0, 1.0, 0.0, 0.0, 0.0, 8.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 8.0, 11.0, 5.0, 0.0, 0.0, 0.0, 0.0, 4.0, 10.0, 9.0, 8.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 16.0, 6.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 13.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 11.0, 0.0, 0.0, 0.0, 0.0, 3.0, 4.0, 4.0, 16.0, 2.0, 0.0, 0.0, 2.0, 15.0, 13.0, 14.0, 13.0, 2.0], [0.0, 1.0, 7.0, 15.0, 16.0, 9.0, 0.0, 0.0, 1.0, 13.0, 14.0, 7.0, 14.0, 14.0, 0.0, 0.0, 0.0, 6.0, 1.0, 8.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 6.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 11.0, 0.0, 0.0, 0.0, 5.0, 1.0, 0.0, 11.0, 16.0, 2.0, 0.0, 0.0, 10.0, 16.0, 16.0, 16.0, 7.0, 0.0], [0.0, 0.0, 0.0, 5.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 9.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 16.0, 8.0, 0.0], [0.0, 1.0, 8.0, 12.0, 15.0, 14.0, 4.0, 0.0, 0.0, 3.0, 11.0, 8.0, 8.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 7.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 13.0, 0.0, 0.0, 0.0, 0.0, 7.0, 8.0, 14.0, 15.0, 0.0, 0.0, 0.0, 0.0, 14.0, 15.0, 11.0, 2.0, 0.0, 0.0], [0.0, 0.0, 0.0, 6.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 9.0, 15.0, 3.0, 0.0, 0.0, 1.0, 15.0, 7.0, 5.0, 15.0, 0.0, 0.0, 0.0, 9.0, 16.0, 4.0, 11.0, 14.0, 10.0, 0.0, 0.0, 9.0, 16.0, 16.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 2.0, 4.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 10.0, 0.0, 0.0, 0.0], [0.0, 0.0, 6.0, 12.0, 13.0, 6.0, 0.0, 0.0, 0.0, 6.0, 16.0, 9.0, 12.0, 16.0, 2.0, 0.0, 0.0, 7.0, 16.0, 9.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 11.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 9.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 9.0, 9.0, 9.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 12.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 4.0, 11.0, 15.0, 15.0, 2.0, 0.0, 1.0, 10.0, 16.0, 13.0, 14.0, 14.0, 2.0, 0.0, 5.0, 16.0, 16.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 9.0, 8.0, 8.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 2.0, 0.0, 0.0], [0.0, 0.0, 0.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 12.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 13.0, 0.0, 0.0, 0.0], [0.0, 2.0, 10.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 12.0, 14.0, 12.0, 16.0, 5.0, 0.0, 0.0, 0.0, 2.0, 0.0, 4.0, 16.0, 7.0, 1.0, 0.0, 0.0, 0.0, 4.0, 15.0, 16.0, 16.0, 10.0, 0.0, 0.0, 1.0, 16.0, 16.0, 12.0, 5.0, 2.0, 0.0, 0.0, 0.0, 15.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 6.0, 12.0, 13.0, 5.0, 0.0, 0.0, 0.0, 2.0, 16.0, 9.0, 8.0, 15.0, 2.0, 0.0, 0.0, 8.0, 12.0, 0.0, 3.0, 15.0, 8.0, 0.0, 0.0, 4.0, 15.0, 12.0, 16.0, 13.0, 1.0, 0.0, 0.0, 0.0, 2.0, 2.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 14.0, 0.0, 0.0, 0.0], [0.0, 1.0, 8.0, 11.0, 13.0, 12.0, 0.0, 0.0, 0.0, 5.0, 9.0, 0.0, 4.0, 16.0, 1.0, 0.0, 0.0, 7.0, 5.0, 0.0, 5.0, 16.0, 5.0, 0.0, 0.0, 1.0, 13.0, 11.0, 13.0, 6.0, 8.0, 0.0, 0.0, 0.0, 3.0, 4.0, 1.0, 4.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 5.0, 0.0, 0.0, 2.0, 8.0, 1.0, 2.0, 14.0, 0.0, 0.0, 0.0, 0.0, 8.0, 13.0, 11.0, 3.0, 0.0, 0.0], [0.0, 0.0, 9.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 13.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 7.0, 9.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 8.0, 6.0, 0.0, 0.0, 0.0, 9.0, 15.0, 12.0, 16.0, 16.0, 9.0], [0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 12.0, 13.0, 5.0, 1.0, 0.0, 0.0, 0.0, 0.0, 15.0, 7.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 5.0, 16.0, 4.0, 0.0, 0.0, 0.0, 5.0, 3.0, 1.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 8.0, 8.0, 8.0, 12.0, 2.0, 0.0, 0.0, 12.0, 16.0, 14.0, 14.0, 15.0, 1.0, 0.0, 0.0, 14.0, 9.0, 0.0, 12.0, 6.0, 0.0, 0.0, 0.0, 10.0, 2.0, 8.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 5.0, 15.0, 12.0, 8.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 10.0, 13.0, 3.0, 0.0, 0.0, 5.0, 16.0, 9.0, 0.0, 8.0, 4.0, 0.0, 0.0, 4.0, 13.0, 1.0, 0.0, 4.0, 8.0, 0.0, 0.0, 4.0, 8.0, 0.0, 0.0, 8.0, 4.0, 0.0, 0.0, 1.0, 14.0, 0.0, 0.0, 11.0, 3.0, 0.0, 0.0, 0.0, 12.0, 9.0, 9.0, 15.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 15.0, 4.0, 0.0, 0.0], [0.0, 0.0, 6.0, 15.0, 16.0, 15.0, 11.0, 0.0, 0.0, 1.0, 15.0, 14.0, 8.0, 8.0, 7.0, 0.0, 0.0, 4.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 5.0, 0.0, 0.0, 0.0], [0.0, 0.0, 8.0, 14.0, 15.0, 6.0, 0.0, 0.0, 0.0, 4.0, 16.0, 12.0, 14.0, 11.0, 0.0, 0.0, 0.0, 4.0, 8.0, 1.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 7.0, 0.0, 0.0, 0.0, 4.0, 11.0, 8.0, 14.0, 7.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 10.0, 0.0, 0.0], [0.0, 0.0, 7.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 16.0, 12.0, 10.0, 15.0, 1.0, 0.0, 0.0, 0.0, 10.0, 4.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 7.0, 0.0, 0.0, 0.0, 8.0, 4.0, 10.0, 15.0, 2.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 6.0, 0.0, 0.0], [0.0, 0.0, 1.0, 11.0, 15.0, 6.0, 0.0, 0.0, 0.0, 2.0, 15.0, 10.0, 16.0, 15.0, 0.0, 0.0, 0.0, 1.0, 14.0, 5.0, 6.0, 11.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 10.0, 8.0, 6.0, 15.0, 1.0, 0.0, 0.0, 0.0, 9.0, 9.0, 4.0, 16.0, 3.0, 0.0, 0.0, 0.0, 1.0, 15.0, 15.0, 6.0, 0.0, 0.0], [0.0, 0.0, 2.0, 16.0, 14.0, 2.0, 0.0, 0.0, 0.0, 1.0, 12.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 4.0, 16.0, 12.0, 12.0, 12.0, 0.0, 0.0, 0.0, 1.0, 15.0, 11.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 14.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 10.0, 12.0, 6.0, 15.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 16.0, 12.0, 0.0, 0.0], [0.0, 0.0, 2.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 5.0, 16.0, 9.0, 6.0, 16.0, 4.0, 0.0, 0.0, 5.0, 16.0, 3.0, 1.0, 14.0, 7.0, 0.0, 0.0, 6.0, 16.0, 4.0, 0.0, 16.0, 8.0, 0.0, 0.0, 3.0, 16.0, 12.0, 6.0, 16.0, 12.0, 0.0, 0.0, 0.0, 14.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 2.0, 13.0, 15.0, 8.0, 0.0, 0.0], [0.0, 3.0, 16.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 14.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 8.0, 8.0, 10.0, 5.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 14.0, 3.0, 0.0], [0.0, 0.0, 9.0, 12.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 12.0, 6.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 1.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 12.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 12.0, 0.0, 0.0, 0.0, 9.0, 7.0, 4.0, 10.0, 11.0, 0.0, 0.0, 0.0, 9.0, 14.0, 16.0, 14.0, 5.0, 0.0], [0.0, 0.0, 6.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 9.0, 16.0, 0.0, 0.0, 0.0, 0.0, 3.0, 7.0, 5.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 8.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 8.0, 0.0, 0.0, 0.0, 0.0, 4.0, 10.0, 15.0, 2.0, 0.0, 0.0, 0.0, 5.0, 16.0, 12.0, 4.0, 0.0, 0.0], [0.0, 0.0, 5.0, 12.0, 16.0, 15.0, 2.0, 0.0, 0.0, 6.0, 15.0, 9.0, 10.0, 15.0, 4.0, 0.0, 0.0, 3.0, 14.0, 3.0, 1.0, 14.0, 4.0, 0.0, 0.0, 0.0, 10.0, 16.0, 15.0, 13.0, 1.0, 0.0, 0.0, 0.0, 6.0, 15.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 15.0, 3.0, 2.0, 15.0, 3.0, 0.0, 0.0, 0.0, 16.0, 8.0, 1.0, 14.0, 4.0, 0.0, 0.0, 0.0, 4.0, 15.0, 16.0, 11.0, 2.0, 0.0], [0.0, 0.0, 6.0, 13.0, 16.0, 6.0, 0.0, 0.0, 0.0, 3.0, 16.0, 14.0, 15.0, 16.0, 1.0, 0.0, 0.0, 0.0, 5.0, 0.0, 8.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 3.0, 0.0, 0.0, 3.0, 15.0, 16.0, 16.0, 16.0, 9.0, 0.0, 0.0, 5.0, 13.0, 14.0, 16.0, 11.0, 3.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 7.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 11.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 15.0, 3.0, 0.0, 0.0], [0.0, 0.0, 4.0, 13.0, 13.0, 4.0, 0.0, 0.0, 0.0, 0.0, 16.0, 10.0, 10.0, 8.0, 0.0, 0.0, 0.0, 0.0, 14.0, 7.0, 6.0, 11.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 15.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 9.0, 0.0, 0.0, 1.0, 4.0, 4.0, 6.0, 12.0, 10.0, 0.0, 0.0, 1.0, 6.0, 11.0, 15.0, 12.0, 1.0, 0.0], [0.0, 0.0, 12.0, 16.0, 16.0, 15.0, 3.0, 0.0, 0.0, 4.0, 16.0, 16.0, 6.0, 2.0, 1.0, 0.0, 0.0, 14.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 15.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 6.0, 4.0, 1.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 14.0, 16.0, 3.0, 0.0, 0.0, 2.0, 14.0, 9.0, 0.0, 11.0, 9.0, 0.0, 0.0, 0.0, 10.0, 14.0, 8.0, 15.0, 5.0, 0.0, 0.0, 0.0, 3.0, 13.0, 16.0, 8.0, 0.0, 0.0], [0.0, 2.0, 14.0, 16.0, 14.0, 4.0, 0.0, 0.0, 0.0, 4.0, 11.0, 5.0, 13.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 4.0, 0.0, 0.0, 0.0, 1.0, 3.0, 11.0, 15.0, 2.0, 0.0, 0.0, 1.0, 12.0, 16.0, 9.0, 2.0, 0.0, 0.0], [0.0, 0.0, 7.0, 16.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 9.0, 15.0, 2.0, 0.0, 0.0, 1.0, 15.0, 12.0, 1.0, 9.0, 8.0, 0.0, 0.0, 4.0, 16.0, 0.0, 0.0, 7.0, 10.0, 0.0, 0.0, 7.0, 13.0, 0.0, 0.0, 10.0, 11.0, 0.0, 0.0, 7.0, 12.0, 0.0, 2.0, 15.0, 6.0, 0.0, 0.0, 3.0, 15.0, 12.0, 14.0, 14.0, 1.0, 0.0, 0.0, 0.0, 10.0, 16.0, 14.0, 4.0, 0.0, 0.0], [0.0, 6.0, 13.0, 5.0, 8.0, 8.0, 1.0, 0.0, 0.0, 8.0, 16.0, 16.0, 16.0, 16.0, 6.0, 0.0, 0.0, 6.0, 16.0, 9.0, 6.0, 4.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 9.0, 0.0, 0.0, 0.0, 1.0, 8.0, 13.0, 15.0, 3.0, 0.0, 0.0, 0.0, 4.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0], [0.0, 0.0, 10.0, 16.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 12.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 6.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 16.0, 13.0, 8.0, 5.0, 0.0, 0.0, 6.0, 16.0, 10.0, 9.0, 12.0, 15.0], [0.0, 0.0, 0.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 9.0, 0.0, 2.0, 0.0, 0.0, 0.0, 7.0, 15.0, 1.0, 5.0, 15.0, 1.0, 0.0, 0.0, 14.0, 10.0, 4.0, 11.0, 12.0, 3.0, 0.0, 2.0, 16.0, 16.0, 16.0, 16.0, 13.0, 2.0, 0.0, 0.0, 3.0, 4.0, 11.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 4.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 9.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 7.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 5.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 4.0, 9.0, 13.0, 16.0, 11.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 13.0, 3.0, 0.0], [0.0, 0.0, 10.0, 12.0, 12.0, 15.0, 4.0, 0.0, 0.0, 0.0, 16.0, 8.0, 8.0, 5.0, 3.0, 0.0, 0.0, 4.0, 15.0, 8.0, 6.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 12.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 3.0, 0.0, 0.0, 0.0, 11.0, 4.0, 8.0, 15.0, 3.0, 0.0, 0.0, 0.0, 10.0, 16.0, 15.0, 5.0, 0.0, 0.0], [0.0, 0.0, 0.0, 4.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 16.0, 11.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 14.0, 11.0, 11.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 5.0, 0.0, 0.0], [0.0, 0.0, 5.0, 11.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 16.0, 12.0, 4.0, 3.0, 0.0, 0.0, 0.0, 4.0, 16.0, 6.0, 3.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 6.0, 0.0, 0.0, 0.0, 0.0, 1.0, 8.0, 15.0, 1.0, 0.0, 0.0, 0.0, 6.0, 16.0, 13.0, 1.0, 0.0, 0.0], [0.0, 0.0, 4.0, 15.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 13.0, 11.0, 11.0, 15.0, 0.0, 0.0, 0.0, 0.0, 15.0, 13.0, 15.0, 16.0, 7.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 11.0, 2.0, 0.0, 0.0, 0.0, 5.0, 15.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 16.0, 9.0, 12.0, 11.0, 0.0, 0.0, 0.0, 2.0, 16.0, 6.0, 8.0, 16.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 13.0, 8.0, 0.0, 0.0], [0.0, 1.0, 12.0, 12.0, 12.0, 15.0, 6.0, 0.0, 0.0, 1.0, 14.0, 5.0, 5.0, 4.0, 1.0, 0.0, 0.0, 0.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 15.0, 8.0, 0.0, 0.0, 0.0, 1.0, 4.0, 4.0, 5.0, 12.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 4.0, 0.0, 0.0, 2.0, 7.0, 2.0, 10.0, 12.0, 0.0, 0.0, 0.0, 2.0, 16.0, 15.0, 8.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 9.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 10.0, 0.0, 2.0, 0.0, 0.0, 0.0, 10.0, 16.0, 5.0, 7.0, 15.0, 1.0, 0.0, 0.0, 2.0, 11.0, 15.0, 16.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 5.0, 0.0, 0.0], [0.0, 1.0, 5.0, 11.0, 15.0, 4.0, 0.0, 0.0, 0.0, 8.0, 16.0, 13.0, 6.0, 2.0, 0.0, 0.0, 0.0, 11.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 11.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 5.0, 12.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 11.0, 0.0, 0.0, 0.0, 1.0, 6.0, 0.0, 10.0, 11.0, 0.0, 0.0, 0.0, 2.0, 12.0, 16.0, 15.0, 2.0, 0.0], [0.0, 1.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 16.0, 14.0, 2.0, 0.0, 0.0, 6.0, 16.0, 11.0, 8.0, 8.0, 3.0, 0.0, 0.0, 5.0, 16.0, 11.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 14.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 14.0, 14.0, 4.0, 0.0, 0.0, 0.0], [0.0, 0.0, 7.0, 13.0, 11.0, 1.0, 0.0, 0.0, 0.0, 6.0, 14.0, 12.0, 14.0, 9.0, 0.0, 0.0, 0.0, 5.0, 14.0, 3.0, 10.0, 9.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 9.0, 9.0, 3.0, 15.0, 4.0, 0.0, 0.0, 0.0, 12.0, 5.0, 1.0, 11.0, 8.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 9.0, 1.0, 0.0], [0.0, 0.0, 0.0, 4.0, 13.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 3.0, 14.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 2.0, 8.0, 15.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 12.0, 0.0, 0.0], [0.0, 0.0, 0.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 10.0, 0.0, 8.0, 2.0, 0.0, 0.0, 4.0, 16.0, 5.0, 11.0, 16.0, 8.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 16.0, 3.0, 0.0, 0.0, 2.0, 13.0, 9.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 15.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 10.0, 12.0, 3.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 15.0, 14.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 10.0, 16.0, 1.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 10.0, 1.0, 0.0, 0.0, 1.0, 16.0, 12.0, 16.0, 8.0, 0.0, 0.0, 0.0, 1.0, 16.0, 3.0, 4.0, 16.0, 4.0, 0.0, 0.0, 0.0, 12.0, 11.0, 4.0, 16.0, 9.0, 0.0, 0.0, 0.0, 2.0, 10.0, 14.0, 13.0, 4.0, 0.0], [0.0, 0.0, 2.0, 13.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 11.0, 0.0, 0.0, 0.0, 3.0, 16.0, 10.0, 3.0, 15.0, 8.0, 0.0, 0.0, 0.0, 16.0, 8.0, 0.0, 13.0, 10.0, 0.0, 0.0, 0.0, 12.0, 15.0, 1.0, 15.0, 9.0, 0.0, 0.0, 0.0, 2.0, 11.0, 16.0, 16.0, 2.0, 0.0], [0.0, 0.0, 0.0, 2.0, 9.0, 16.0, 10.0, 0.0, 0.0, 0.0, 7.0, 15.0, 8.0, 7.0, 12.0, 0.0, 0.0, 1.0, 15.0, 3.0, 0.0, 11.0, 12.0, 0.0, 0.0, 8.0, 14.0, 9.0, 13.0, 16.0, 8.0, 0.0, 0.0, 1.0, 7.0, 7.0, 3.0, 13.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 4.0, 0.0, 0.0], [0.0, 4.0, 15.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 2.0, 6.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 16.0, 13.0, 12.0, 6.0, 0.0, 0.0, 5.0, 16.0, 15.0, 16.0, 12.0, 3.0, 0.0], [0.0, 0.0, 8.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 1.0, 12.0, 10.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 2.0, 0.0, 0.0, 0.0, 4.0, 12.0, 14.0, 16.0, 12.0, 5.0, 0.0, 0.0, 12.0, 16.0, 16.0, 14.0, 12.0, 5.0, 0.0, 0.0, 0.0, 6.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 8.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.0, 12.0, 16.0, 16.0, 15.0, 0.0, 0.0, 0.0, 9.0, 10.0, 7.0, 12.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 13.0, 0.0, 0.0, 0.0, 1.0, 9.0, 9.0, 16.0, 11.0, 1.0, 0.0, 0.0, 13.0, 16.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 1.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 2.0, 0.0, 0.0, 0.0], [0.0, 0.0, 11.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 6.0, 12.0, 4.0, 13.0, 4.0, 0.0, 0.0, 0.0, 10.0, 10.0, 0.0, 4.0, 14.0, 0.0, 0.0, 0.0, 7.0, 13.0, 5.0, 13.0, 16.0, 2.0, 0.0, 0.0, 1.0, 10.0, 12.0, 12.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 12.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 15.0, 0.0, 0.0, 0.0, 11.0, 8.0, 4.0, 5.0, 16.0, 1.0], [0.0, 0.0, 3.0, 13.0, 15.0, 5.0, 0.0, 0.0, 0.0, 1.0, 15.0, 13.0, 10.0, 15.0, 0.0, 0.0, 0.0, 2.0, 16.0, 3.0, 2.0, 9.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 4.0, 9.0, 11.0, 1.0, 0.0, 0.0, 0.0, 15.0, 3.0, 0.0, 8.0, 8.0, 0.0, 0.0, 0.0, 3.0, 12.0, 15.0, 12.0, 7.0, 0.0], [0.0, 1.0, 15.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 0.0, 0.0, 3.0, 2.0, 0.0, 0.0, 2.0, 16.0, 3.0, 2.0, 13.0, 3.0, 0.0, 0.0, 0.0, 11.0, 14.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 13.0, 14.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 4.0, 16.0, 3.0, 0.0, 0.0, 0.0, 1.0, 15.0, 16.0, 12.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 10.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 12.0, 8.0, 2.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 16.0, 10.0, 1.0, 0.0, 0.0, 7.0, 16.0, 12.0, 12.0, 16.0, 4.0, 0.0, 0.0, 0.0, 9.0, 15.0, 12.0, 5.0, 0.0], [0.0, 0.0, 1.0, 9.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 9.0, 14.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 12.0, 16.0, 13.0, 3.0, 0.0, 0.0, 2.0, 15.0, 13.0, 4.0, 3.0, 13.0, 0.0, 0.0, 0.0, 9.0, 8.0, 2.0, 4.0, 16.0, 1.0, 0.0, 0.0, 0.0, 9.0, 12.0, 12.0, 8.0, 0.0], [0.0, 0.0, 10.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 9.0, 8.0, 13.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 4.0, 0.0, 0.0, 0.0, 7.0, 3.0, 5.0, 16.0, 2.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 10.0, 0.0, 0.0], [0.0, 0.0, 10.0, 11.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 13.0, 13.0, 1.0, 0.0, 0.0, 0.0, 8.0, 11.0, 0.0, 14.0, 4.0, 0.0, 0.0, 0.0, 0.0, 13.0, 15.0, 13.0, 0.0, 0.0, 0.0, 1.0, 11.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 3.0, 9.0, 10.0, 0.0, 0.0, 0.0, 0.0, 14.0, 6.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 7.0, 1.0, 0.0, 0.0], [0.0, 0.0, 4.0, 15.0, 14.0, 10.0, 1.0, 0.0, 0.0, 0.0, 7.0, 9.0, 0.0, 9.0, 8.0, 0.0, 0.0, 0.0, 11.0, 9.0, 2.0, 13.0, 7.0, 0.0, 0.0, 0.0, 4.0, 15.0, 14.0, 4.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 15.0, 1.0, 0.0, 0.0, 0.0, 4.0, 14.0, 1.0, 13.0, 7.0, 0.0, 0.0, 0.0, 7.0, 13.0, 1.0, 5.0, 13.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 16.0, 16.0, 1.0, 0.0], [0.0, 0.0, 2.0, 16.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 3.0, 6.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 13.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 13.0, 10.0, 3.0, 0.0, 0.0, 0.0, 4.0, 11.0, 15.0, 16.0, 10.0, 0.0], [0.0, 3.0, 11.0, 15.0, 12.0, 7.0, 1.0, 0.0, 0.0, 4.0, 16.0, 13.0, 11.0, 9.0, 6.0, 0.0, 0.0, 4.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 12.0, 0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 6.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 9.0, 13.0, 0.0, 6.0, 8.0, 0.0, 0.0, 3.0, 15.0, 3.0, 0.0, 15.0, 9.0, 0.0, 1.0, 13.0, 12.0, 4.0, 7.0, 15.0, 3.0, 0.0, 7.0, 16.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 6.0, 12.0, 10.0, 14.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 0.0, 0.0, 0.0], [0.0, 0.0, 10.0, 10.0, 12.0, 7.0, 0.0, 0.0, 0.0, 0.0, 15.0, 13.0, 5.0, 12.0, 5.0, 0.0, 0.0, 4.0, 13.0, 4.0, 0.0, 2.0, 8.0, 0.0, 0.0, 8.0, 4.0, 0.0, 0.0, 3.0, 8.0, 0.0, 0.0, 8.0, 4.0, 0.0, 0.0, 7.0, 5.0, 0.0, 0.0, 6.0, 6.0, 0.0, 0.0, 11.0, 2.0, 0.0, 0.0, 1.0, 13.0, 3.0, 3.0, 12.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 16.0, 7.0, 0.0, 0.0], [0.0, 0.0, 6.0, 8.0, 11.0, 14.0, 14.0, 0.0, 0.0, 1.0, 16.0, 16.0, 13.0, 12.0, 7.0, 0.0, 0.0, 0.0, 16.0, 7.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 1.0, 5.0, 6.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 12.0, 9.0, 0.0, 0.0, 0.0, 0.0, 12.0, 10.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 10.0, 14.0, 8.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 3.0, 14.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 9.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 4.0, 0.0, 4.0, 2.0, 0.0, 0.0, 12.0, 12.0, 7.0, 14.0, 16.0, 10.0, 0.0, 0.0, 13.0, 16.0, 14.0, 11.0, 16.0, 4.0, 0.0, 0.0, 2.0, 2.0, 0.0, 11.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 9.0, 0.0, 0.0], [0.0, 0.0, 10.0, 16.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 13.0, 13.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 4.0, 8.0, 15.0, 15.0, 4.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 13.0, 3.0, 0.0, 0.0, 0.0, 1.0, 12.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 7.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 10.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 11.0, 13.0, 10.0, 16.0, 0.0, 0.0, 0.0, 0.0, 12.0, 1.0, 4.0, 16.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 13.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 12.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 14.0, 7.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 14.0, 15.0, 5.0, 0.0], [0.0, 0.0, 2.0, 10.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 3.0, 12.0, 0.0, 0.0, 0.0, 0.0, 6.0, 13.0, 3.0, 15.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 15.0, 9.0, 0.0, 0.0, 0.0, 2.0, 16.0, 11.0, 9.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 2.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 6.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 6.0, 0.0, 0.0, 0.0], [0.0, 0.0, 12.0, 16.0, 14.0, 8.0, 0.0, 0.0, 0.0, 7.0, 16.0, 10.0, 14.0, 16.0, 0.0, 0.0, 0.0, 4.0, 16.0, 11.0, 14.0, 16.0, 4.0, 0.0, 0.0, 0.0, 5.0, 14.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 6.0, 0.0, 0.0, 2.0, 12.0, 9.0, 16.0, 15.0, 1.0, 0.0, 0.0, 1.0, 9.0, 16.0, 14.0, 3.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 13.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 16.0, 10.0, 1.0, 0.0, 0.0, 4.0, 16.0, 14.0, 16.0, 16.0, 11.0, 0.0, 0.0, 7.0, 16.0, 13.0, 15.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 7.0, 15.0, 11.0, 0.0, 0.0, 0.0, 0.0, 11.0, 8.0, 3.0, 13.0, 0.0, 0.0, 0.0, 10.0, 6.0, 2.0, 12.0, 11.0, 0.0, 0.0, 1.0, 16.0, 12.0, 16.0, 16.0, 7.0, 0.0, 0.0, 2.0, 16.0, 14.0, 7.0, 12.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 6.0, 0.0, 0.0], [0.0, 0.0, 4.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 15.0, 4.0, 0.0, 0.0, 0.0, 4.0, 16.0, 9.0, 4.0, 16.0, 2.0, 0.0, 0.0, 1.0, 15.0, 13.0, 6.0, 16.0, 11.0, 0.0, 0.0, 0.0, 4.0, 13.0, 16.0, 15.0, 5.0, 0.0], [0.0, 0.0, 1.0, 13.0, 14.0, 1.0, 0.0, 0.0, 0.0, 1.0, 11.0, 16.0, 10.0, 12.0, 0.0, 0.0, 0.0, 6.0, 16.0, 15.0, 0.0, 13.0, 3.0, 0.0, 0.0, 7.0, 14.0, 5.0, 0.0, 8.0, 9.0, 0.0, 0.0, 6.0, 13.0, 0.0, 0.0, 8.0, 11.0, 0.0, 0.0, 4.0, 15.0, 0.0, 1.0, 14.0, 9.0, 0.0, 0.0, 0.0, 14.0, 8.0, 12.0, 16.0, 3.0, 0.0, 0.0, 0.0, 3.0, 15.0, 15.0, 4.0, 0.0, 0.0], [0.0, 0.0, 9.0, 13.0, 14.0, 5.0, 0.0, 0.0, 0.0, 4.0, 16.0, 10.0, 13.0, 16.0, 0.0, 0.0, 0.0, 0.0, 13.0, 15.0, 14.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 7.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 3.0, 0.0, 0.0, 1.0, 15.0, 5.0, 8.0, 16.0, 2.0, 0.0, 0.0, 0.0, 7.0, 15.0, 16.0, 9.0, 0.0, 0.0], [0.0, 0.0, 4.0, 9.0, 13.0, 5.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 11.0, 8.0, 5.0, 16.0, 0.0, 0.0, 0.0, 0.0, 7.0, 10.0, 13.0, 10.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 9.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 8.0, 0.0, 0.0], [0.0, 0.0, 3.0, 12.0, 5.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 14.0, 16.0, 6.0, 0.0, 0.0, 0.0, 6.0, 16.0, 0.0, 6.0, 16.0, 6.0, 0.0, 0.0, 5.0, 16.0, 11.0, 14.0, 16.0, 4.0, 0.0, 0.0, 0.0, 8.0, 10.0, 12.0, 16.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 7.0, 15.0, 1.0, 0.0, 0.0, 0.0, 8.0, 10.0, 10.0, 16.0, 2.0, 0.0, 0.0, 0.0, 2.0, 13.0, 14.0, 7.0, 0.0, 0.0], [0.0, 0.0, 1.0, 8.0, 11.0, 13.0, 15.0, 3.0, 0.0, 0.0, 7.0, 16.0, 10.0, 10.0, 16.0, 5.0, 0.0, 1.0, 13.0, 3.0, 0.0, 9.0, 14.0, 0.0, 0.0, 3.0, 15.0, 0.0, 2.0, 15.0, 4.0, 0.0, 0.0, 5.0, 8.0, 0.0, 10.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 7.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 11.0, 0.0, 0.0, 0.0, 4.0, 16.0, 11.0, 13.0, 14.0, 0.0, 0.0, 0.0, 7.0, 12.0, 1.0, 3.0, 13.0, 0.0, 0.0, 0.0, 4.0, 10.0, 0.0, 0.0, 16.0, 0.0, 0.0, 0.0, 2.0, 14.0, 0.0, 1.0, 16.0, 1.0, 0.0, 0.0, 0.0, 9.0, 7.0, 9.0, 14.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 15.0, 3.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 2.0, 0.0, 0.0, 3.0, 13.0, 16.0, 16.0, 16.0, 0.0, 0.0, 0.0, 9.0, 16.0, 12.0, 16.0, 14.0, 0.0, 0.0, 0.0, 1.0, 3.0, 0.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 7.0, 0.0], [0.0, 0.0, 9.0, 12.0, 15.0, 15.0, 2.0, 0.0, 0.0, 1.0, 15.0, 14.0, 11.0, 16.0, 7.0, 0.0, 0.0, 0.0, 2.0, 0.0, 2.0, 16.0, 4.0, 0.0, 0.0, 0.0, 2.0, 4.0, 10.0, 15.0, 2.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 13.0, 16.0, 10.0, 1.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 9.0, 0.0, 0.0, 0.0, 0.0], [0.0, 2.0, 15.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 15.0, 14.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 10.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 10.0, 8.0, 6.0, 1.0, 0.0, 0.0, 2.0, 15.0, 16.0, 16.0, 16.0, 7.0, 0.0], [0.0, 0.0, 0.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 9.0, 10.0, 0.0, 0.0, 0.0, 2.0, 16.0, 2.0, 1.0, 11.0, 1.0, 0.0, 0.0, 1.0, 15.0, 0.0, 0.0, 5.0, 8.0, 0.0, 0.0, 2.0, 14.0, 0.0, 0.0, 5.0, 10.0, 0.0, 0.0, 0.0, 13.0, 2.0, 0.0, 2.0, 13.0, 0.0, 0.0, 0.0, 7.0, 9.0, 0.0, 7.0, 11.0, 0.0, 0.0, 0.0, 0.0, 11.0, 13.0, 16.0, 2.0, 0.0], [0.0, 0.0, 7.0, 11.0, 13.0, 8.0, 1.0, 0.0, 0.0, 1.0, 15.0, 9.0, 8.0, 6.0, 0.0, 0.0, 0.0, 10.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 6.0, 5.0, 10.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 16.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 14.0, 4.0, 0.0, 0.0, 0.0, 1.0, 10.0, 14.0, 2.0, 0.0, 0.0, 0.0], [0.0, 4.0, 13.0, 9.0, 8.0, 3.0, 0.0, 0.0, 0.0, 5.0, 16.0, 14.0, 12.0, 12.0, 6.0, 0.0, 0.0, 8.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 12.0, 14.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 14.0, 3.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.0, 13.0, 16.0, 7.0, 0.0, 0.0, 0.0, 1.0, 12.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 10.0, 11.0, 4.0, 0.0, 0.0, 0.0, 6.0, 16.0, 14.0, 13.0, 16.0, 3.0, 0.0, 0.0, 1.0, 11.0, 11.0, 2.0, 14.0, 10.0, 0.0, 0.0, 0.0, 2.0, 15.0, 16.0, 15.0, 6.0, 0.0], [0.0, 0.0, 4.0, 11.0, 15.0, 7.0, 0.0, 0.0, 0.0, 2.0, 15.0, 14.0, 9.0, 15.0, 1.0, 0.0, 0.0, 8.0, 15.0, 1.0, 6.0, 16.0, 5.0, 0.0, 0.0, 6.0, 14.0, 13.0, 15.0, 6.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 11.0, 15.0, 1.0, 0.0, 0.0, 0.0, 1.0, 12.0, 3.0, 7.0, 9.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 16.0, 6.0, 0.0, 0.0], [0.0, 0.0, 2.0, 11.0, 13.0, 5.0, 0.0, 0.0, 0.0, 1.0, 14.0, 9.0, 8.0, 14.0, 0.0, 0.0, 0.0, 6.0, 13.0, 1.0, 2.0, 16.0, 2.0, 0.0, 0.0, 7.0, 7.0, 0.0, 0.0, 12.0, 5.0, 0.0, 0.0, 7.0, 9.0, 0.0, 0.0, 3.0, 9.0, 0.0, 0.0, 2.0, 12.0, 0.0, 0.0, 4.0, 11.0, 0.0, 0.0, 0.0, 12.0, 6.0, 4.0, 14.0, 7.0, 0.0, 0.0, 0.0, 3.0, 13.0, 16.0, 9.0, 0.0, 0.0], [0.0, 0.0, 11.0, 8.0, 4.0, 13.0, 16.0, 3.0, 0.0, 2.0, 16.0, 16.0, 16.0, 14.0, 9.0, 1.0, 0.0, 4.0, 13.0, 6.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 16.0, 14.0, 12.0, 2.0, 0.0, 0.0, 8.0, 15.0, 15.0, 12.0, 12.0, 2.0, 0.0, 0.0, 0.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 9.0, 14.0, 16.0, 16.0, 2.0, 0.0, 0.0, 7.0, 15.0, 7.0, 4.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 12.0, 0.0, 0.0, 0.0, 0.0, 15.0, 2.0, 8.0, 14.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 12.0, 3.0, 0.0, 0.0], [0.0, 0.0, 5.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 14.0, 14.0, 1.0, 12.0, 0.0, 0.0, 0.0, 0.0, 15.0, 10.0, 0.0, 7.0, 4.0, 0.0, 0.0, 2.0, 16.0, 7.0, 0.0, 2.0, 9.0, 0.0, 0.0, 2.0, 16.0, 8.0, 0.0, 6.0, 11.0, 0.0, 0.0, 1.0, 12.0, 14.0, 14.0, 16.0, 5.0, 0.0, 0.0, 0.0, 4.0, 15.0, 16.0, 8.0, 1.0, 0.0], [0.0, 0.0, 6.0, 14.0, 9.0, 5.0, 2.0, 0.0, 0.0, 7.0, 15.0, 6.0, 2.0, 12.0, 8.0, 0.0, 0.0, 5.0, 15.0, 2.0, 8.0, 15.0, 1.0, 0.0, 0.0, 1.0, 12.0, 14.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 10.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 9.0, 2.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 11.0, 3.0, 0.0, 0.0], [0.0, 2.0, 13.0, 16.0, 16.0, 15.0, 4.0, 0.0, 0.0, 7.0, 12.0, 8.0, 8.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 11.0, 0.0, 0.0, 0.0, 4.0, 4.0, 7.0, 16.0, 10.0, 0.0, 0.0, 2.0, 15.0, 16.0, 16.0, 12.0, 1.0, 0.0], [0.0, 0.0, 6.0, 13.0, 15.0, 16.0, 11.0, 0.0, 0.0, 0.0, 10.0, 11.0, 8.0, 8.0, 5.0, 0.0, 0.0, 2.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 11.0, 7.0, 8.0, 5.0, 0.0, 0.0, 0.0, 7.0, 16.0, 14.0, 10.0, 14.0, 2.0, 0.0, 0.0, 1.0, 7.0, 1.0, 2.0, 12.0, 3.0, 0.0, 0.0, 0.0, 5.0, 8.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 8.0, 12.0, 5.0, 0.0, 0.0, 0.0], [0.0, 0.0, 7.0, 14.0, 16.0, 13.0, 1.0, 0.0, 0.0, 9.0, 15.0, 8.0, 10.0, 16.0, 7.0, 0.0, 0.0, 5.0, 1.0, 0.0, 14.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 7.0, 0.0, 0.0, 0.0, 9.0, 7.0, 6.0, 16.0, 9.0, 0.0, 0.0, 0.0, 5.0, 15.0, 16.0, 11.0, 3.0, 0.0], [0.0, 0.0, 5.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 13.0, 15.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 13.0, 4.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 16.0, 9.0, 13.0, 3.0, 0.0, 0.0, 5.0, 2.0, 9.0, 16.0, 14.0, 3.0], [0.0, 0.0, 0.0, 12.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 13.0, 8.0, 4.0, 0.0, 0.0, 0.0, 3.0, 16.0, 15.0, 11.0, 15.0, 7.0, 0.0, 0.0, 2.0, 15.0, 10.0, 0.0, 4.0, 15.0, 3.0, 0.0, 0.0, 8.0, 12.0, 4.0, 6.0, 16.0, 5.0, 0.0, 0.0, 1.0, 11.0, 16.0, 16.0, 13.0, 0.0], [0.0, 2.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 10.0, 12.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 15.0, 12.0, 5.0, 0.0, 0.0, 2.0, 16.0, 16.0, 16.0, 16.0, 15.0, 2.0, 0.0, 2.0, 15.0, 14.0, 12.0, 12.0, 7.0, 0.0], [0.0, 0.0, 5.0, 11.0, 16.0, 16.0, 5.0, 0.0, 0.0, 3.0, 15.0, 11.0, 10.0, 16.0, 4.0, 0.0, 0.0, 0.0, 4.0, 0.0, 10.0, 14.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 9.0, 16.0, 14.0, 4.0, 1.0, 0.0, 0.0, 0.0, 1.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 1.0, 6.0, 4.0, 12.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 9.0, 0.0, 0.0, 0.0, 4.0, 6.0, 11.0, 16.0, 1.0, 0.0, 0.0, 0.0, 15.0, 16.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 2.0, 10.0, 11.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 10.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 10.0, 12.0, 16.0, 16.0, 8.0, 0.0, 0.0, 4.0, 16.0, 16.0, 11.0, 5.0, 4.0, 0.0, 0.0, 10.0, 12.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 10.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 4.0, 8.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 11.0, 0.0, 0.0, 0.0], [0.0, 0.0, 11.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 1.0, 16.0, 15.0, 13.0, 15.0, 1.0, 0.0, 0.0, 0.0, 14.0, 15.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 11.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 10.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 9.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 4.0, 2.0, 1.0, 0.0, 0.0, 0.0, 12.0, 13.0, 1.0, 14.0, 8.0, 1.0, 0.0, 1.0, 16.0, 16.0, 16.0, 16.0, 15.0, 3.0, 0.0, 0.0, 5.0, 8.0, 11.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 3.0, 0.0, 0.0], [0.0, 0.0, 3.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 14.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 11.0, 2.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 13.0, 1.0, 0.0, 0.0, 2.0, 16.0, 14.0, 15.0, 16.0, 5.0, 0.0, 0.0, 0.0, 5.0, 15.0, 14.0, 7.0, 0.0, 0.0], [0.0, 0.0, 3.0, 12.0, 8.0, 3.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 13.0, 13.0, 1.0, 0.0, 0.0, 0.0, 13.0, 8.0, 0.0, 9.0, 4.0, 0.0, 0.0, 0.0, 16.0, 2.0, 0.0, 6.0, 6.0, 0.0, 0.0, 4.0, 12.0, 0.0, 0.0, 10.0, 3.0, 0.0, 0.0, 3.0, 12.0, 0.0, 0.0, 13.0, 2.0, 0.0, 0.0, 0.0, 12.0, 4.0, 12.0, 10.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 13.0, 2.0, 0.0, 0.0], [0.0, 0.0, 0.0, 8.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 13.0, 1.0, 1.0, 0.0, 0.0, 0.0, 10.0, 15.0, 3.0, 15.0, 11.0, 0.0, 0.0, 7.0, 16.0, 7.0, 1.0, 16.0, 8.0, 0.0, 0.0, 9.0, 16.0, 13.0, 14.0, 16.0, 5.0, 0.0, 0.0, 1.0, 10.0, 15.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 4.0, 0.0, 0.0], [0.0, 0.0, 3.0, 10.0, 14.0, 3.0, 0.0, 0.0, 0.0, 8.0, 16.0, 11.0, 10.0, 13.0, 0.0, 0.0, 0.0, 7.0, 14.0, 0.0, 1.0, 15.0, 2.0, 0.0, 0.0, 2.0, 16.0, 9.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 12.0, 16.0, 15.0, 15.0, 2.0, 0.0, 0.0, 0.0, 12.0, 10.0, 0.0, 8.0, 8.0, 0.0, 0.0, 0.0, 9.0, 12.0, 4.0, 7.0, 12.0, 0.0, 0.0, 0.0, 2.0, 11.0, 16.0, 16.0, 9.0, 0.0], [0.0, 0.0, 14.0, 12.0, 12.0, 13.0, 0.0, 0.0, 0.0, 1.0, 16.0, 8.0, 8.0, 6.0, 0.0, 0.0, 0.0, 4.0, 15.0, 8.0, 4.0, 0.0, 0.0, 0.0, 0.0, 2.0, 12.0, 12.0, 15.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 7.0, 0.0, 0.0, 5.0, 14.0, 4.0, 7.0, 15.0, 2.0, 0.0, 0.0, 1.0, 10.0, 16.0, 15.0, 5.0, 0.0, 0.0], [0.0, 0.0, 5.0, 15.0, 13.0, 2.0, 0.0, 0.0, 0.0, 1.0, 15.0, 11.0, 8.0, 13.0, 0.0, 0.0, 0.0, 5.0, 14.0, 0.0, 0.0, 14.0, 5.0, 0.0, 0.0, 9.0, 16.0, 1.0, 0.0, 7.0, 9.0, 0.0, 0.0, 9.0, 13.0, 0.0, 0.0, 5.0, 14.0, 0.0, 0.0, 6.0, 16.0, 2.0, 0.0, 5.0, 15.0, 0.0, 0.0, 2.0, 14.0, 11.0, 5.0, 14.0, 12.0, 0.0, 0.0, 0.0, 5.0, 15.0, 16.0, 15.0, 3.0, 0.0], [0.0, 0.0, 13.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 5.0, 15.0, 6.0, 11.0, 13.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 11.0, 15.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 11.0, 0.0, 0.0, 0.0, 3.0, 4.0, 9.0, 16.0, 6.0, 0.0, 0.0, 0.0, 15.0, 16.0, 16.0, 10.0, 0.0, 0.0], [0.0, 0.0, 10.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 12.0, 9.0, 0.0, 0.0, 0.0, 0.0, 8.0, 9.0, 13.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 14.0, 4.0, 5.0, 8.0, 3.0, 0.0, 0.0, 8.0, 16.0, 16.0, 16.0, 16.0, 9.0], [0.0, 0.0, 4.0, 14.0, 11.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 14.0, 4.0, 0.0, 0.0], [0.0, 0.0, 4.0, 12.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 15.0, 12.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 9.0, 7.0, 0.0, 0.0, 0.0, 0.0, 4.0, 7.0, 15.0, 13.0, 7.0, 0.0, 0.0, 6.0, 16.0, 16.0, 15.0, 10.0, 3.0, 0.0, 0.0, 1.0, 4.0, 12.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 12.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 6.0, 14.0, 13.0, 4.0, 0.0, 0.0, 0.0, 4.0, 16.0, 11.0, 10.0, 15.0, 0.0, 0.0, 0.0, 9.0, 11.0, 0.0, 12.0, 11.0, 0.0, 0.0, 0.0, 7.0, 11.0, 8.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 13.0, 12.0, 9.0, 0.0, 0.0, 0.0, 0.0, 12.0, 8.0, 0.0, 15.0, 1.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 11.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 16.0, 5.0, 0.0, 0.0, 1.0, 9.0, 15.0, 16.0, 16.0, 4.0, 0.0, 0.0, 4.0, 12.0, 7.0, 3.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 4.0, 0.0], [0.0, 0.0, 1.0, 10.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 1.0, 5.0, 2.0, 0.0, 0.0, 0.0, 3.0, 16.0, 15.0, 11.0, 9.0, 5.0, 0.0, 0.0, 2.0, 16.0, 6.0, 0.0, 2.0, 14.0, 0.0, 0.0, 0.0, 10.0, 10.0, 1.0, 5.0, 15.0, 0.0, 0.0, 0.0, 1.0, 12.0, 15.0, 14.0, 6.0, 0.0], [0.0, 3.0, 15.0, 16.0, 13.0, 1.0, 0.0, 0.0, 0.0, 10.0, 13.0, 9.0, 16.0, 4.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 13.0, 12.0, 7.0, 2.0, 0.0, 0.0, 2.0, 13.0, 13.0, 13.0, 16.0, 15.0, 0.0], [0.0, 0.0, 15.0, 16.0, 16.0, 15.0, 9.0, 0.0, 0.0, 6.0, 16.0, 13.0, 12.0, 12.0, 11.0, 2.0, 0.0, 3.0, 15.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 8.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 3.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 6.0, 13.0, 7.0, 0.0, 0.0, 0.0, 0.0, 10.0, 13.0, 6.0, 15.0, 0.0, 0.0, 0.0, 0.0, 12.0, 8.0, 4.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 5.0, 15.0, 2.0, 0.0, 0.0, 0.0, 14.0, 10.0, 2.0, 5.0, 11.0, 0.0, 0.0, 0.0, 2.0, 7.0, 13.0, 15.0, 8.0, 0.0], [0.0, 0.0, 10.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 2.0, 12.0, 9.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 8.0, 5.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 16.0, 2.0, 0.0], [0.0, 0.0, 10.0, 16.0, 14.0, 2.0, 0.0, 0.0, 0.0, 3.0, 16.0, 9.0, 8.0, 14.0, 0.0, 0.0, 0.0, 5.0, 16.0, 3.0, 2.0, 15.0, 6.0, 0.0, 0.0, 5.0, 16.0, 3.0, 0.0, 12.0, 10.0, 0.0, 0.0, 7.0, 14.0, 0.0, 0.0, 12.0, 11.0, 0.0, 0.0, 7.0, 16.0, 1.0, 3.0, 16.0, 5.0, 0.0, 0.0, 4.0, 16.0, 7.0, 12.0, 11.0, 1.0, 0.0, 0.0, 0.0, 10.0, 16.0, 14.0, 3.0, 0.0, 0.0], [0.0, 0.0, 3.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 15.0, 16.0, 15.0, 2.0, 0.0, 0.0, 1.0, 16.0, 8.0, 4.0, 8.0, 11.0, 0.0, 0.0, 1.0, 16.0, 11.0, 7.0, 10.0, 12.0, 0.0, 0.0, 0.0, 5.0, 10.0, 12.0, 15.0, 7.0, 0.0], [0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 10.0, 10.0, 5.0, 12.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 6.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 12.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 6.0, 0.0, 0.0, 0.0, 2.0, 9.0, 11.0, 14.0, 1.0, 0.0, 0.0, 0.0, 5.0, 16.0, 15.0, 5.0, 0.0, 0.0], [0.0, 0.0, 3.0, 15.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 10.0, 1.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 12.0, 15.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 12.0, 1.0, 0.0, 0.0, 0.0], [0.0, 2.0, 16.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 4.0, 16.0, 6.0, 8.0, 7.0, 1.0, 0.0, 0.0, 4.0, 16.0, 7.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 5.0, 4.0, 10.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 6.0, 0.0, 0.0, 2.0, 14.0, 4.0, 4.0, 16.0, 8.0, 0.0, 0.0, 3.0, 13.0, 16.0, 16.0, 15.0, 1.0, 0.0], [0.0, 0.0, 0.0, 14.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 10.0, 2.0, 0.0, 0.0, 0.0, 0.0, 14.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 14.0, 4.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 8.0, 16.0, 2.0, 0.0, 0.0, 0.0, 14.0, 11.0, 0.0, 13.0, 9.0, 0.0, 0.0, 0.0, 9.0, 14.0, 6.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 14.0, 0.0, 0.0], [0.0, 0.0, 1.0, 11.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 14.0, 4.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 12.0, 5.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 16.0, 3.0, 0.0, 0.0, 3.0, 15.0, 5.0, 0.0, 15.0, 13.0, 0.0, 0.0, 0.0, 11.0, 16.0, 14.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 9.0, 1.0, 0.0], [0.0, 0.0, 5.0, 15.0, 16.0, 14.0, 1.0, 0.0, 0.0, 0.0, 11.0, 13.0, 9.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 11.0, 0.0, 0.0, 0.0, 0.0, 7.0, 13.0, 15.0, 12.0, 1.0, 0.0, 0.0, 0.0, 7.0, 14.0, 14.0, 12.0, 4.0, 0.0, 0.0, 0.0, 0.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 10.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.0, 13.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 14.0, 12.0, 14.0, 16.0, 1.0, 0.0, 0.0, 0.0, 6.0, 0.0, 7.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 8.0, 13.0, 16.0, 14.0, 4.0, 0.0, 0.0, 5.0, 16.0, 16.0, 14.0, 12.0, 4.0, 0.0, 0.0, 0.0, 3.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 13.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 13.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 4.0, 0.0, 0.0, 0.0, 3.0, 9.0, 13.0, 16.0, 12.0, 5.0, 0.0, 0.0, 3.0, 15.0, 16.0, 16.0, 16.0, 16.0], [0.0, 0.0, 2.0, 10.0, 15.0, 1.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 13.0, 13.0, 0.0, 0.0, 0.0, 5.0, 16.0, 12.0, 1.0, 12.0, 1.0, 0.0, 0.0, 7.0, 13.0, 5.0, 0.0, 7.0, 5.0, 0.0, 0.0, 2.0, 14.0, 0.0, 0.0, 7.0, 10.0, 0.0, 0.0, 0.0, 12.0, 2.0, 0.0, 12.0, 7.0, 0.0, 0.0, 0.0, 9.0, 12.0, 12.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 6.0, 0.0, 0.0], [0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 13.0, 0.0, 0.0, 0.0, 8.0, 9.0, 8.0, 15.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 7.0, 0.0, 0.0, 0.0, 4.0, 8.0, 12.0, 16.0, 5.0, 0.0, 0.0, 3.0, 16.0, 16.0, 16.0, 14.0, 7.0, 0.0, 0.0, 0.0, 3.0, 8.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 3.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 10.0, 0.0, 1.0, 3.0, 0.0, 0.0, 5.0, 16.0, 3.0, 1.0, 12.0, 15.0, 0.0, 0.0, 11.0, 16.0, 8.0, 14.0, 15.0, 3.0, 0.0, 0.0, 6.0, 16.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 12.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 8.0, 11.0, 15.0, 7.0, 0.0, 0.0, 0.0, 6.0, 13.0, 10.0, 16.0, 7.0, 0.0, 0.0, 3.0, 16.0, 14.0, 12.0, 15.0, 4.0, 0.0, 0.0, 1.0, 11.0, 8.0, 1.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 4.0, 0.0, 0.0], [0.0, 0.0, 9.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 2.0, 16.0, 9.0, 11.0, 11.0, 0.0, 0.0, 0.0, 0.0, 15.0, 7.0, 12.0, 16.0, 3.0, 0.0, 0.0, 0.0, 7.0, 16.0, 15.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 10.0, 0.0, 0.0, 0.0, 11.0, 2.0, 6.0, 16.0, 6.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 11.0, 1.0, 0.0], [0.0, 0.0, 3.0, 12.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 10.0, 11.0, 7.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 10.0, 0.0, 0.0, 5.0, 11.0, 8.0, 9.0, 16.0, 3.0, 0.0, 0.0, 9.0, 16.0, 16.0, 16.0, 16.0, 9.0, 0.0, 0.0, 1.0, 4.0, 9.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 5.0, 0.0, 0.0, 0.0], [0.0, 0.0, 10.0, 7.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 9.0, 10.0, 12.0, 15.0, 2.0, 0.0, 0.0, 0.0, 4.0, 11.0, 10.0, 11.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 10.0, 1.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 1.0, 12.0, 0.0, 0.0, 0.0, 0.0, 1.0, 10.0, 2.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 14.0, 5.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 4.0, 14.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 5.0, 15.0, 16.0, 15.0, 3.0, 0.0, 0.0, 4.0, 15.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 15.0, 6.0, 0.0], [0.0, 0.0, 5.0, 13.0, 11.0, 1.0, 0.0, 0.0, 0.0, 3.0, 16.0, 11.0, 8.0, 12.0, 0.0, 0.0, 0.0, 5.0, 16.0, 0.0, 0.0, 13.0, 3.0, 0.0, 0.0, 5.0, 13.0, 0.0, 0.0, 6.0, 7.0, 0.0, 0.0, 7.0, 10.0, 0.0, 0.0, 8.0, 7.0, 0.0, 0.0, 4.0, 13.0, 0.0, 1.0, 14.0, 5.0, 0.0, 0.0, 1.0, 15.0, 5.0, 12.0, 10.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 10.0, 1.0, 0.0, 0.0], [0.0, 3.0, 15.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 12.0, 15.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 4.0, 1.0, 0.0, 0.0, 1.0, 10.0, 16.0, 16.0, 16.0, 11.0, 0.0, 0.0, 4.0, 16.0, 14.0, 12.0, 8.0, 3.0, 0.0], [0.0, 0.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 9.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 15.0, 8.0, 12.0, 11.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 12.0, 2.0, 0.0], [0.0, 2.0, 15.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 11.0, 12.0, 0.0, 0.0, 0.0, 0.0, 8.0, 10.0, 0.0, 16.0, 0.0, 0.0, 0.0, 0.0, 1.0, 3.0, 2.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 13.0, 13.0, 15.0, 3.0, 0.0, 2.0, 13.0, 14.0, 12.0, 12.0, 8.0, 1.0], [0.0, 0.0, 0.0, 11.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 1.0, 10.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 15.0, 0.0, 0.0], [0.0, 0.0, 6.0, 8.0, 12.0, 14.0, 0.0, 0.0, 0.0, 5.0, 16.0, 15.0, 12.0, 7.0, 0.0, 0.0, 0.0, 8.0, 16.0, 13.0, 4.0, 0.0, 0.0, 0.0, 0.0, 2.0, 11.0, 8.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 5.0, 0.0, 0.0, 1.0, 9.0, 8.0, 12.0, 14.0, 1.0, 0.0, 0.0, 0.0, 10.0, 15.0, 12.0, 3.0, 0.0, 0.0], [0.0, 0.0, 0.0, 9.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 13.0, 4.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 11.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 16.0, 2.0, 0.0, 10.0, 8.0, 0.0, 0.0, 0.0, 8.0, 12.0, 4.0, 13.0, 7.0, 0.0, 0.0, 0.0, 1.0, 9.0, 16.0, 11.0, 1.0, 0.0], [0.0, 4.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 9.0, 10.0, 6.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 7.0, 2.0, 2.0, 0.0, 0.0, 1.0, 12.0, 16.0, 15.0, 16.0, 15.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 12.0, 11.0, 0.0], [0.0, 0.0, 0.0, 10.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 14.0, 6.0, 6.0, 15.0, 0.0, 0.0, 0.0, 4.0, 16.0, 4.0, 0.0, 15.0, 3.0, 0.0, 0.0, 5.0, 15.0, 5.0, 0.0, 11.0, 5.0, 0.0, 0.0, 0.0, 12.0, 11.0, 0.0, 13.0, 5.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 1.0, 8.0, 13.0, 8.0, 0.0, 0.0], [0.0, 0.0, 1.0, 12.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 11.0, 13.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 10.0, 12.0, 8.0, 1.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 2.0, 15.0, 16.0, 13.0, 2.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 12.0, 6.0, 0.0, 0.0], [0.0, 0.0, 5.0, 14.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 0.0, 0.0, 0.0, 0.0, 9.0, 14.0, 16.0, 16.0, 13.0, 0.0, 0.0, 2.0, 16.0, 16.0, 15.0, 7.0, 1.0, 0.0, 0.0, 0.0, 1.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 13.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 11.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 15.0, 9.0, 7.0, 2.0, 0.0, 0.0, 0.0, 12.0, 14.0, 13.0, 12.0, 5.0, 0.0], [0.0, 0.0, 1.0, 11.0, 12.0, 13.0, 14.0, 5.0, 0.0, 0.0, 7.0, 15.0, 11.0, 10.0, 16.0, 6.0, 0.0, 0.0, 10.0, 7.0, 0.0, 2.0, 16.0, 2.0, 0.0, 1.0, 16.0, 1.0, 0.0, 12.0, 8.0, 0.0, 0.0, 2.0, 11.0, 0.0, 4.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 7.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 10.0, 13.0, 5.0, 0.0, 0.0, 0.0, 3.0, 14.0, 16.0, 12.0, 15.0, 0.0, 0.0, 0.0, 10.0, 16.0, 8.0, 11.0, 16.0, 0.0, 0.0, 0.0, 8.0, 14.0, 5.0, 14.0, 9.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 11.0, 0.0, 0.0], [0.0, 1.0, 13.0, 15.0, 12.0, 12.0, 5.0, 0.0, 0.0, 4.0, 16.0, 8.0, 8.0, 6.0, 0.0, 0.0, 0.0, 7.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 13.0, 15.0, 7.0, 0.0, 0.0, 0.0, 1.0, 6.0, 5.0, 8.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 14.0, 1.0, 0.0, 0.0, 0.0, 3.0, 14.0, 10.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 11.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 3.0, 16.0, 9.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 6.0, 1.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 4.0, 5.0, 8.0, 1.0, 0.0, 0.0, 13.0, 16.0, 16.0, 16.0, 16.0, 6.0], [0.0, 0.0, 0.0, 11.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 6.0, 16.0, 15.0, 16.0, 3.0, 0.0, 0.0, 0.0, 11.0, 11.0, 10.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 2.0, 0.0, 0.0], [0.0, 0.0, 0.0, 12.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 7.0, 0.0, 0.0], [0.0, 1.0, 13.0, 16.0, 15.0, 6.0, 0.0, 0.0, 0.0, 10.0, 15.0, 9.0, 11.0, 15.0, 0.0, 0.0, 0.0, 7.0, 9.0, 0.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 10.0, 9.0, 16.0, 2.0, 0.0, 0.0, 13.0, 16.0, 15.0, 11.0, 4.0, 0.0], [0.0, 0.0, 9.0, 13.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 10.0, 0.0, 0.0, 0.0, 3.0, 7.0, 12.0, 14.0, 16.0, 2.0, 0.0, 0.0, 7.0, 12.0, 12.0, 12.0, 11.0, 0.0], [0.0, 0.0, 2.0, 13.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 10.0, 14.0, 10.0, 11.0, 8.0, 0.0, 0.0, 0.0, 16.0, 1.0, 0.0, 0.0, 9.0, 0.0, 0.0, 3.0, 13.0, 0.0, 0.0, 0.0, 8.0, 0.0, 0.0, 4.0, 12.0, 0.0, 0.0, 1.0, 8.0, 0.0, 0.0, 5.0, 12.0, 0.0, 0.0, 10.0, 0.0, 0.0, 0.0, 0.0, 15.0, 8.0, 7.0, 10.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 14.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 6.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 15.0, 16.0, 13.0, 5.0, 0.0, 0.0, 0.0, 10.0, 16.0, 5.0, 11.0, 14.0, 0.0, 0.0, 0.0, 7.0, 15.0, 5.0, 10.0, 14.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 16.0, 9.0, 0.0], [0.0, 0.0, 10.0, 16.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 1.0, 5.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 2.0, 10.0, 16.0, 11.0, 6.0, 0.0, 0.0, 7.0, 16.0, 16.0, 15.0, 12.0, 7.0, 0.0, 0.0, 11.0, 10.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.0, 14.0, 13.0, 12.0, 14.0, 0.0, 0.0, 0.0, 11.0, 14.0, 12.0, 15.0, 9.0, 0.0, 0.0, 0.0, 16.0, 5.0, 3.0, 16.0, 2.0, 0.0, 0.0, 1.0, 9.0, 1.0, 10.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 14.0, 6.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 13.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 6.0, 15.0, 15.0, 1.0, 0.0, 0.0, 0.0, 4.0, 16.0, 13.0, 16.0, 4.0, 0.0, 0.0, 0.0, 10.0, 11.0, 2.0, 16.0, 2.0, 0.0, 0.0, 0.0, 1.0, 1.0, 10.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 1.0, 12.0, 9.0, 0.0, 0.0, 0.0, 11.0, 15.0, 14.0, 16.0, 9.0, 0.0, 0.0, 0.0, 8.0, 16.0, 12.0, 5.0, 0.0, 0.0], [0.0, 0.0, 2.0, 14.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 13.0, 13.0, 9.0, 11.0, 0.0, 0.0, 0.0, 0.0, 16.0, 7.0, 0.0, 12.0, 0.0, 0.0, 0.0, 3.0, 16.0, 5.0, 0.0, 10.0, 5.0, 0.0, 0.0, 5.0, 16.0, 1.0, 0.0, 8.0, 5.0, 0.0, 0.0, 3.0, 16.0, 1.0, 0.0, 10.0, 5.0, 0.0, 0.0, 0.0, 16.0, 8.0, 5.0, 14.0, 3.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 9.0, 1.0, 0.0], [0.0, 0.0, 1.0, 6.0, 8.0, 9.0, 3.0, 0.0, 0.0, 0.0, 13.0, 15.0, 12.0, 11.0, 7.0, 0.0, 0.0, 0.0, 13.0, 11.0, 0.0, 9.0, 7.0, 0.0, 0.0, 0.0, 5.0, 15.0, 15.0, 15.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 11.0, 9.0, 0.0, 16.0, 1.0, 0.0, 0.0, 0.0, 9.0, 10.0, 10.0, 13.0, 0.0, 0.0, 0.0, 0.0, 3.0, 11.0, 9.0, 2.0, 0.0, 0.0], [0.0, 0.0, 2.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 16.0, 3.0, 0.0, 0.0], [0.0, 0.0, 0.0, 4.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 6.0, 0.0, 10.0, 1.0, 0.0, 0.0, 12.0, 12.0, 1.0, 7.0, 15.0, 1.0, 0.0, 5.0, 16.0, 3.0, 0.0, 14.0, 10.0, 0.0, 2.0, 16.0, 13.0, 8.0, 8.0, 16.0, 3.0, 0.0, 8.0, 16.0, 16.0, 16.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 6.0, 0.0, 0.0], [0.0, 0.0, 0.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 2.0, 0.0, 0.0], [0.0, 2.0, 15.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 3.0, 11.0, 5.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 3.0, 3.0, 4.0, 3.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 16.0, 11.0, 0.0], [0.0, 0.0, 2.0, 11.0, 13.0, 4.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 13.0, 15.0, 0.0, 0.0, 0.0, 0.0, 16.0, 9.0, 1.0, 3.0, 0.0, 0.0, 0.0, 4.0, 16.0, 6.0, 14.0, 9.0, 1.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 16.0, 6.0, 0.0, 0.0, 1.0, 16.0, 14.0, 4.0, 16.0, 8.0, 0.0, 0.0, 0.0, 12.0, 16.0, 13.0, 16.0, 2.0, 0.0, 0.0, 0.0, 2.0, 10.0, 16.0, 7.0, 0.0, 0.0], [0.0, 0.0, 0.0, 6.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 7.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 16.0, 11.0, 2.0, 0.0, 0.0, 2.0, 16.0, 13.0, 3.0, 8.0, 12.0, 0.0, 0.0, 0.0, 8.0, 15.0, 5.0, 4.0, 16.0, 2.0, 0.0, 0.0, 0.0, 4.0, 14.0, 16.0, 13.0, 0.0], [0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 9.0, 7.0, 9.0, 12.0, 0.0, 0.0, 0.0, 9.0, 8.0, 0.0, 12.0, 9.0, 0.0, 0.0, 4.0, 16.0, 8.0, 12.0, 16.0, 2.0, 0.0, 0.0, 5.0, 16.0, 16.0, 10.0, 15.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 5.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 10.0, 0.0, 0.0], [0.0, 0.0, 5.0, 14.0, 15.0, 2.0, 0.0, 0.0, 0.0, 6.0, 16.0, 10.0, 15.0, 8.0, 0.0, 0.0, 0.0, 1.0, 4.0, 0.0, 8.0, 8.0, 0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 6.0, 10.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 2.0, 0.0, 0.0, 0.0], [0.0, 0.0, 9.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 4.0, 16.0, 14.0, 8.0, 11.0, 11.0, 0.0, 0.0, 11.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 13.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 7.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 15.0, 16.0, 16.0, 16.0, 1.0, 0.0, 0.0, 10.0, 13.0, 8.0, 15.0, 8.0, 0.0, 0.0, 0.0, 14.0, 5.0, 3.0, 16.0, 2.0, 0.0, 0.0, 0.0, 1.0, 0.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 2.0, 5.0, 16.0, 9.0, 1.0, 0.0, 0.0, 0.0, 15.0, 16.0, 16.0, 14.0, 3.0, 0.0, 0.0, 0.0, 1.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 2.0, 0.0, 0.0, 0.0], [0.0, 1.0, 8.0, 14.0, 15.0, 2.0, 0.0, 0.0, 0.0, 2.0, 13.0, 9.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 9.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 6.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 12.0, 14.0, 0.0, 0.0, 0.0, 5.0, 13.0, 5.0, 6.0, 16.0, 1.0, 0.0, 0.0, 1.0, 9.0, 12.0, 13.0, 9.0, 0.0, 0.0], [0.0, 0.0, 0.0, 8.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 7.0, 1.0, 9.0, 3.0, 0.0, 2.0, 15.0, 12.0, 0.0, 13.0, 16.0, 4.0, 0.0, 9.0, 16.0, 10.0, 10.0, 16.0, 11.0, 0.0, 0.0, 4.0, 15.0, 16.0, 16.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 14.0, 0.0, 0.0, 0.0], [0.0, 0.0, 5.0, 16.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 8.0, 13.0, 9.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 4.0, 0.0, 0.0, 0.0, 11.0, 10.0, 10.0, 15.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 16.0, 15.0, 13.0, 4.0, 0.0, 0.0, 7.0, 16.0, 13.0, 10.0, 8.0, 3.0], [0.0, 0.0, 1.0, 11.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 15.0, 16.0, 14.0, 3.0, 0.0, 0.0, 2.0, 16.0, 11.0, 2.0, 7.0, 12.0, 0.0, 0.0, 0.0, 14.0, 11.0, 4.0, 9.0, 13.0, 0.0, 0.0, 0.0, 2.0, 11.0, 16.0, 15.0, 6.0, 0.0], [0.0, 0.0, 5.0, 10.0, 14.0, 16.0, 11.0, 0.0, 0.0, 2.0, 15.0, 15.0, 5.0, 4.0, 1.0, 0.0, 0.0, 2.0, 16.0, 9.0, 4.0, 1.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 16.0, 11.0, 0.0, 0.0, 0.0, 2.0, 9.0, 1.0, 0.0, 14.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 7.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 7.0, 0.0, 0.0, 0.0], [0.0, 1.0, 12.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 4.0, 16.0, 10.0, 4.0, 1.0, 1.0, 0.0, 0.0, 6.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 12.0, 5.0, 0.0, 0.0, 0.0, 0.0, 7.0, 11.0, 11.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 10.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 9.0, 15.0, 15.0, 1.0, 0.0, 0.0, 0.0, 13.0, 14.0, 8.0, 12.0, 4.0, 0.0, 0.0, 5.0, 11.0, 1.0, 2.0, 13.0, 1.0, 0.0, 0.0, 1.0, 4.0, 0.0, 11.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 13.0, 6.0, 0.0, 0.0, 0.0, 0.0, 4.0, 10.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 4.0, 0.0, 0.0], [0.0, 0.0, 8.0, 13.0, 14.0, 5.0, 0.0, 0.0, 0.0, 5.0, 13.0, 4.0, 11.0, 9.0, 0.0, 0.0, 0.0, 4.0, 13.0, 1.0, 12.0, 14.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 11.0, 12.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 8.0, 0.0, 0.0, 0.0, 2.0, 2.0, 0.0, 11.0, 7.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 13.0, 2.0, 0.0], [0.0, 0.0, 3.0, 11.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 10.0, 14.0, 6.0, 0.0, 0.0, 0.0, 15.0, 7.0, 0.0, 11.0, 8.0, 0.0, 0.0, 3.0, 16.0, 2.0, 0.0, 8.0, 8.0, 0.0, 0.0, 4.0, 12.0, 0.0, 0.0, 9.0, 8.0, 0.0, 0.0, 6.0, 15.0, 1.0, 0.0, 12.0, 8.0, 0.0, 0.0, 3.0, 15.0, 10.0, 8.0, 15.0, 4.0, 0.0, 0.0, 0.0, 5.0, 12.0, 14.0, 9.0, 0.0, 0.0], [0.0, 0.0, 0.0, 6.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 10.0, 1.0, 2.0, 0.0, 0.0, 0.0, 6.0, 16.0, 4.0, 12.0, 10.0, 0.0, 0.0, 0.0, 14.0, 11.0, 0.0, 16.0, 8.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 1.0, 11.0, 12.0, 12.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 4.0, 0.0, 0.0], [0.0, 0.0, 2.0, 10.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 15.0, 10.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 12.0, 16.0, 12.0, 13.0, 8.0, 0.0, 0.0, 0.0, 1.0, 7.0, 1.0, 10.0, 11.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 8.0, 12.0, 0.0, 0.0, 3.0, 15.0, 10.0, 2.0, 11.0, 12.0, 0.0, 0.0, 0.0, 3.0, 10.0, 16.0, 16.0, 10.0, 0.0], [0.0, 0.0, 7.0, 8.0, 13.0, 16.0, 15.0, 1.0, 0.0, 0.0, 7.0, 7.0, 4.0, 11.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 13.0, 1.0, 0.0, 0.0, 4.0, 8.0, 8.0, 15.0, 15.0, 6.0, 0.0, 0.0, 2.0, 11.0, 15.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 7.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 13.0, 13.0, 0.0, 0.0, 0.0, 0.0, 6.0, 12.0, 14.0, 16.0, 12.0, 5.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 16.0, 15.0], [0.0, 0.0, 2.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 12.0, 12.0, 11.0, 1.0, 0.0, 0.0, 4.0, 16.0, 14.0, 10.0, 14.0, 11.0, 0.0, 0.0, 2.0, 15.0, 10.0, 6.0, 16.0, 10.0, 0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 10.0, 1.0, 0.0], [0.0, 0.0, 5.0, 12.0, 15.0, 10.0, 1.0, 0.0, 0.0, 2.0, 14.0, 7.0, 4.0, 9.0, 7.0, 0.0, 0.0, 7.0, 15.0, 7.0, 0.0, 9.0, 8.0, 0.0, 0.0, 1.0, 5.0, 15.0, 11.0, 13.0, 3.0, 0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 15.0, 9.0, 12.0, 7.0, 0.0, 0.0, 0.0, 0.0, 15.0, 5.0, 8.0, 12.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 16.0, 11.0, 0.0, 0.0], [0.0, 0.0, 9.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 13.0, 15.0, 7.0, 0.0, 0.0, 0.0, 7.0, 16.0, 6.0, 4.0, 16.0, 3.0, 0.0, 0.0, 9.0, 14.0, 0.0, 0.0, 11.0, 10.0, 0.0, 0.0, 8.0, 13.0, 0.0, 0.0, 7.0, 13.0, 0.0, 0.0, 7.0, 13.0, 0.0, 0.0, 7.0, 16.0, 0.0, 0.0, 3.0, 16.0, 7.0, 7.0, 15.0, 6.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 10.0, 0.0, 0.0], [0.0, 0.0, 1.0, 15.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 1.0, 0.0, 0.0, 0.0, 3.0, 10.0, 13.0, 13.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 12.0, 4.0, 0.0, 0.0, 0.0, 1.0, 13.0, 15.0, 14.0, 16.0, 7.0], [0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 16.0, 2.0, 0.0, 0.0, 3.0, 12.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 10.0, 16.0, 15.0, 16.0, 15.0, 0.0, 0.0, 0.0, 1.0, 4.0, 0.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 6.0, 0.0], [0.0, 0.0, 3.0, 6.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 13.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 9.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 4.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 14.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 11.0, 14.0, 4.0, 0.0, 0.0, 0.0, 5.0, 13.0, 4.0, 9.0, 7.0, 0.0, 0.0, 0.0, 7.0, 10.0, 10.0, 13.0, 2.0, 0.0, 0.0, 0.0, 1.0, 9.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 8.0, 7.0, 9.0, 12.0, 0.0, 0.0, 0.0, 0.0, 12.0, 0.0, 1.0, 14.0, 5.0, 0.0, 0.0, 0.0, 11.0, 6.0, 0.0, 7.0, 8.0, 0.0, 0.0, 0.0, 2.0, 15.0, 16.0, 15.0, 4.0, 0.0], [0.0, 0.0, 6.0, 11.0, 16.0, 16.0, 3.0, 0.0, 0.0, 5.0, 16.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 4.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 13.0, 0.0, 0.0, 0.0], [0.0, 0.0, 9.0, 15.0, 16.0, 15.0, 2.0, 0.0, 0.0, 4.0, 16.0, 5.0, 3.0, 1.0, 0.0, 0.0, 0.0, 4.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 9.0, 14.0, 15.0, 2.0, 0.0, 0.0, 5.0, 13.0, 9.0, 8.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 11.0, 14.0, 0.0, 0.0, 0.0, 0.0, 11.0, 12.0, 7.0, 1.0, 0.0, 0.0], [0.0, 1.0, 14.0, 16.0, 16.0, 14.0, 1.0, 0.0, 0.0, 0.0, 10.0, 13.0, 6.0, 4.0, 0.0, 0.0, 0.0, 3.0, 15.0, 11.0, 3.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 10.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 11.0, 0.0, 0.0, 0.0, 0.0, 7.0, 12.0, 16.0, 5.0, 0.0, 0.0, 0.0, 2.0, 15.0, 15.0, 5.0, 0.0, 0.0, 0.0], [0.0, 5.0, 16.0, 16.0, 16.0, 11.0, 1.0, 0.0, 0.0, 4.0, 16.0, 15.0, 10.0, 8.0, 1.0, 0.0, 0.0, 4.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 13.0, 16.0, 2.0, 0.0, 0.0, 0.0, 2.0, 15.0, 16.0, 9.0, 0.0, 0.0, 0.0], [0.0, 0.0, 13.0, 14.0, 9.0, 1.0, 0.0, 0.0, 0.0, 5.0, 16.0, 12.0, 12.0, 12.0, 1.0, 0.0, 0.0, 6.0, 16.0, 2.0, 2.0, 16.0, 5.0, 0.0, 0.0, 3.0, 16.0, 5.0, 3.0, 16.0, 9.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 6.0, 7.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 5.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 12.0, 0.0, 0.0], [0.0, 0.0, 1.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 15.0, 2.0, 0.0, 4.0, 0.0, 0.0, 2.0, 16.0, 6.0, 0.0, 7.0, 16.0, 2.0, 0.0, 8.0, 16.0, 6.0, 6.0, 16.0, 12.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 1.0, 4.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 8.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 14.0, 1.0, 6.0, 0.0, 0.0, 0.0, 7.0, 16.0, 5.0, 3.0, 16.0, 8.0, 0.0, 0.0, 8.0, 16.0, 8.0, 14.0, 16.0, 2.0, 0.0, 0.0, 0.0, 6.0, 14.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 0.0, 0.0, 0.0], [0.0, 0.0, 8.0, 12.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 15.0, 6.0, 10.0, 5.0, 0.0, 0.0, 0.0, 4.0, 12.0, 2.0, 8.0, 6.0, 0.0, 0.0, 0.0, 8.0, 14.0, 14.0, 8.0, 13.0, 5.0, 0.0, 0.0, 3.0, 7.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 2.0, 0.0, 0.0, 0.0, 5.0, 2.0, 5.0, 12.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 15.0, 2.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 6.0, 2.0, 2.0, 0.0, 0.0, 0.0, 7.0, 15.0, 0.0, 9.0, 8.0, 0.0, 0.0, 5.0, 16.0, 10.0, 0.0, 16.0, 6.0, 0.0, 0.0, 4.0, 15.0, 16.0, 13.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 4.0, 0.0, 0.0], [0.0, 0.0, 3.0, 13.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 14.0, 8.0, 7.0, 15.0, 1.0, 0.0, 0.0, 3.0, 16.0, 0.0, 0.0, 9.0, 6.0, 0.0, 0.0, 6.0, 13.0, 0.0, 0.0, 4.0, 8.0, 0.0, 0.0, 4.0, 9.0, 0.0, 0.0, 4.0, 8.0, 0.0, 0.0, 1.0, 13.0, 0.0, 0.0, 5.0, 8.0, 0.0, 0.0, 0.0, 14.0, 7.0, 0.0, 11.0, 4.0, 0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 14.0, 0.0, 0.0], [0.0, 0.0, 8.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 6.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 7.0, 0.0, 0.0, 0.0, 0.0, 4.0, 8.0, 14.0, 14.0, 8.0, 4.0, 0.0, 0.0, 9.0, 16.0, 16.0, 16.0, 16.0, 13.0], [0.0, 3.0, 15.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 14.0, 13.0, 10.0, 16.0, 2.0, 0.0, 0.0, 0.0, 5.0, 3.0, 2.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 8.0, 0.0, 2.0, 0.0, 0.0, 0.0, 8.0, 16.0, 14.0, 16.0, 15.0, 0.0, 0.0, 2.0, 16.0, 16.0, 15.0, 12.0, 9.0, 0.0], [0.0, 0.0, 0.0, 11.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 11.0, 2.0, 0.0, 0.0, 0.0, 3.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 12.0, 4.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 13.0, 15.0, 11.0, 0.0, 0.0, 0.0, 1.0, 15.0, 8.0, 3.0, 16.0, 5.0, 0.0, 0.0, 0.0, 9.0, 14.0, 5.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 10.0, 0.0], [0.0, 0.0, 3.0, 11.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 10.0, 12.0, 5.0, 16.0, 0.0, 0.0, 0.0, 0.0, 7.0, 10.0, 6.0, 15.0, 4.0, 0.0, 0.0, 0.0, 2.0, 13.0, 16.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 11.0, 0.0, 0.0, 0.0, 9.0, 7.0, 0.0, 8.0, 11.0, 0.0, 0.0, 0.0, 3.0, 9.0, 16.0, 16.0, 6.0, 0.0], [0.0, 1.0, 13.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 12.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 9.0, 6.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 4.0, 1.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 6.0, 16.0, 5.0, 0.0, 0.0, 8.0, 12.0, 13.0, 16.0, 16.0, 11.0], [0.0, 0.0, 10.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 16.0, 12.0, 5.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 9.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 14.0, 2.0, 0.0, 0.0, 1.0, 10.0, 4.0, 1.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 7.0, 0.0, 0.0, 0.0, 7.0, 8.0, 14.0, 16.0, 3.0, 0.0, 0.0, 0.0, 6.0, 13.0, 10.0, 4.0, 0.0, 0.0], [0.0, 0.0, 10.0, 11.0, 7.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 4.0, 16.0, 6.0, 5.0, 15.0, 2.0, 0.0, 0.0, 8.0, 12.0, 0.0, 0.0, 5.0, 8.0, 0.0, 0.0, 8.0, 10.0, 0.0, 0.0, 5.0, 8.0, 0.0, 0.0, 6.0, 13.0, 1.0, 5.0, 14.0, 5.0, 0.0, 0.0, 0.0, 14.0, 13.0, 15.0, 11.0, 1.0, 0.0, 0.0, 0.0, 7.0, 12.0, 8.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 14.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 1.0, 16.0, 15.0, 8.0, 14.0, 9.0, 0.0, 0.0, 0.0, 14.0, 12.0, 0.0, 12.0, 13.0, 0.0, 0.0, 0.0, 6.0, 14.0, 7.0, 16.0, 10.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 13.0, 1.0, 0.0], [0.0, 0.0, 6.0, 13.0, 12.0, 2.0, 0.0, 0.0, 0.0, 0.0, 7.0, 7.0, 10.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 1.0, 0.0, 0.0, 1.0, 7.0, 0.0, 0.0, 7.0, 11.0, 0.0, 0.0, 1.0, 16.0, 4.0, 0.0, 9.0, 11.0, 0.0, 0.0, 0.0, 5.0, 13.0, 12.0, 16.0, 3.0, 0.0], [0.0, 0.0, 3.0, 8.0, 11.0, 13.0, 14.0, 0.0, 0.0, 2.0, 13.0, 16.0, 13.0, 13.0, 13.0, 0.0, 0.0, 1.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 11.0, 10.0, 1.0, 0.0, 0.0, 0.0, 3.0, 16.0, 14.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 8.0, 3.0, 9.0, 11.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 2.0, 12.0, 13.0, 2.0, 0.0, 0.0], [0.0, 0.0, 2.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 5.0, 0.0, 2.0, 0.0, 0.0, 2.0, 15.0, 10.0, 0.0, 11.0, 16.0, 1.0, 0.0, 10.0, 16.0, 4.0, 6.0, 16.0, 10.0, 0.0, 0.0, 6.0, 16.0, 16.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 6.0, 13.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 5.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.0, 10.0, 12.0, 12.0, 2.0, 0.0, 0.0, 1.0, 13.0, 12.0, 6.0, 13.0, 8.0, 0.0, 0.0, 8.0, 16.0, 8.0, 8.0, 14.0, 1.0, 0.0, 0.0, 5.0, 14.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 3.0, 16.0, 2.0, 15.0, 10.0, 0.0, 0.0, 0.0, 4.0, 16.0, 8.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 15.0, 3.0, 0.0, 0.0], [0.0, 0.0, 0.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 3.0, 11.0, 7.0, 0.0, 0.0, 0.0, 12.0, 16.0, 8.0, 16.0, 9.0, 1.0, 0.0, 0.0, 10.0, 16.0, 16.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 11.0, 0.0, 0.0, 0.0], [0.0, 3.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 15.0, 3.0, 1.0, 1.0, 0.0, 0.0, 0.0, 6.0, 16.0, 4.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 14.0, 14.0, 13.0, 1.0, 0.0, 0.0, 0.0, 2.0, 15.0, 13.0, 1.0, 0.0, 0.0, 0.0, 1.0, 14.0, 8.0, 15.0, 0.0, 0.0, 0.0, 0.0, 5.0, 12.0, 0.0, 12.0, 3.0, 0.0, 0.0, 0.0, 2.0, 14.0, 13.0, 15.0, 3.0, 0.0, 0.0], [0.0, 0.0, 15.0, 16.0, 13.0, 6.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 9.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 12.0, 1.0, 0.0, 0.0, 1.0, 8.0, 4.0, 9.0, 16.0, 3.0, 0.0, 0.0, 5.0, 14.0, 7.0, 10.0, 15.0, 1.0, 0.0, 0.0, 2.0, 12.0, 16.0, 14.0, 6.0, 0.0, 0.0], [0.0, 0.0, 0.0, 7.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 2.0, 0.0, 3.0, 1.0, 0.0, 0.0, 8.0, 10.0, 0.0, 2.0, 16.0, 2.0, 0.0, 1.0, 15.0, 4.0, 3.0, 9.0, 12.0, 0.0, 0.0, 8.0, 16.0, 16.0, 16.0, 16.0, 6.0, 0.0, 0.0, 1.0, 4.0, 3.0, 9.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 10.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 16.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 3.0, 12.0, 14.0, 11.0, 0.0, 0.0], [0.0, 0.0, 0.0, 8.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 10.0, 0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 16.0, 4.0, 11.0, 11.0, 0.0, 0.0, 0.0, 11.0, 15.0, 2.0, 14.0, 10.0, 1.0, 0.0, 0.0, 13.0, 16.0, 16.0, 16.0, 13.0, 1.0, 0.0, 0.0, 0.0, 4.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 9.0, 0.0, 0.0, 0.0], [0.0, 0.0, 14.0, 16.0, 13.0, 9.0, 1.0, 0.0, 0.0, 0.0, 12.0, 10.0, 9.0, 16.0, 3.0, 0.0, 0.0, 0.0, 9.0, 14.0, 13.0, 13.0, 1.0, 0.0, 0.0, 0.0, 3.0, 13.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 12.0, 9.0, 0.0, 0.0, 0.0, 0.0, 13.0, 8.0, 14.0, 6.0, 0.0, 0.0, 0.0, 1.0, 16.0, 11.0, 15.0, 1.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 6.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 13.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 7.0, 16.0, 3.0, 0.0, 0.0, 1.0, 14.0, 3.0, 0.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 10.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 4.0, 0.0, 0.0, 0.0], [0.0, 2.0, 15.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 8.0, 14.0, 8.0, 14.0, 8.0, 0.0, 0.0, 0.0, 7.0, 5.0, 2.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 9.0, 8.0, 8.0, 2.0, 0.0, 0.0, 2.0, 15.0, 16.0, 16.0, 16.0, 13.0, 0.0], [0.0, 1.0, 11.0, 12.0, 14.0, 6.0, 0.0, 0.0, 0.0, 1.0, 6.0, 4.0, 8.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 14.0, 0.0, 0.0, 0.0, 5.0, 12.0, 15.0, 9.0, 1.0, 0.0, 0.0], [0.0, 0.0, 4.0, 12.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 11.0, 0.0, 0.0], [0.0, 1.0, 11.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 12.0, 9.0, 0.0, 0.0, 0.0, 0.0, 7.0, 8.0, 7.0, 12.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 4.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 9.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 16.0, 16.0, 15.0, 10.0, 0.0, 0.0, 0.0, 13.0, 11.0, 8.0, 12.0, 8.0, 0.0], [0.0, 0.0, 9.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 5.0, 16.0, 14.0, 15.0, 16.0, 1.0, 0.0, 0.0, 2.0, 11.0, 1.0, 10.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 7.0, 8.0, 7.0, 0.0, 0.0, 0.0, 9.0, 16.0, 15.0, 14.0, 2.0, 0.0, 0.0, 0.0, 9.0, 16.0, 13.0, 1.0, 0.0, 0.0], [0.0, 0.0, 7.0, 11.0, 12.0, 14.0, 2.0, 0.0, 0.0, 8.0, 16.0, 9.0, 4.0, 3.0, 0.0, 0.0, 0.0, 10.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 12.0, 16.0, 14.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 9.0, 0.0, 0.0, 0.0, 2.0, 4.0, 8.0, 15.0, 9.0, 0.0, 0.0, 0.0, 10.0, 16.0, 13.0, 8.0, 0.0, 0.0], [0.0, 0.0, 9.0, 16.0, 16.0, 13.0, 2.0, 0.0, 0.0, 2.0, 15.0, 2.0, 3.0, 3.0, 0.0, 0.0, 0.0, 7.0, 9.0, 0.0, 1.0, 4.0, 0.0, 0.0, 0.0, 8.0, 12.0, 7.0, 13.0, 14.0, 7.0, 0.0, 0.0, 6.0, 16.0, 8.0, 0.0, 5.0, 8.0, 0.0, 0.0, 1.0, 3.0, 0.0, 0.0, 9.0, 6.0, 0.0, 0.0, 0.0, 3.0, 4.0, 1.0, 15.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 12.0, 7.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 11.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 15.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 16.0, 13.0, 0.0, 0.0, 0.0, 6.0, 16.0, 9.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 14.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 12.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 16.0, 11.0, 0.0, 0.0, 0.0, 2.0, 12.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 6.0, 11.0, 5.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 8.0, 0.0, 0.0], [0.0, 0.0, 4.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 11.0, 8.0, 0.0, 0.0, 0.0, 3.0, 16.0, 13.0, 1.0, 14.0, 2.0, 0.0, 0.0, 4.0, 16.0, 0.0, 0.0, 13.0, 4.0, 0.0, 0.0, 4.0, 15.0, 0.0, 0.0, 13.0, 8.0, 0.0, 0.0, 1.0, 16.0, 1.0, 0.0, 14.0, 5.0, 0.0, 0.0, 0.0, 10.0, 8.0, 7.0, 15.0, 1.0, 0.0, 0.0, 0.0, 2.0, 13.0, 16.0, 8.0, 0.0, 0.0], [0.0, 0.0, 0.0, 4.0, 12.0, 15.0, 4.0, 0.0, 0.0, 0.0, 3.0, 14.0, 4.0, 10.0, 8.0, 0.0, 0.0, 0.0, 4.0, 12.0, 5.0, 14.0, 2.0, 0.0, 0.0, 0.0, 4.0, 16.0, 14.0, 3.0, 0.0, 0.0, 0.0, 1.0, 12.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 6.0, 13.0, 1.0, 12.0, 6.0, 0.0, 0.0, 0.0, 0.0, 10.0, 13.0, 5.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 10.0, 16.0, 8.0, 0.0], [0.0, 0.0, 9.0, 13.0, 15.0, 10.0, 1.0, 0.0, 0.0, 1.0, 15.0, 5.0, 7.0, 16.0, 5.0, 0.0, 0.0, 0.0, 15.0, 7.0, 10.0, 16.0, 6.0, 0.0, 0.0, 0.0, 2.0, 7.0, 7.0, 10.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 5.0, 0.0, 0.0, 0.0, 14.0, 6.0, 10.0, 12.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 12.0, 1.0, 0.0, 0.0], [0.0, 0.0, 5.0, 15.0, 14.0, 3.0, 0.0, 0.0, 0.0, 2.0, 14.0, 7.0, 4.0, 13.0, 0.0, 0.0, 0.0, 2.0, 15.0, 5.0, 5.0, 16.0, 1.0, 0.0, 0.0, 0.0, 7.0, 15.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 3.0, 7.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 0.0, 0.0, 0.0, 8.0, 9.0, 4.0, 2.0, 16.0, 1.0, 0.0, 0.0, 4.0, 11.0, 13.0, 16.0, 11.0, 0.0], [0.0, 0.0, 2.0, 15.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 9.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 14.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 10.0, 5.0, 0.0, 0.0, 0.0, 1.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 12.0, 5.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 1.0, 15.0, 7.0, 4.0, 16.0, 4.0, 0.0, 0.0, 0.0, 11.0, 13.0, 4.0, 16.0, 9.0, 0.0, 0.0, 0.0, 1.0, 12.0, 16.0, 16.0, 9.0, 0.0], [0.0, 0.0, 2.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 3.0, 14.0, 3.0, 7.0, 16.0, 3.0, 0.0, 0.0, 7.0, 8.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 5.0, 8.0, 0.0, 0.0, 4.0, 8.0, 0.0, 0.0, 4.0, 12.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 0.0, 14.0, 9.0, 8.0, 16.0, 2.0, 0.0, 0.0, 0.0, 3.0, 14.0, 15.0, 6.0, 0.0, 0.0], [0.0, 1.0, 8.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 6.0, 16.0, 12.0, 16.0, 4.0, 0.0, 0.0, 0.0, 1.0, 7.0, 0.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 14.0, 1.0, 4.0, 5.0, 0.0, 0.0, 0.0, 13.0, 12.0, 11.0, 15.0, 3.0, 0.0, 0.0, 0.0, 12.0, 16.0, 12.0, 3.0, 0.0, 0.0], [0.0, 0.0, 9.0, 12.0, 12.0, 13.0, 7.0, 0.0, 0.0, 0.0, 15.0, 5.0, 5.0, 4.0, 2.0, 0.0, 0.0, 4.0, 15.0, 10.0, 4.0, 0.0, 0.0, 0.0, 0.0, 2.0, 11.0, 11.0, 15.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 7.0, 0.0, 0.0, 5.0, 11.0, 4.0, 5.0, 14.0, 1.0, 0.0, 0.0, 0.0, 9.0, 16.0, 13.0, 3.0, 0.0, 0.0], [0.0, 0.0, 3.0, 9.0, 14.0, 9.0, 0.0, 0.0, 0.0, 5.0, 16.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 11.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 7.0, 0.0, 0.0, 0.0], [0.0, 0.0, 11.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 2.0, 16.0, 10.0, 11.0, 15.0, 0.0, 0.0, 0.0, 0.0, 15.0, 4.0, 4.0, 16.0, 3.0, 0.0, 0.0, 0.0, 3.0, 3.0, 5.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 2.0, 12.0, 15.0, 6.0, 4.0, 1.0, 0.0, 0.0, 10.0, 16.0, 16.0, 16.0, 16.0, 10.0], [0.0, 0.0, 2.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 0.0, 6.0, 7.0, 2.0, 0.0, 0.0, 4.0, 12.0, 13.0, 15.0, 14.0, 12.0, 0.0, 0.0, 0.0, 13.0, 12.0, 2.0, 11.0, 14.0, 0.0, 0.0, 0.0, 3.0, 13.0, 16.0, 13.0, 1.0, 0.0], [0.0, 0.0, 10.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 8.0, 16.0, 1.0, 6.0, 16.0, 5.0, 0.0, 0.0, 8.0, 11.0, 0.0, 0.0, 9.0, 12.0, 0.0, 0.0, 10.0, 8.0, 0.0, 0.0, 8.0, 12.0, 0.0, 0.0, 8.0, 11.0, 0.0, 0.0, 8.0, 11.0, 0.0, 0.0, 3.0, 16.0, 10.0, 8.0, 15.0, 9.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 10.0, 1.0, 0.0], [0.0, 0.0, 8.0, 16.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 10.0, 11.0, 9.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 3.0, 0.0, 0.0, 0.0, 3.0, 12.0, 2.0, 14.0, 4.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 5.0, 0.0, 0.0], [0.0, 0.0, 8.0, 12.0, 15.0, 16.0, 5.0, 0.0, 0.0, 0.0, 10.0, 11.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 10.0, 12.0, 11.0, 1.0, 0.0, 0.0, 1.0, 16.0, 13.0, 8.0, 14.0, 7.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 13.0, 3.0, 0.0, 0.0, 0.0, 1.0, 6.0, 12.0, 10.0, 0.0, 0.0, 0.0, 0.0, 10.0, 10.0, 7.0, 0.0, 0.0, 0.0], [0.0, 0.0, 7.0, 13.0, 10.0, 1.0, 0.0, 0.0, 0.0, 1.0, 15.0, 3.0, 9.0, 10.0, 0.0, 0.0, 0.0, 3.0, 16.0, 4.0, 13.0, 11.0, 0.0, 0.0, 0.0, 0.0, 6.0, 12.0, 12.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 11.0, 0.0, 0.0, 1.0, 11.0, 2.0, 0.0, 7.0, 11.0, 0.0, 0.0, 0.0, 7.0, 13.0, 16.0, 15.0, 4.0, 0.0], [0.0, 0.0, 6.0, 15.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 13.0, 14.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0, 14.0, 11.0, 13.0, 15.0, 5.0, 0.0, 0.0, 0.0, 9.0, 16.0, 15.0, 8.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 8.0, 16.0, 2.0, 14.0, 10.0, 0.0, 0.0, 0.0, 5.0, 16.0, 9.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 13.0, 2.0, 0.0, 0.0], [0.0, 0.0, 0.0, 6.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 12.0, 7.0, 12.0, 2.0, 0.0, 0.0, 0.0, 9.0, 16.0, 15.0, 13.0, 6.0, 0.0, 0.0, 0.0, 13.0, 14.0, 2.0, 13.0, 6.0, 0.0, 0.0, 0.0, 14.0, 15.0, 13.0, 16.0, 4.0, 0.0, 0.0, 0.0, 1.0, 10.0, 16.0, 10.0, 0.0, 0.0], [0.0, 0.0, 1.0, 13.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 10.0, 12.0, 12.0, 3.0, 0.0, 0.0, 3.0, 16.0, 11.0, 5.0, 9.0, 12.0, 0.0, 0.0, 1.0, 13.0, 11.0, 4.0, 13.0, 11.0, 0.0, 0.0, 0.0, 1.0, 12.0, 16.0, 11.0, 2.0, 0.0], [0.0, 0.0, 7.0, 12.0, 13.0, 4.0, 0.0, 0.0, 0.0, 0.0, 16.0, 6.0, 6.0, 2.0, 0.0, 0.0, 0.0, 4.0, 13.0, 7.0, 8.0, 2.0, 0.0, 0.0, 0.0, 7.0, 16.0, 10.0, 10.0, 14.0, 1.0, 0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 10.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 0.0, 11.0, 1.0, 0.0, 10.0, 8.0, 0.0, 0.0, 0.0, 8.0, 15.0, 15.0, 15.0, 2.0, 0.0], [0.0, 0.0, 6.0, 12.0, 12.0, 6.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 2.0, 13.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 2.0, 11.0, 10.0, 4.0, 0.0, 0.0], [0.0, 0.0, 0.0, 12.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 3.0, 12.0, 4.0, 0.0, 0.0, 1.0, 16.0, 5.0, 8.0, 14.0, 0.0, 0.0, 0.0, 9.0, 15.0, 0.0, 13.0, 10.0, 2.0, 0.0, 0.0, 10.0, 15.0, 12.0, 16.0, 16.0, 9.0, 0.0, 0.0, 6.0, 16.0, 16.0, 15.0, 9.0, 1.0, 0.0, 0.0, 0.0, 0.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 3.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 16.0, 13.0, 16.0, 16.0, 6.0, 0.0, 0.0, 1.0, 16.0, 11.0, 4.0, 7.0, 12.0, 0.0, 0.0, 0.0, 11.0, 12.0, 5.0, 13.0, 9.0, 0.0, 0.0, 0.0, 1.0, 12.0, 15.0, 11.0, 2.0, 0.0], [0.0, 0.0, 1.0, 9.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 14.0, 11.0, 5.0, 11.0, 0.0, 0.0, 0.0, 4.0, 15.0, 1.0, 4.0, 14.0, 0.0, 0.0, 0.0, 0.0, 6.0, 1.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 10.0, 0.0, 0.0, 0.0, 12.0, 9.0, 4.0, 4.0, 15.0, 0.0, 0.0, 0.0, 1.0, 10.0, 16.0, 15.0, 11.0, 1.0], [0.0, 0.0, 2.0, 12.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 13.0, 13.0, 15.0, 12.0, 0.0, 0.0, 0.0, 5.0, 15.0, 2.0, 10.0, 6.0, 0.0, 0.0, 0.0, 2.0, 14.0, 13.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 13.0, 1.0, 0.0, 0.0, 0.0, 1.0, 15.0, 3.0, 14.0, 7.0, 0.0, 0.0, 0.0, 6.0, 13.0, 1.0, 16.0, 4.0, 0.0, 0.0, 0.0, 1.0, 12.0, 16.0, 11.0, 0.0, 0.0], [0.0, 0.0, 4.0, 10.0, 11.0, 4.0, 0.0, 0.0, 0.0, 1.0, 11.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 5.0, 12.0, 12.0, 12.0, 1.0, 0.0], [0.0, 0.0, 0.0, 8.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 16.0, 16.0, 14.0, 1.0, 0.0, 0.0, 4.0, 16.0, 1.0, 4.0, 15.0, 6.0, 0.0, 0.0, 0.0, 5.0, 14.0, 15.0, 10.0, 0.0], [0.0, 0.0, 8.0, 14.0, 12.0, 3.0, 0.0, 0.0, 0.0, 6.0, 16.0, 6.0, 14.0, 14.0, 0.0, 0.0, 0.0, 6.0, 13.0, 0.0, 8.0, 14.0, 0.0, 0.0, 0.0, 2.0, 14.0, 14.0, 14.0, 16.0, 3.0, 0.0, 0.0, 0.0, 2.0, 4.0, 6.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 3.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 8.0, 0.0, 0.0], [0.0, 0.0, 10.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 4.0, 16.0, 14.0, 10.0, 8.0, 0.0, 0.0, 0.0, 13.0, 16.0, 8.0, 1.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 13.0, 2.0, 0.0, 0.0, 0.0, 3.0, 8.0, 9.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 5.0, 12.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 5.0, 0.0, 0.0, 0.0], [0.0, 0.0, 6.0, 12.0, 15.0, 9.0, 1.0, 0.0, 0.0, 5.0, 14.0, 4.0, 5.0, 16.0, 3.0, 0.0, 0.0, 9.0, 8.0, 3.0, 13.0, 16.0, 4.0, 0.0, 0.0, 3.0, 15.0, 15.0, 7.0, 10.0, 8.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 12.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 2.0, 0.0, 0.0, 0.0, 7.0, 4.0, 5.0, 13.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 15.0, 4.0, 0.0, 0.0], [0.0, 0.0, 12.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 3.0, 16.0, 10.0, 2.0, 2.0, 0.0, 0.0, 0.0, 4.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 12.0, 12.0, 9.0, 1.0, 0.0, 0.0, 1.0, 15.0, 16.0, 12.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 11.0, 0.0, 0.0, 0.0, 3.0, 9.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 10.0, 12.0, 12.0, 4.0, 0.0, 0.0], [0.0, 0.0, 4.0, 13.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 15.0, 10.0, 7.0, 16.0, 1.0, 0.0, 0.0, 7.0, 12.0, 0.0, 12.0, 7.0, 0.0, 0.0, 0.0, 9.0, 5.0, 3.0, 16.0, 2.0, 0.0, 0.0, 0.0, 2.0, 11.0, 16.0, 16.0, 12.0, 7.0, 0.0, 0.0, 5.0, 10.0, 16.0, 12.0, 8.0, 3.0, 0.0, 0.0, 0.0, 3.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 11.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 12.0, 14.0, 10.0, 16.0, 5.0, 0.0, 0.0, 0.0, 16.0, 7.0, 13.0, 16.0, 4.0, 0.0, 0.0, 0.0, 9.0, 15.0, 13.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 4.0, 0.0, 0.0, 8.0, 16.0, 14.0, 9.0, 16.0, 4.0, 0.0, 0.0, 0.0, 2.0, 10.0, 15.0, 15.0, 2.0, 0.0], [0.0, 0.0, 0.0, 1.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 13.0, 0.0, 9.0, 7.0, 0.0, 0.0, 2.0, 15.0, 4.0, 0.0, 15.0, 5.0, 0.0, 2.0, 13.0, 14.0, 11.0, 10.0, 15.0, 0.0, 0.0, 11.0, 15.0, 13.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 6.0, 0.0, 0.0], [0.0, 0.0, 5.0, 14.0, 14.0, 2.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 7.0, 4.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 1.0, 9.0, 16.0, 16.0, 8.0, 0.0, 0.0, 3.0, 15.0, 16.0, 16.0, 10.0, 2.0, 0.0, 0.0, 4.0, 16.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 13.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 13.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 10.0, 13.0, 5.0, 16.0, 0.0, 0.0, 0.0, 0.0, 9.0, 9.0, 4.0, 14.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 14.0, 6.0, 8.0, 9.0, 0.0, 0.0, 0.0, 0.0, 13.0, 7.0, 1.0, 11.0, 5.0, 0.0, 0.0, 0.0, 3.0, 11.0, 15.0, 16.0, 12.0, 0.0], [0.0, 0.0, 3.0, 12.0, 15.0, 14.0, 3.0, 0.0, 0.0, 1.0, 16.0, 5.0, 0.0, 8.0, 12.0, 0.0, 0.0, 6.0, 16.0, 11.0, 2.0, 13.0, 7.0, 0.0, 0.0, 2.0, 9.0, 15.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 9.0, 5.0, 6.0, 15.0, 0.0, 0.0, 0.0, 0.0, 11.0, 10.0, 7.0, 16.0, 2.0, 0.0, 0.0, 0.0, 3.0, 12.0, 16.0, 13.0, 0.0, 0.0], [0.0, 0.0, 10.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 13.0, 11.0, 11.0, 0.0, 0.0, 0.0, 10.0, 16.0, 12.0, 15.0, 16.0, 4.0, 0.0, 0.0, 3.0, 12.0, 12.0, 14.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 4.0, 0.0, 0.0, 0.0, 4.0, 4.0, 15.0, 15.0, 0.0, 0.0, 0.0, 1.0, 12.0, 15.0, 12.0, 3.0, 0.0, 0.0], [0.0, 0.0, 2.0, 13.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 2.0, 0.0, 4.0, 0.0, 0.0, 5.0, 16.0, 10.0, 1.0, 13.0, 15.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 8.0, 15.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 11.0, 0.0, 0.0, 0.0], [0.0, 2.0, 15.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 13.0, 15.0, 0.0, 0.0, 0.0, 0.0, 11.0, 11.0, 5.0, 16.0, 4.0, 0.0, 0.0, 0.0, 1.0, 1.0, 7.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 8.0, 8.0, 2.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 16.0, 11.0, 0.0, 0.0, 2.0, 15.0, 13.0, 6.0, 4.0, 1.0, 0.0], [0.0, 0.0, 1.0, 10.0, 13.0, 12.0, 5.0, 0.0, 0.0, 0.0, 13.0, 13.0, 4.0, 4.0, 12.0, 0.0, 0.0, 3.0, 16.0, 7.0, 4.0, 12.0, 6.0, 0.0, 0.0, 2.0, 15.0, 16.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 1.0, 9.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 13.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 14.0, 11.0, 9.0, 0.0, 0.0, 0.0, 0.0, 15.0, 7.0, 6.0, 12.0, 0.0, 0.0, 0.0, 0.0, 8.0, 5.0, 9.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 12.0, 4.0, 4.0, 1.0, 0.0, 0.0, 1.0, 15.0, 16.0, 16.0, 16.0, 7.0], [0.0, 2.0, 16.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 1.0, 9.0, 5.0, 12.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 14.0, 2.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 5.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 13.0, 1.0, 0.0, 0.0], [0.0, 2.0, 16.0, 12.0, 12.0, 14.0, 7.0, 0.0, 0.0, 3.0, 16.0, 9.0, 8.0, 8.0, 4.0, 0.0, 0.0, 2.0, 16.0, 10.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 13.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 12.0, 10.0, 0.0, 0.0, 0.0, 6.0, 3.0, 0.0, 9.0, 11.0, 0.0, 0.0, 0.0, 11.0, 11.0, 9.0, 16.0, 3.0, 0.0, 0.0, 0.0, 3.0, 12.0, 15.0, 7.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 12.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 9.0, 2.0, 0.0, 0.0, 0.0, 0.0, 12.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 11.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 14.0, 15.0, 8.0, 0.0, 0.0, 0.0, 1.0, 16.0, 6.0, 0.0, 12.0, 8.0, 0.0, 0.0, 0.0, 12.0, 12.0, 4.0, 13.0, 12.0, 0.0, 0.0, 0.0, 1.0, 11.0, 16.0, 16.0, 4.0, 0.0], [0.0, 0.0, 8.0, 12.0, 11.0, 6.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 13.0, 2.0, 0.0, 0.0, 2.0, 14.0, 16.0, 16.0, 14.0, 2.0, 0.0, 0.0, 2.0, 13.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 1.0, 11.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 4.0, 11.0, 12.0, 7.0, 0.0, 0.0], [0.0, 4.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 5.0, 8.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 3.0, 6.0, 9.0, 0.0, 0.0, 3.0, 15.0, 15.0, 8.0, 13.0, 15.0, 0.0, 0.0, 4.0, 15.0, 16.0, 16.0, 16.0, 7.0, 0.0], [0.0, 0.0, 0.0, 11.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 12.0, 8.0, 2.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 12.0, 14.0, 3.0, 0.0, 0.0, 4.0, 16.0, 8.0, 0.0, 10.0, 9.0, 0.0, 0.0, 1.0, 12.0, 15.0, 9.0, 14.0, 10.0, 0.0, 0.0, 0.0, 2.0, 10.0, 13.0, 11.0, 1.0, 0.0], [0.0, 0.0, 0.0, 2.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 2.0, 4.0, 10.0, 12.0, 0.0, 0.0, 0.0, 3.0, 15.0, 14.0, 10.0, 8.0, 0.0, 0.0, 0.0, 8.0, 15.0, 1.0, 11.0, 4.0, 0.0, 0.0, 0.0, 1.0, 8.0, 15.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 11.0, 0.0, 0.0], [0.0, 1.0, 8.0, 13.0, 16.0, 13.0, 0.0, 0.0, 0.0, 1.0, 13.0, 12.0, 15.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 16.0, 4.0, 0.0, 0.0, 0.0, 13.0, 16.0, 15.0, 7.0, 0.0, 0.0], [0.0, 0.0, 0.0, 13.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 15.0, 16.0, 14.0, 5.0, 0.0, 0.0, 0.0, 13.0, 7.0, 0.0, 0.0, 13.0, 1.0, 0.0, 0.0, 10.0, 6.0, 0.0, 5.0, 14.0, 0.0, 0.0, 0.0, 2.0, 13.0, 12.0, 15.0, 4.0, 0.0], [0.0, 0.0, 5.0, 13.0, 15.0, 6.0, 0.0, 0.0, 0.0, 2.0, 16.0, 9.0, 16.0, 13.0, 0.0, 0.0, 0.0, 4.0, 14.0, 0.0, 10.0, 16.0, 2.0, 0.0, 0.0, 4.0, 15.0, 11.0, 15.0, 16.0, 1.0, 0.0, 0.0, 0.0, 7.0, 10.0, 3.0, 13.0, 8.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 12.0, 5.0, 0.0, 0.0, 0.0, 13.0, 11.0, 4.0, 16.0, 4.0, 0.0, 0.0, 0.0, 7.0, 14.0, 16.0, 11.0, 1.0, 0.0], [0.0, 0.0, 0.0, 2.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 10.0, 5.0, 7.0, 0.0, 0.0, 0.0, 11.0, 15.0, 2.0, 13.0, 7.0, 0.0, 0.0, 10.0, 16.0, 8.0, 8.0, 16.0, 6.0, 0.0, 0.0, 8.0, 12.0, 12.0, 13.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 3.0, 0.0, 0.0], [0.0, 0.0, 2.0, 10.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 14.0, 9.0, 6.0, 16.0, 16.0, 0.0, 0.0, 0.0, 16.0, 6.0, 5.0, 14.0, 11.0, 0.0, 0.0, 0.0, 5.0, 14.0, 14.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 3.0, 0.0, 0.0, 0.0, 3.0, 1.0, 4.0, 16.0, 3.0, 0.0, 0.0, 2.0, 15.0, 13.0, 11.0, 13.0, 1.0, 0.0, 0.0, 0.0, 3.0, 12.0, 13.0, 4.0, 0.0, 0.0], [0.0, 0.0, 3.0, 11.0, 16.0, 13.0, 0.0, 0.0, 0.0, 3.0, 15.0, 15.0, 13.0, 16.0, 0.0, 0.0, 0.0, 6.0, 8.0, 2.0, 9.0, 14.0, 0.0, 0.0, 0.0, 0.0, 4.0, 7.0, 15.0, 14.0, 5.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 14.0, 6.0, 0.0, 0.0, 1.0, 8.0, 13.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 5.0, 12.0, 1.0, 6.0, 0.0, 0.0, 0.0, 0.0, 11.0, 12.0, 0.0, 16.0, 2.0, 0.0, 0.0, 0.0, 16.0, 5.0, 0.0, 12.0, 4.0, 0.0, 0.0, 3.0, 15.0, 0.0, 0.0, 8.0, 4.0, 0.0, 0.0, 7.0, 12.0, 0.0, 0.0, 4.0, 7.0, 0.0, 0.0, 2.0, 15.0, 1.0, 1.0, 12.0, 5.0, 0.0, 0.0, 0.0, 16.0, 11.0, 12.0, 15.0, 3.0, 0.0, 0.0, 0.0, 4.0, 12.0, 12.0, 3.0, 0.0, 0.0], [0.0, 0.0, 15.0, 16.0, 11.0, 3.0, 0.0, 0.0, 0.0, 0.0, 4.0, 10.0, 15.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 16.0, 9.0, 0.0, 0.0, 0.0, 6.0, 15.0, 16.0, 12.0, 3.0, 0.0, 0.0, 0.0, 15.0, 14.0, 7.0, 1.0, 0.0, 0.0], [0.0, 1.0, 11.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 10.0, 9.0, 6.0, 14.0, 0.0, 0.0, 0.0, 0.0, 3.0, 10.0, 4.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 16.0, 13.0, 11.0, 2.0, 0.0, 0.0, 1.0, 12.0, 12.0, 12.0, 15.0, 11.0, 0.0], [0.0, 0.0, 3.0, 13.0, 16.0, 5.0, 0.0, 0.0, 0.0, 6.0, 15.0, 9.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 2.0, 15.0, 10.0, 0.0, 0.0, 0.0, 4.0, 5.0, 12.0, 16.0, 14.0, 0.0, 0.0, 0.0, 5.0, 13.0, 14.0, 8.0, 2.0, 0.0], [0.0, 1.0, 10.0, 15.0, 15.0, 3.0, 0.0, 0.0, 0.0, 6.0, 13.0, 4.0, 10.0, 12.0, 0.0, 0.0, 0.0, 4.0, 11.0, 0.0, 7.0, 15.0, 0.0, 0.0, 0.0, 2.0, 14.0, 16.0, 16.0, 14.0, 2.0, 0.0, 0.0, 0.0, 1.0, 4.0, 3.0, 10.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 12.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0, 2.0, 13.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 16.0, 11.0, 0.0], [0.0, 0.0, 0.0, 7.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 16.0, 16.0, 8.0, 0.0, 0.0, 8.0, 15.0, 15.0, 14.0, 8.0, 5.0, 0.0, 0.0, 0.0, 0.0, 11.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 1.0, 0.0, 0.0], [0.0, 0.0, 7.0, 13.0, 16.0, 5.0, 0.0, 0.0, 0.0, 6.0, 15.0, 7.0, 6.0, 14.0, 0.0, 0.0, 0.0, 9.0, 5.0, 1.0, 10.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 6.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 0.0, 0.0, 0.0, 4.0, 5.0, 2.0, 5.0, 13.0, 0.0, 0.0, 0.0, 6.0, 12.0, 16.0, 14.0, 5.0, 0.0], [0.0, 3.0, 14.0, 14.0, 16.0, 16.0, 10.0, 0.0, 0.0, 9.0, 15.0, 9.0, 7.0, 1.0, 0.0, 0.0, 0.0, 10.0, 16.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 14.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 11.0, 10.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 4.0, 0.0, 0.0, 0.0], [0.0, 1.0, 13.0, 16.0, 15.0, 5.0, 0.0, 0.0, 0.0, 4.0, 16.0, 7.0, 14.0, 12.0, 0.0, 0.0, 0.0, 3.0, 12.0, 2.0, 11.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 14.0, 16.0, 7.0, 0.0, 0.0, 1.0, 16.0, 16.0, 15.0, 12.0, 5.0, 0.0], [0.0, 1.0, 9.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 8.0, 11.0, 3.0, 7.0, 11.0, 1.0, 0.0, 0.0, 12.0, 7.0, 0.0, 3.0, 16.0, 4.0, 0.0, 0.0, 3.0, 13.0, 12.0, 14.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 1.0, 0.0, 0.0, 0.0, 8.0, 13.0, 14.0, 5.0, 0.0, 0.0], [0.0, 0.0, 11.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 3.0, 2.0, 14.0, 3.0, 0.0, 0.0, 9.0, 15.0, 0.0, 12.0, 15.0, 0.0, 0.0, 0.0, 6.0, 16.0, 15.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 10.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 8.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 1.0, 15.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 3.0, 12.0, 15.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 7.0, 0.0, 0.0], [0.0, 0.0, 0.0, 11.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 10.0, 4.0, 10.0, 0.0, 0.0, 0.0, 6.0, 15.0, 2.0, 15.0, 8.0, 0.0, 0.0, 0.0, 10.0, 12.0, 4.0, 16.0, 7.0, 6.0, 0.0, 0.0, 10.0, 16.0, 15.0, 16.0, 14.0, 6.0, 0.0, 0.0, 3.0, 8.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 11.0, 0.0, 0.0, 0.0], [0.0, 0.0, 5.0, 8.0, 11.0, 5.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 12.0, 12.0, 0.0, 0.0, 0.0, 1.0, 16.0, 9.0, 0.0, 9.0, 3.0, 0.0, 0.0, 3.0, 16.0, 6.0, 0.0, 6.0, 6.0, 0.0, 0.0, 3.0, 11.0, 1.0, 0.0, 5.0, 6.0, 0.0, 0.0, 0.0, 12.0, 0.0, 0.0, 11.0, 6.0, 0.0, 0.0, 0.0, 14.0, 5.0, 12.0, 15.0, 1.0, 0.0, 0.0, 0.0, 6.0, 16.0, 13.0, 2.0, 0.0, 0.0], [0.0, 0.0, 1.0, 5.0, 12.0, 13.0, 0.0, 0.0, 0.0, 0.0, 11.0, 13.0, 15.0, 16.0, 1.0, 0.0, 0.0, 2.0, 14.0, 0.0, 10.0, 12.0, 4.0, 0.0, 0.0, 5.0, 13.0, 12.0, 3.0, 12.0, 0.0, 0.0, 0.0, 0.0, 5.0, 6.0, 0.0, 12.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 2.0, 0.0, 0.0, 0.0, 4.0, 5.0, 0.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 13.0, 0.0, 0.0], [0.0, 0.0, 2.0, 16.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 14.0, 15.0, 0.0, 0.0, 0.0, 0.0, 15.0, 10.0, 0.0, 16.0, 7.0, 0.0, 0.0, 4.0, 16.0, 1.0, 0.0, 12.0, 5.0, 0.0, 0.0, 4.0, 15.0, 0.0, 0.0, 12.0, 5.0, 0.0, 0.0, 5.0, 16.0, 6.0, 0.0, 16.0, 0.0, 0.0, 0.0, 0.0, 14.0, 13.0, 8.0, 15.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 16.0, 6.0, 0.0, 0.0], [0.0, 0.0, 6.0, 14.0, 16.0, 15.0, 1.0, 0.0, 0.0, 9.0, 16.0, 12.0, 9.0, 16.0, 3.0, 0.0, 0.0, 12.0, 16.0, 11.0, 14.0, 13.0, 0.0, 0.0, 0.0, 7.0, 15.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 8.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 9.0, 16.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 7.0, 8.0, 10.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 4.0, 0.0, 0.0, 0.0, 5.0, 13.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 3.0, 5.0, 14.0, 15.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 11.0, 4.0, 0.0, 0.0], [0.0, 0.0, 4.0, 12.0, 13.0, 5.0, 0.0, 0.0, 0.0, 3.0, 15.0, 8.0, 10.0, 15.0, 2.0, 0.0, 0.0, 3.0, 14.0, 2.0, 2.0, 15.0, 3.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 13.0, 15.0, 3.0, 0.0, 0.0, 0.0, 2.0, 11.0, 1.0, 12.0, 5.0, 0.0, 0.0, 0.0, 7.0, 9.0, 1.0, 14.0, 2.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 7.0, 0.0, 0.0], [0.0, 0.0, 0.0, 11.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 5.0, 8.0, 14.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 8.0, 0.0, 0.0], [0.0, 2.0, 14.0, 16.0, 12.0, 6.0, 0.0, 0.0, 0.0, 1.0, 10.0, 8.0, 14.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 2.0, 0.0, 0.0, 0.0, 7.0, 10.0, 15.0, 15.0, 2.0, 0.0, 0.0, 3.0, 13.0, 11.0, 7.0, 2.0, 0.0, 0.0], [0.0, 0.0, 0.0, 11.0, 5.0, 3.0, 11.0, 0.0, 0.0, 0.0, 7.0, 14.0, 2.0, 12.0, 9.0, 0.0, 0.0, 2.0, 15.0, 6.0, 3.0, 16.0, 5.0, 0.0, 0.0, 7.0, 16.0, 8.0, 13.0, 16.0, 13.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 7.0, 1.0, 0.0, 0.0, 0.0, 4.0, 10.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.0, 11.0, 16.0, 15.0, 1.0, 0.0, 0.0, 1.0, 16.0, 14.0, 10.0, 16.0, 2.0, 0.0, 0.0, 5.0, 12.0, 0.0, 8.0, 12.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 13.0, 2.0, 0.0, 0.0, 0.0, 11.0, 14.0, 15.0, 12.0, 5.0, 0.0, 0.0, 0.0, 0.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 5.0, 0.0, 0.0, 0.0], [0.0, 0.0, 8.0, 16.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 4.0, 8.0, 13.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 3.0, 12.0, 16.0, 8.0, 2.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 14.0, 8.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 12.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 7.0, 4.0, 2.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 16.0, 7.0, 0.0, 0.0, 1.0, 14.0, 15.0, 4.0, 11.0, 15.0, 0.0, 0.0, 0.0, 5.0, 14.0, 16.0, 12.0, 6.0, 0.0], [0.0, 0.0, 8.0, 12.0, 14.0, 12.0, 3.0, 0.0, 0.0, 0.0, 12.0, 5.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 16.0, 2.0, 4.0, 1.0, 0.0, 0.0, 0.0, 4.0, 16.0, 14.0, 12.0, 15.0, 4.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 11.0, 5.0, 0.0, 0.0, 6.0, 14.0, 1.0, 2.0, 15.0, 1.0, 0.0, 0.0, 0.0, 8.0, 14.0, 16.0, 4.0, 0.0, 0.0], [0.0, 0.0, 8.0, 16.0, 14.0, 15.0, 0.0, 0.0, 0.0, 0.0, 13.0, 11.0, 15.0, 11.0, 0.0, 0.0, 0.0, 4.0, 14.0, 1.0, 13.0, 5.0, 0.0, 0.0, 0.0, 3.0, 7.0, 8.0, 16.0, 4.0, 3.0, 0.0, 0.0, 0.0, 14.0, 16.0, 16.0, 13.0, 7.0, 0.0, 0.0, 0.0, 5.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 6.0, 0.0, 0.0, 0.0, 0.0], [0.0, 5.0, 15.0, 12.0, 12.0, 12.0, 4.0, 0.0, 0.0, 10.0, 14.0, 12.0, 12.0, 9.0, 7.0, 0.0, 0.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 13.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 14.0, 0.0, 0.0, 0.0, 1.0, 12.0, 16.0, 16.0, 11.0, 0.0, 0.0, 0.0, 11.0, 16.0, 12.0, 16.0, 8.0, 0.0, 0.0, 0.0, 6.0, 4.0, 7.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 11.0, 0.0, 0.0], [0.0, 0.0, 2.0, 10.0, 16.0, 12.0, 0.0, 0.0, 0.0, 2.0, 15.0, 14.0, 8.0, 1.0, 0.0, 0.0, 0.0, 2.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 14.0, 13.0, 3.0, 0.0, 0.0, 0.0, 10.0, 16.0, 7.0, 11.0, 12.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 4.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 2.0, 0.0, 0.0, 0.0, 2.0, 14.0, 16.0, 8.0, 0.0, 0.0], [0.0, 0.0, 6.0, 14.0, 10.0, 2.0, 0.0, 0.0, 0.0, 0.0, 15.0, 15.0, 13.0, 15.0, 3.0, 0.0, 0.0, 2.0, 16.0, 10.0, 0.0, 13.0, 9.0, 0.0, 0.0, 1.0, 16.0, 5.0, 0.0, 12.0, 5.0, 0.0, 0.0, 0.0, 16.0, 3.0, 0.0, 13.0, 6.0, 0.0, 0.0, 1.0, 15.0, 5.0, 6.0, 13.0, 1.0, 0.0, 0.0, 0.0, 16.0, 11.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 11.0, 1.0, 0.0, 0.0], [0.0, 0.0, 9.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 14.0, 13.0, 7.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 10.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 9.0, 12.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 0.0, 0.0, 0.0, 3.0, 8.0, 9.0, 15.0, 15.0, 0.0, 0.0, 0.0, 5.0, 12.0, 12.0, 9.0, 1.0, 0.0], [0.0, 0.0, 1.0, 10.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 11.0, 3.0, 0.0, 0.0, 0.0, 0.0, 16.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 13.0, 0.0, 6.0, 10.0, 3.0, 0.0, 0.0, 3.0, 15.0, 13.0, 12.0, 10.0, 12.0, 0.0, 0.0, 0.0, 10.0, 16.0, 4.0, 5.0, 14.0, 0.0, 0.0, 0.0, 0.0, 9.0, 15.0, 14.0, 9.0, 0.0], [0.0, 2.0, 13.0, 13.0, 11.0, 9.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 16.0, 15.0, 10.0, 0.0, 0.0, 11.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 11.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 11.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 15.0, 4.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 9.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 6.0, 12.0, 0.0, 0.0, 0.0, 1.0, 12.0, 8.0, 5.0, 14.0, 0.0, 0.0, 0.0, 6.0, 14.0, 0.0, 12.0, 7.0, 0.0, 0.0, 0.0, 14.0, 6.0, 2.0, 16.0, 9.0, 5.0, 0.0, 0.0, 16.0, 13.0, 13.0, 16.0, 15.0, 4.0, 0.0, 1.0, 15.0, 16.0, 16.0, 12.0, 2.0, 0.0, 0.0, 0.0, 3.0, 3.0, 13.0, 4.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 8.0, 14.0, 15.0, 7.0, 0.0, 0.0, 0.0, 4.0, 16.0, 12.0, 15.0, 14.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 11.0, 12.0, 0.0, 0.0, 0.0, 2.0, 4.0, 6.0, 14.0, 15.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 16.0, 5.0, 0.0, 0.0, 8.0, 12.0, 7.0, 14.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 7.0, 0.0, 0.0, 0.0], [0.0, 2.0, 12.0, 13.0, 16.0, 15.0, 1.0, 0.0, 0.0, 8.0, 16.0, 14.0, 11.0, 7.0, 0.0, 0.0, 0.0, 8.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 6.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 15.0, 0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 16.0, 11.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 13.0, 1.0, 0.0, 0.0], [0.0, 1.0, 13.0, 13.0, 10.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 15.0, 8.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 4.0, 0.0, 0.0, 3.0, 13.0, 16.0, 14.0, 16.0, 1.0, 0.0, 0.0, 2.0, 7.0, 4.0, 8.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 2.0, 0.0], [0.0, 0.0, 4.0, 10.0, 13.0, 6.0, 0.0, 0.0, 0.0, 1.0, 16.0, 14.0, 12.0, 16.0, 3.0, 0.0, 0.0, 4.0, 16.0, 6.0, 3.0, 16.0, 4.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 5.0, 0.0, 0.0, 0.0, 5.0, 7.0, 7.0, 16.0, 4.0, 0.0, 0.0, 0.0, 2.0, 14.0, 15.0, 9.0, 0.0, 0.0], [0.0, 0.0, 5.0, 16.0, 15.0, 4.0, 0.0, 0.0, 0.0, 3.0, 15.0, 13.0, 13.0, 12.0, 0.0, 0.0, 0.0, 7.0, 14.0, 1.0, 0.0, 16.0, 5.0, 0.0, 0.0, 12.0, 9.0, 0.0, 1.0, 11.0, 10.0, 0.0, 0.0, 10.0, 10.0, 0.0, 0.0, 7.0, 13.0, 0.0, 0.0, 6.0, 15.0, 0.0, 0.0, 8.0, 12.0, 0.0, 0.0, 1.0, 14.0, 7.0, 6.0, 15.0, 11.0, 0.0, 0.0, 0.0, 5.0, 15.0, 16.0, 14.0, 3.0, 0.0], [0.0, 0.0, 0.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 12.0, 9.0, 1.0, 0.0, 0.0, 0.0, 15.0, 16.0, 14.0, 13.0, 12.0, 0.0, 0.0, 0.0, 14.0, 15.0, 7.0, 0.0, 15.0, 6.0, 0.0, 0.0, 9.0, 14.0, 4.0, 7.0, 15.0, 8.0, 0.0, 0.0, 1.0, 13.0, 16.0, 16.0, 12.0, 1.0], [0.0, 0.0, 2.0, 16.0, 10.0, 1.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 13.0, 8.0, 9.0, 13.0, 0.0, 0.0, 0.0, 0.0, 12.0, 10.0, 7.0, 16.0, 0.0, 0.0, 0.0, 0.0, 3.0, 13.0, 15.0, 10.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 12.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 11.0, 0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 3.0, 7.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 15.0, 5.0, 0.0], [0.0, 0.0, 4.0, 14.0, 11.0, 3.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 12.0, 14.0, 1.0, 0.0, 0.0, 1.0, 14.0, 12.0, 0.0, 13.0, 3.0, 0.0, 0.0, 5.0, 16.0, 6.0, 0.0, 8.0, 6.0, 0.0, 0.0, 8.0, 16.0, 0.0, 0.0, 9.0, 8.0, 0.0, 0.0, 7.0, 16.0, 3.0, 7.0, 16.0, 5.0, 0.0, 0.0, 3.0, 15.0, 13.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 4.0, 15.0, 12.0, 2.0, 0.0, 0.0], [0.0, 1.0, 9.0, 15.0, 13.0, 4.0, 0.0, 0.0, 0.0, 8.0, 12.0, 4.0, 8.0, 8.0, 0.0, 0.0, 0.0, 9.0, 11.0, 0.0, 5.0, 16.0, 3.0, 0.0, 0.0, 4.0, 13.0, 8.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 4.0, 8.0, 2.0, 11.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 8.0, 0.0, 0.0, 0.0, 7.0, 0.0, 3.0, 14.0, 6.0, 0.0, 0.0, 0.0, 7.0, 16.0, 14.0, 8.0, 0.0, 0.0], [0.0, 2.0, 13.0, 16.0, 16.0, 16.0, 15.0, 2.0, 0.0, 8.0, 16.0, 12.0, 8.0, 4.0, 1.0, 0.0, 0.0, 5.0, 16.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 13.0, 12.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 4.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 13.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 12.0, 6.0, 14.0, 4.0, 0.0, 0.0, 0.0, 4.0, 16.0, 9.0, 15.0, 13.0, 3.0, 0.0, 0.0, 4.0, 12.0, 12.0, 16.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 3.0, 0.0, 0.0], [0.0, 0.0, 0.0, 8.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 14.0, 11.0, 13.0, 2.0, 0.0, 0.0, 0.0, 13.0, 16.0, 14.0, 14.0, 10.0, 0.0, 0.0, 0.0, 10.0, 15.0, 1.0, 5.0, 13.0, 0.0, 0.0, 0.0, 6.0, 16.0, 8.0, 14.0, 12.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 16.0, 4.0, 0.0], [0.0, 0.0, 12.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 6.0, 4.0, 10.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 5.0, 9.0, 16.0, 16.0, 12.0, 0.0, 0.0, 3.0, 16.0, 16.0, 11.0, 3.0, 0.0, 0.0, 0.0, 0.0, 7.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 15.0, 12.0, 2.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 12.0, 12.0, 0.0, 0.0, 0.0], [0.0, 3.0, 15.0, 13.0, 12.0, 8.0, 1.0, 0.0, 0.0, 4.0, 16.0, 14.0, 12.0, 12.0, 2.0, 0.0, 0.0, 0.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 16.0, 2.0, 0.0, 0.0, 0.0], [0.0, 2.0, 16.0, 16.0, 16.0, 16.0, 3.0, 0.0, 0.0, 1.0, 16.0, 13.0, 6.0, 4.0, 0.0, 0.0, 0.0, 9.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 13.0, 7.0, 0.0, 0.0, 0.0, 0.0, 3.0, 11.0, 13.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 1.0, 0.0, 0.0, 2.0, 11.0, 5.0, 12.0, 14.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 16.0, 5.0, 0.0, 0.0], [0.0, 0.0, 0.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 6.0, 11.0, 9.0, 3.0, 0.0, 0.0, 1.0, 15.0, 16.0, 12.0, 8.0, 11.0, 0.0, 0.0, 0.0, 9.0, 13.0, 2.0, 6.0, 16.0, 2.0, 0.0, 0.0, 0.0, 11.0, 16.0, 14.0, 7.0, 0.0], [0.0, 0.0, 0.0, 0.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 7.0, 16.0, 8.0, 16.0, 2.0, 0.0, 0.0, 0.0, 1.0, 5.0, 6.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 0.0, 0.0], [0.0, 0.0, 4.0, 15.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 6.0, 9.0, 12.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 9.0, 0.0, 0.0, 0.0, 0.0, 2.0, 4.0, 15.0, 10.0, 4.0, 0.0, 0.0, 2.0, 15.0, 16.0, 16.0, 15.0, 7.0, 0.0, 0.0, 0.0, 8.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 13.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 5.0, 13.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 12.0, 6.0, 4.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 10.0, 13.0, 2.0, 0.0, 0.0, 2.0, 12.0, 16.0, 16.0, 12.0, 5.0, 0.0, 0.0, 1.0, 6.0, 9.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 11.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 10.0, 2.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 10.0, 1.0, 0.0, 0.0, 4.0, 16.0, 6.0, 2.0, 14.0, 7.0, 0.0, 0.0, 0.0, 11.0, 15.0, 12.0, 15.0, 8.0, 0.0, 0.0, 0.0, 2.0, 14.0, 15.0, 6.0, 0.0, 0.0], [0.0, 1.0, 6.0, 12.0, 15.0, 5.0, 0.0, 0.0, 0.0, 7.0, 14.0, 14.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 11.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 7.0, 0.0, 0.0, 0.0, 2.0, 4.0, 5.0, 14.0, 7.0, 0.0, 0.0, 0.0, 6.0, 16.0, 14.0, 8.0, 0.0, 0.0], [0.0, 2.0, 11.0, 16.0, 13.0, 2.0, 0.0, 0.0, 0.0, 11.0, 15.0, 12.0, 16.0, 7.0, 0.0, 0.0, 0.0, 7.0, 6.0, 0.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 4.0, 3.0, 3.0, 0.0, 0.0, 3.0, 15.0, 16.0, 15.0, 15.0, 11.0, 0.0, 0.0, 2.0, 13.0, 12.0, 9.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 4.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 15.0, 8.0, 0.0, 0.0, 0.0, 1.0, 15.0, 8.0, 14.0, 7.0, 0.0, 0.0, 0.0, 6.0, 16.0, 8.0, 14.0, 14.0, 4.0, 0.0, 0.0, 10.0, 16.0, 16.0, 16.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 13.0, 2.0, 0.0, 0.0], [0.0, 0.0, 7.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 13.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 6.0, 4.0, 8.0, 1.0, 0.0, 0.0, 14.0, 16.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 7.0, 16.0, 15.0, 7.0, 3.0, 0.0], [0.0, 0.0, 1.0, 10.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 7.0, 3.0, 5.0, 0.0, 0.0, 3.0, 16.0, 7.0, 3.0, 16.0, 11.0, 0.0, 0.0, 9.0, 14.0, 1.0, 10.0, 14.0, 2.0, 0.0, 0.0, 11.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 2.0, 4.0, 8.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 9.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 4.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 8.0, 1.0, 0.0, 0.0, 0.0, 0.0, 6.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 12.0, 11.0, 10.0, 0.0, 0.0, 0.0, 4.0, 16.0, 15.0, 8.0, 11.0, 10.0, 0.0, 0.0, 1.0, 7.0, 15.0, 4.0, 3.0, 12.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 16.0, 8.0, 0.0], [0.0, 0.0, 9.0, 13.0, 16.0, 5.0, 0.0, 0.0, 0.0, 3.0, 16.0, 8.0, 4.0, 13.0, 0.0, 0.0, 0.0, 6.0, 10.0, 1.0, 0.0, 9.0, 2.0, 0.0, 0.0, 5.0, 4.0, 0.0, 0.0, 4.0, 8.0, 0.0, 0.0, 8.0, 4.0, 0.0, 0.0, 4.0, 8.0, 0.0, 0.0, 6.0, 6.0, 0.0, 0.0, 4.0, 9.0, 0.0, 0.0, 0.0, 13.0, 2.0, 0.0, 7.0, 8.0, 0.0, 0.0, 0.0, 8.0, 12.0, 13.0, 15.0, 2.0, 0.0], [0.0, 0.0, 0.0, 9.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0, 4.0, 7.0, 7.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 3.0, 0.0, 0.0, 4.0, 9.0, 8.0, 10.0, 13.0, 1.0, 0.0, 0.0, 4.0, 16.0, 15.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 7.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 11.0, 12.0, 1.0, 0.0, 0.0, 0.0, 2.0, 14.0, 9.0, 9.0, 8.0, 0.0, 0.0, 0.0, 10.0, 12.0, 0.0, 13.0, 6.0, 0.0, 0.0, 0.0, 6.0, 5.0, 2.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 10.0, 9.0, 1.0, 0.0, 0.0, 0.0, 6.0, 7.0, 0.0, 12.0, 6.0, 0.0, 0.0, 0.0, 1.0, 12.0, 16.0, 16.0, 5.0, 0.0], [0.0, 0.0, 6.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 15.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 6.0, 9.0, 14.0, 15.0, 13.0, 7.0, 0.0, 0.0, 5.0, 15.0, 16.0, 16.0, 15.0, 3.0], [0.0, 0.0, 9.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 2.0, 16.0, 8.0, 9.0, 16.0, 0.0, 0.0, 0.0, 1.0, 8.0, 0.0, 13.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 14.0, 0.0, 0.0, 0.0, 9.0, 6.0, 0.0, 11.0, 15.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 16.0, 6.0, 0.0], [0.0, 0.0, 7.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 16.0, 11.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 3.0, 9.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 1.0, 2.0, 5.0, 14.0, 8.0, 0.0, 0.0, 5.0, 14.0, 0.0, 0.0, 9.0, 15.0, 0.0, 0.0, 4.0, 16.0, 7.0, 6.0, 13.0, 14.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 4.0, 0.0], [0.0, 0.0, 10.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 7.0, 1.0, 7.0, 0.0, 0.0, 0.0, 0.0, 15.0, 7.0, 12.0, 16.0, 2.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 12.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 11.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 3.0, 16.0, 7.0, 14.0, 16.0, 1.0, 0.0, 0.0, 0.0, 15.0, 14.0, 15.0, 16.0, 6.0, 0.0, 0.0, 0.0, 2.0, 10.0, 9.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 8.0, 0.0, 0.0, 0.0, 6.0, 11.0, 6.0, 15.0, 5.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 12.0, 0.0, 0.0], [0.0, 0.0, 0.0, 8.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 15.0, 16.0, 9.0, 2.0, 0.0, 0.0, 6.0, 15.0, 16.0, 16.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 14.0, 0.0, 0.0, 0.0], [0.0, 0.0, 5.0, 14.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 16.0, 10.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0.0, 14.0, 2.0, 8.0, 16.0, 6.0, 0.0, 0.0, 0.0, 12.0, 7.0, 10.0, 16.0, 8.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 15.0, 8.0, 0.0, 0.0, 1.0, 3.0, 1.0, 2.0, 11.0, 9.0, 0.0, 0.0, 1.0, 15.0, 6.0, 4.0, 12.0, 11.0, 0.0, 0.0, 0.0, 6.0, 16.0, 14.0, 12.0, 3.0, 0.0], [0.0, 0.0, 10.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 5.0, 8.0, 13.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 12.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 13.0, 13.0, 5.0, 1.0, 0.0, 0.0, 0.0, 1.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 13.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 6.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 14.0, 2.0, 0.0, 0.0, 0.0, 1.0, 15.0, 16.0, 11.0, 2.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 4.0, 11.0, 12.0, 8.0, 0.0, 0.0], [0.0, 0.0, 2.0, 11.0, 10.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 13.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0.0, 13.0, 0.0, 0.0, 14.0, 5.0, 0.0, 0.0, 3.0, 9.0, 0.0, 0.0, 9.0, 6.0, 0.0, 0.0, 5.0, 9.0, 0.0, 0.0, 5.0, 8.0, 0.0, 0.0, 6.0, 12.0, 0.0, 0.0, 8.0, 4.0, 0.0, 0.0, 0.0, 14.0, 11.0, 5.0, 14.0, 1.0, 0.0, 0.0, 0.0, 3.0, 13.0, 14.0, 5.0, 0.0, 0.0], [0.0, 0.0, 1.0, 13.0, 12.0, 5.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 4.0, 13.0, 2.0, 0.0, 0.0, 2.0, 16.0, 4.0, 0.0, 8.0, 5.0, 0.0, 0.0, 7.0, 12.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 6.0, 12.0, 0.0, 0.0, 5.0, 8.0, 0.0, 0.0, 3.0, 16.0, 0.0, 0.0, 8.0, 7.0, 0.0, 0.0, 1.0, 15.0, 8.0, 6.0, 15.0, 3.0, 0.0, 0.0, 0.0, 2.0, 13.0, 15.0, 6.0, 0.0, 0.0], [0.0, 0.0, 1.0, 12.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 12.0, 8.0, 14.0, 14.0, 3.0, 0.0, 0.0, 4.0, 16.0, 16.0, 9.0, 12.0, 8.0, 0.0, 0.0, 0.0, 13.0, 8.0, 0.0, 11.0, 8.0, 0.0, 0.0, 0.0, 1.0, 14.0, 16.0, 11.0, 1.0, 0.0], [0.0, 0.0, 7.0, 16.0, 16.0, 15.0, 5.0, 0.0, 0.0, 0.0, 9.0, 12.0, 15.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 15.0, 1.0, 0.0, 0.0, 0.0, 7.0, 12.0, 15.0, 15.0, 8.0, 0.0, 0.0, 1.0, 16.0, 16.0, 16.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 1.0, 7.0, 0.0, 0.0, 0.0, 2.0, 16.0, 4.0, 9.0, 13.0, 0.0, 0.0, 0.0, 8.0, 11.0, 6.0, 16.0, 1.0, 2.0, 0.0, 0.0, 12.0, 10.0, 12.0, 14.0, 12.0, 11.0, 0.0, 0.0, 11.0, 16.0, 16.0, 14.0, 7.0, 1.0, 0.0, 0.0, 1.0, 7.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 9.0, 16.0, 11.0, 1.0, 0.0, 0.0, 0.0, 5.0, 16.0, 10.0, 16.0, 9.0, 0.0, 0.0, 0.0, 6.0, 14.0, 1.0, 9.0, 15.0, 0.0, 0.0, 0.0, 1.0, 15.0, 6.0, 11.0, 16.0, 2.0, 0.0, 0.0, 0.0, 7.0, 16.0, 15.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 3.0, 1.0, 11.0, 9.0, 0.0, 0.0, 3.0, 14.0, 9.0, 9.0, 14.0, 12.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 13.0, 3.0, 0.0], [0.0, 0.0, 0.0, 7.0, 14.0, 16.0, 5.0, 0.0, 0.0, 0.0, 7.0, 16.0, 12.0, 16.0, 8.0, 0.0, 0.0, 0.0, 4.0, 2.0, 1.0, 16.0, 4.0, 0.0, 0.0, 0.0, 3.0, 12.0, 12.0, 16.0, 8.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 15.0, 5.0, 0.0, 0.0, 0.0, 5.0, 5.0, 13.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 8.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 9.0, 13.0, 1.0, 0.0, 0.0, 0.0, 1.0, 12.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 16.0, 15.0, 3.0, 0.0, 0.0, 2.0, 16.0, 11.0, 1.0, 9.0, 11.0, 0.0, 0.0, 0.0, 11.0, 13.0, 6.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 15.0, 2.0, 0.0], [0.0, 0.0, 7.0, 15.0, 16.0, 15.0, 0.0, 0.0, 0.0, 2.0, 15.0, 2.0, 5.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 11.0, 2.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 5.0, 13.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 12.0, 0.0, 0.0, 0.0, 0.0], [0.0, 4.0, 10.0, 15.0, 16.0, 16.0, 14.0, 0.0, 0.0, 11.0, 16.0, 14.0, 8.0, 5.0, 2.0, 0.0, 0.0, 6.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 10.0, 0.0, 0.0, 0.0, 0.0, 5.0, 10.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 5.0, 15.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 12.0, 7.0, 11.0, 6.0, 0.0, 0.0, 0.0, 0.0, 9.0, 12.0, 15.0, 1.0, 0.0, 0.0, 0.0, 1.0, 8.0, 16.0, 4.0, 0.0, 0.0, 0.0, 3.0, 15.0, 8.0, 13.0, 0.0, 0.0, 0.0, 0.0, 7.0, 12.0, 0.0, 10.0, 7.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 10.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 13.0, 10.0, 0.0, 0.0], [0.0, 1.0, 11.0, 10.0, 8.0, 1.0, 1.0, 0.0, 0.0, 3.0, 15.0, 11.0, 3.0, 12.0, 6.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 13.0, 7.0, 1.0, 16.0, 4.0, 0.0, 0.0, 1.0, 15.0, 4.0, 7.0, 14.0, 0.0, 0.0, 0.0, 1.0, 14.0, 14.0, 8.0, 1.0, 0.0, 0.0], [0.0, 0.0, 2.0, 12.0, 12.0, 12.0, 9.0, 2.0, 0.0, 0.0, 9.0, 15.0, 12.0, 13.0, 16.0, 5.0, 0.0, 0.0, 12.0, 8.0, 0.0, 8.0, 10.0, 0.0, 0.0, 1.0, 16.0, 3.0, 3.0, 15.0, 2.0, 0.0, 0.0, 1.0, 3.0, 0.0, 12.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 3.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 13.0, 13.0, 0.0, 0.0, 0.0, 1.0, 16.0, 8.0, 1.0, 16.0, 2.0, 0.0, 0.0, 7.0, 16.0, 6.0, 0.0, 11.0, 8.0, 0.0, 0.0, 7.0, 16.0, 4.0, 0.0, 11.0, 8.0, 0.0, 0.0, 4.0, 15.0, 1.0, 1.0, 15.0, 7.0, 0.0, 0.0, 0.0, 13.0, 12.0, 14.0, 15.0, 1.0, 0.0, 0.0, 0.0, 3.0, 14.0, 16.0, 6.0, 0.0, 0.0], [0.0, 0.0, 2.0, 12.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 7.0, 16.0, 6.0, 4.0, 13.0, 0.0, 0.0, 0.0, 8.0, 16.0, 6.0, 0.0, 13.0, 5.0, 0.0, 0.0, 1.0, 16.0, 5.0, 0.0, 7.0, 9.0, 0.0, 0.0, 0.0, 16.0, 8.0, 0.0, 8.0, 12.0, 0.0, 0.0, 0.0, 13.0, 14.0, 14.0, 16.0, 10.0, 0.0, 0.0, 0.0, 4.0, 14.0, 15.0, 7.0, 0.0, 0.0], [0.0, 0.0, 10.0, 14.0, 8.0, 1.0, 0.0, 0.0, 0.0, 2.0, 16.0, 14.0, 6.0, 1.0, 0.0, 0.0, 0.0, 0.0, 15.0, 15.0, 8.0, 15.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 15.0, 12.0, 0.0, 0.0, 0.0, 4.0, 16.0, 6.0, 4.0, 16.0, 6.0, 0.0, 0.0, 8.0, 16.0, 10.0, 8.0, 16.0, 8.0, 0.0, 0.0, 1.0, 8.0, 12.0, 14.0, 12.0, 1.0, 0.0], [0.0, 0.0, 5.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 9.0, 4.0, 13.0, 0.0, 0.0, 0.0, 0.0, 16.0, 5.0, 11.0, 13.0, 0.0, 0.0, 0.0, 0.0, 15.0, 13.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 10.0, 7.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 12.0, 14.0, 6.0, 0.0, 0.0, 0.0, 5.0, 12.0, 0.0, 2.0, 13.0, 0.0, 0.0, 0.0, 4.0, 12.0, 0.0, 0.0, 4.0, 7.0, 0.0, 0.0, 8.0, 5.0, 0.0, 0.0, 4.0, 8.0, 0.0, 0.0, 5.0, 8.0, 0.0, 0.0, 5.0, 10.0, 0.0, 0.0, 0.0, 14.0, 3.0, 4.0, 14.0, 6.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 10.0, 0.0, 0.0], [0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 10.0, 12.0, 10.0, 16.0, 2.0, 0.0, 0.0, 0.0, 13.0, 6.0, 7.0, 13.0, 0.0, 0.0, 0.0, 0.0, 10.0, 1.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 9.0, 10.0, 16.0, 8.0, 3.0, 0.0, 0.0, 1.0, 12.0, 15.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 1.0, 16.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 9.0, 14.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 6.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 13.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 9.0, 4.0, 1.0, 0.0, 0.0, 3.0, 16.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 5.0, 16.0, 11.0, 9.0, 6.0, 2.0], [0.0, 0.0, 3.0, 12.0, 15.0, 11.0, 2.0, 0.0, 0.0, 0.0, 11.0, 13.0, 7.0, 13.0, 8.0, 0.0, 0.0, 7.0, 15.0, 1.0, 5.0, 15.0, 3.0, 0.0, 0.0, 1.0, 12.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 15.0, 15.0, 2.0, 0.0, 0.0, 0.0, 2.0, 13.0, 0.0, 10.0, 5.0, 0.0, 0.0, 0.0, 4.0, 11.0, 4.0, 11.0, 6.0, 0.0, 0.0, 0.0, 2.0, 13.0, 16.0, 12.0, 0.0, 0.0], [0.0, 6.0, 16.0, 16.0, 15.0, 7.0, 0.0, 0.0, 0.0, 2.0, 11.0, 12.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 9.0, 0.0, 0.0, 0.0, 3.0, 7.0, 12.0, 16.0, 7.0, 0.0, 0.0, 3.0, 16.0, 16.0, 15.0, 11.0, 1.0, 0.0], [0.0, 0.0, 6.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 13.0, 15.0, 9.0, 6.0, 0.0, 0.0, 0.0, 6.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 4.0, 2.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 6.0, 8.0, 9.0, 16.0, 4.0, 0.0, 0.0, 0.0, 1.0, 3.0, 13.0, 15.0, 1.0, 0.0, 0.0, 0.0, 7.0, 16.0, 15.0, 3.0, 0.0, 0.0], [0.0, 0.0, 0.0, 9.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 10.0, 13.0, 4.0, 13.0, 2.0, 0.0, 0.0, 2.0, 14.0, 0.0, 0.0, 10.0, 6.0, 0.0, 0.0, 4.0, 9.0, 0.0, 0.0, 6.0, 8.0, 0.0, 0.0, 5.0, 8.0, 0.0, 0.0, 8.0, 7.0, 0.0, 0.0, 2.0, 11.0, 1.0, 0.0, 9.0, 5.0, 0.0, 0.0, 0.0, 6.0, 11.0, 4.0, 13.0, 3.0, 0.0, 0.0, 0.0, 1.0, 11.0, 16.0, 12.0, 0.0, 0.0], [0.0, 0.0, 0.0, 8.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 8.0, 13.0, 6.0, 1.0, 0.0, 0.0, 0.0, 1.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 12.0, 12.0, 9.0, 2.0, 0.0, 0.0, 1.0, 15.0, 1.0, 0.0, 9.0, 10.0, 0.0, 0.0, 0.0, 10.0, 9.0, 4.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 5.0, 0.0, 0.0], [0.0, 0.0, 8.0, 16.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 14.0, 2.0, 5.0, 9.0, 0.0, 0.0, 0.0, 0.0, 14.0, 1.0, 5.0, 12.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 7.0, 10.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 12.0, 0.0, 0.0, 0.0, 6.0, 1.0, 0.0, 2.0, 14.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 16.0, 12.0, 0.0], [0.0, 0.0, 3.0, 11.0, 13.0, 1.0, 0.0, 0.0, 0.0, 6.0, 16.0, 11.0, 13.0, 6.0, 0.0, 0.0, 1.0, 16.0, 8.0, 0.0, 11.0, 4.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 9.0, 4.0, 5.0, 0.0, 0.0, 0.0, 1.0, 14.0, 13.0, 12.0, 15.0, 5.0], [0.0, 2.0, 11.0, 16.0, 15.0, 6.0, 0.0, 0.0, 0.0, 11.0, 15.0, 9.0, 14.0, 13.0, 0.0, 0.0, 0.0, 7.0, 1.0, 0.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 9.0, 2.0, 4.0, 4.0, 0.0, 0.0, 2.0, 15.0, 16.0, 16.0, 16.0, 16.0, 1.0], [0.0, 0.0, 7.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 12.0, 13.0, 0.0, 0.0, 0.0, 0.0, 12.0, 7.0, 0.0, 16.0, 4.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 12.0, 9.0, 0.0, 0.0, 0.0, 9.0, 12.0, 8.0, 10.0, 14.0, 0.0], [0.0, 0.0, 15.0, 15.0, 10.0, 2.0, 0.0, 0.0, 0.0, 0.0, 16.0, 5.0, 8.0, 11.0, 0.0, 0.0, 0.0, 0.0, 9.0, 12.0, 1.0, 14.0, 4.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 11.0, 0.0, 0.0, 0.0, 0.0, 10.0, 12.0, 13.0, 11.0, 0.0, 0.0], [0.0, 0.0, 9.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 8.0, 16.0, 5.0, 4.0, 0.0, 0.0, 0.0, 1.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 15.0, 8.0, 7.0, 3.0, 0.0, 0.0, 0.0, 3.0, 12.0, 12.0, 14.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 12.0, 0.0, 0.0, 0.0, 1.0, 2.0, 1.0, 11.0, 10.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 16.0, 4.0, 0.0], [0.0, 0.0, 1.0, 8.0, 10.0, 15.0, 11.0, 0.0, 0.0, 2.0, 14.0, 15.0, 13.0, 16.0, 7.0, 0.0, 0.0, 0.0, 6.0, 0.0, 6.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 11.0, 12.0, 2.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 15.0, 8.0, 0.0, 0.0, 0.0, 3.0, 13.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 5.0, 0.0, 0.0, 0.0], [0.0, 3.0, 15.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 3.0, 10.0, 8.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 11.0, 13.0, 16.0, 16.0, 8.0, 0.0, 0.0, 1.0, 15.0, 16.0, 15.0, 7.0, 2.0, 0.0, 0.0, 0.0, 5.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 8.0, 7.0, 2.0, 0.0, 0.0, 0.0, 8.0, 15.0, 12.0, 13.0, 15.0, 2.0, 0.0, 0.0, 2.0, 15.0, 1.0, 0.0, 7.0, 11.0, 0.0, 0.0, 0.0, 13.0, 8.0, 5.0, 13.0, 9.0, 0.0, 0.0, 0.0, 2.0, 13.0, 16.0, 11.0, 1.0, 0.0], [0.0, 0.0, 7.0, 16.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 8.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 11.0, 0.0, 0.0, 0.0, 1.0, 0.0, 5.0, 15.0, 9.0, 0.0, 0.0, 0.0, 6.0, 15.0, 16.0, 16.0, 2.0, 0.0], [0.0, 0.0, 5.0, 14.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 8.0, 15.0, 1.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 2.0, 0.0], [0.0, 0.0, 5.0, 12.0, 13.0, 2.0, 0.0, 0.0, 0.0, 3.0, 16.0, 14.0, 16.0, 13.0, 1.0, 0.0, 0.0, 4.0, 16.0, 9.0, 16.0, 12.0, 1.0, 0.0, 0.0, 1.0, 9.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 3.0, 16.0, 5.0, 12.0, 16.0, 0.0, 0.0, 0.0, 3.0, 15.0, 7.0, 14.0, 12.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 13.0, 3.0, 0.0, 0.0], [0.0, 0.0, 6.0, 12.0, 13.0, 12.0, 0.0, 0.0, 0.0, 0.0, 14.0, 12.0, 7.0, 16.0, 1.0, 0.0, 0.0, 0.0, 6.0, 6.0, 14.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 1.0, 0.0, 0.0, 0.0, 10.0, 8.0, 3.0, 16.0, 1.0, 0.0, 0.0, 0.0, 4.0, 14.0, 16.0, 12.0, 0.0, 0.0], [0.0, 1.0, 9.0, 16.0, 16.0, 15.0, 3.0, 0.0, 0.0, 8.0, 16.0, 12.0, 8.0, 8.0, 3.0, 0.0, 0.0, 6.0, 16.0, 9.0, 3.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 3.0, 6.0, 4.0, 13.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 13.0, 0.0, 0.0, 0.0, 0.0, 5.0, 8.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 11.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 6.0, 13.0, 16.0, 16.0, 9.0, 0.0, 0.0, 6.0, 16.0, 14.0, 11.0, 16.0, 10.0, 0.0, 0.0, 2.0, 3.0, 0.0, 4.0, 15.0, 4.0, 0.0, 0.0, 2.0, 9.0, 12.0, 16.0, 13.0, 0.0, 0.0, 2.0, 15.0, 16.0, 16.0, 16.0, 3.0, 0.0, 0.0, 4.0, 9.0, 3.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 9.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 13.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 13.0, 10.0, 11.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 11.0, 0.0, 0.0, 0.0, 0.0, 1.0, 6.0, 14.0, 16.0, 8.0, 0.0, 0.0, 0.0, 11.0, 16.0, 15.0, 8.0, 5.0, 0.0, 0.0, 0.0, 2.0, 11.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 2.0, 0.0, 0.0, 0.0], [0.0, 0.0, 6.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 13.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 12.0, 10.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 7.0, 0.0, 0.0, 0.0, 0.0, 1.0, 10.0, 16.0, 10.0, 8.0, 3.0, 0.0, 0.0, 4.0, 16.0, 16.0, 15.0, 16.0, 16.0], [0.0, 0.0, 3.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 11.0, 11.0, 9.0, 10.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 3.0, 13.0, 9.0, 0.0, 0.0, 0.0, 5.0, 15.0, 4.0, 13.0, 11.0, 0.0, 0.0, 0.0, 1.0, 15.0, 15.0, 8.0, 2.0, 0.0], [0.0, 0.0, 4.0, 15.0, 13.0, 2.0, 0.0, 0.0, 0.0, 4.0, 16.0, 15.0, 7.0, 11.0, 0.0, 0.0, 0.0, 8.0, 14.0, 14.0, 0.0, 14.0, 2.0, 0.0, 0.0, 7.0, 9.0, 12.0, 4.0, 8.0, 7.0, 0.0, 0.0, 6.0, 11.0, 0.0, 0.0, 7.0, 9.0, 0.0, 0.0, 2.0, 15.0, 1.0, 0.0, 10.0, 8.0, 0.0, 0.0, 0.0, 11.0, 8.0, 4.0, 15.0, 4.0, 0.0, 0.0, 0.0, 2.0, 14.0, 16.0, 10.0, 0.0, 0.0], [0.0, 0.0, 13.0, 16.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 14.0, 13.0, 11.0, 16.0, 2.0, 0.0, 0.0, 0.0, 11.0, 13.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 14.0, 15.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 3.0, 15.0, 7.0, 0.0, 0.0, 0.0, 6.0, 11.0, 0.0, 15.0, 6.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 15.0, 3.0, 0.0, 0.0], [0.0, 0.0, 5.0, 16.0, 12.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 11.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 2.0, 4.0, 14.0, 14.0, 2.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 10.0, 4.0, 0.0, 0.0, 0.0, 3.0, 10.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 11.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 13.0, 14.0, 8.0, 0.0, 0.0, 0.0, 3.0, 14.0, 3.0, 1.0, 16.0, 3.0, 0.0, 0.0, 7.0, 9.0, 0.0, 0.0, 14.0, 6.0, 0.0, 0.0, 8.0, 4.0, 0.0, 0.0, 16.0, 4.0, 0.0, 0.0, 8.0, 6.0, 0.0, 0.0, 16.0, 0.0, 0.0, 0.0, 3.0, 11.0, 0.0, 1.0, 14.0, 0.0, 0.0, 0.0, 0.0, 12.0, 4.0, 6.0, 11.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 14.0, 1.0, 0.0, 0.0], [0.0, 0.0, 3.0, 13.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 11.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 15.0, 1.0, 14.0, 16.0, 1.0, 0.0, 0.0, 0.0, 11.0, 13.0, 12.0, 13.0, 5.0, 0.0, 0.0, 0.0, 3.0, 8.0, 1.0, 8.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 6.0, 0.0, 0.0, 0.0, 9.0, 4.0, 3.0, 16.0, 2.0, 0.0, 0.0, 0.0, 4.0, 14.0, 14.0, 7.0, 0.0, 0.0], [0.0, 0.0, 0.0, 3.0, 9.0, 16.0, 16.0, 2.0, 0.0, 0.0, 4.0, 16.0, 13.0, 11.0, 16.0, 1.0, 0.0, 0.0, 3.0, 5.0, 0.0, 6.0, 13.0, 0.0, 0.0, 0.0, 0.0, 2.0, 7.0, 14.0, 9.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 9.0, 8.0, 11.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 12.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 15.0, 8.0, 4.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 16.0, 15.0, 3.0, 0.0, 0.0, 1.0, 16.0, 13.0, 4.0, 11.0, 11.0, 0.0, 0.0, 0.0, 11.0, 14.0, 9.0, 15.0, 11.0, 0.0, 0.0, 0.0, 1.0, 14.0, 16.0, 15.0, 6.0, 0.0], [0.0, 0.0, 7.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 9.0, 2.0, 0.0, 0.0, 5.0, 14.0, 16.0, 15.0, 11.0, 4.0, 0.0, 0.0, 5.0, 7.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 11.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 6.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 14.0, 0.0, 7.0, 1.0, 0.0, 0.0, 6.0, 16.0, 3.0, 3.0, 16.0, 9.0, 0.0, 0.0, 11.0, 16.0, 8.0, 11.0, 16.0, 6.0, 0.0, 0.0, 3.0, 15.0, 16.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 3.0, 0.0, 0.0], [0.0, 0.0, 10.0, 16.0, 16.0, 10.0, 1.0, 0.0, 0.0, 4.0, 16.0, 11.0, 11.0, 16.0, 3.0, 0.0, 0.0, 1.0, 9.0, 1.0, 10.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 0.0, 0.0, 2.0, 12.0, 7.0, 4.0, 14.0, 15.0, 1.0, 0.0, 0.0, 11.0, 16.0, 16.0, 15.0, 4.0, 0.0], [0.0, 0.0, 3.0, 14.0, 10.0, 1.0, 0.0, 0.0, 0.0, 2.0, 14.0, 12.0, 15.0, 14.0, 1.0, 0.0, 0.0, 6.0, 13.0, 0.0, 3.0, 14.0, 8.0, 0.0, 0.0, 5.0, 12.0, 0.0, 0.0, 11.0, 8.0, 0.0, 0.0, 4.0, 14.0, 0.0, 0.0, 12.0, 7.0, 0.0, 0.0, 1.0, 14.0, 4.0, 3.0, 16.0, 3.0, 0.0, 0.0, 0.0, 8.0, 12.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 10.0, 3.0, 0.0, 0.0], [0.0, 0.0, 15.0, 12.0, 11.0, 6.0, 2.0, 0.0, 0.0, 4.0, 16.0, 15.0, 12.0, 12.0, 10.0, 0.0, 0.0, 7.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 12.0, 3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 14.0, 2.0, 0.0, 0.0, 0.0, 1.0, 8.0, 8.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 6.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 15.0, 16.0, 7.0, 0.0, 0.0, 0.0, 8.0, 16.0, 3.0, 7.0, 12.0, 0.0, 0.0, 0.0, 6.0, 16.0, 3.0, 0.0, 13.0, 3.0, 0.0, 0.0, 8.0, 10.0, 0.0, 0.0, 12.0, 8.0, 0.0, 0.0, 1.0, 15.0, 2.0, 0.0, 9.0, 11.0, 0.0, 0.0, 0.0, 13.0, 14.0, 10.0, 15.0, 12.0, 0.0, 0.0, 0.0, 3.0, 10.0, 16.0, 14.0, 3.0, 0.0], [0.0, 0.0, 9.0, 14.0, 15.0, 6.0, 0.0, 0.0, 0.0, 2.0, 16.0, 12.0, 1.0, 13.0, 0.0, 0.0, 0.0, 3.0, 12.0, 7.0, 0.0, 8.0, 4.0, 0.0, 0.0, 6.0, 11.0, 4.0, 0.0, 7.0, 2.0, 0.0, 0.0, 4.0, 9.0, 0.0, 0.0, 12.0, 1.0, 0.0, 0.0, 3.0, 9.0, 0.0, 4.0, 11.0, 0.0, 0.0, 0.0, 1.0, 12.0, 5.0, 12.0, 3.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 5.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 14.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 8.0, 4.0, 2.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 15.0, 3.0, 0.0, 0.0, 2.0, 16.0, 12.0, 4.0, 6.0, 16.0, 2.0, 0.0, 0.0, 13.0, 8.0, 3.0, 6.0, 16.0, 6.0, 0.0, 0.0, 2.0, 13.0, 14.0, 16.0, 12.0, 1.0], [0.0, 4.0, 15.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 4.0, 16.0, 15.0, 9.0, 7.0, 1.0, 0.0, 0.0, 0.0, 15.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 5.0, 0.0, 0.0, 0.0, 3.0, 10.0, 13.0, 16.0, 4.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0], [0.0, 0.0, 10.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 15.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 9.0, 16.0, 8.0, 2.0, 0.0, 0.0, 0.0, 5.0, 14.0, 16.0, 11.0, 1.0, 0.0, 0.0, 0.0, 3.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 16.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 7.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 13.0, 1.0, 0.0, 0.0], [0.0, 0.0, 6.0, 14.0, 13.0, 1.0, 0.0, 0.0, 0.0, 3.0, 16.0, 10.0, 5.0, 11.0, 0.0, 0.0, 0.0, 5.0, 16.0, 0.0, 0.0, 13.0, 0.0, 0.0, 0.0, 6.0, 12.0, 0.0, 0.0, 12.0, 3.0, 0.0, 0.0, 7.0, 12.0, 0.0, 0.0, 13.0, 3.0, 0.0, 0.0, 3.0, 11.0, 0.0, 5.0, 12.0, 0.0, 0.0, 0.0, 0.0, 13.0, 4.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 6.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 8.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 1.0, 15.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 6.0, 15.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 3.0, 7.0, 6.0, 6.0, 0.0, 0.0, 0.0, 0.0, 1.0, 10.0, 2.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 11.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 12.0, 5.0, 0.0, 0.0], [0.0, 0.0, 4.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 3.0, 0.0, 7.0, 4.0, 0.0, 0.0, 12.0, 16.0, 6.0, 11.0, 16.0, 7.0, 0.0, 0.0, 7.0, 16.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 6.0, 0.0, 0.0, 0.0], [0.0, 0.0, 7.0, 16.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 1.0, 14.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 13.0, 6.0, 0.0, 0.0], [0.0, 5.0, 15.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 6.0, 10.0, 11.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 15.0, 12.0, 11.0, 0.0, 0.0, 6.0, 16.0, 16.0, 16.0, 13.0, 3.0, 0.0], [0.0, 0.0, 0.0, 1.0, 11.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 9.0, 15.0, 15.0, 7.0, 0.0, 0.0, 0.0, 5.0, 16.0, 3.0, 16.0, 4.0, 0.0, 0.0, 0.0, 10.0, 13.0, 9.0, 16.0, 14.0, 8.0, 0.0, 0.0, 3.0, 15.0, 16.0, 16.0, 13.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 2.0, 0.0, 0.0], [0.0, 7.0, 12.0, 14.0, 16.0, 8.0, 0.0, 0.0, 0.0, 8.0, 16.0, 14.0, 15.0, 11.0, 0.0, 0.0, 0.0, 2.0, 11.0, 2.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 8.0, 5.0, 0.0, 0.0, 8.0, 13.0, 16.0, 16.0, 12.0, 5.0, 0.0, 0.0, 7.0, 16.0, 12.0, 3.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 9.0, 12.0, 16.0, 8.0, 0.0, 0.0, 0.0, 15.0, 15.0, 8.0, 8.0, 2.0, 0.0, 0.0, 4.0, 16.0, 11.0, 4.0, 1.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 11.0, 9.0, 8.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 3.0, 13.0, 9.0, 1.0, 0.0, 0.0], [0.0, 0.0, 4.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 15.0, 16.0, 9.0, 0.0, 0.0, 0.0, 8.0, 13.0, 0.0, 3.0, 15.0, 1.0, 0.0, 0.0, 8.0, 12.0, 0.0, 0.0, 8.0, 6.0, 0.0, 0.0, 8.0, 12.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 5.0, 13.0, 1.0, 0.0, 8.0, 8.0, 0.0, 0.0, 2.0, 15.0, 14.0, 12.0, 15.0, 6.0, 0.0, 0.0, 0.0, 5.0, 16.0, 15.0, 8.0, 0.0, 0.0], [0.0, 4.0, 15.0, 16.0, 15.0, 4.0, 0.0, 0.0, 0.0, 11.0, 16.0, 14.0, 15.0, 16.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0, 16.0, 14.0, 2.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 15.0, 16.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 11.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 15.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 16.0, 12.0, 8.0, 15.0, 0.0, 0.0, 0.0, 7.0, 16.0, 4.0, 0.0, 11.0, 5.0, 0.0, 0.0, 10.0, 15.0, 0.0, 0.0, 8.0, 9.0, 0.0, 0.0, 10.0, 14.0, 0.0, 0.0, 8.0, 11.0, 0.0, 0.0, 6.0, 16.0, 4.0, 0.0, 11.0, 9.0, 0.0, 0.0, 1.0, 15.0, 7.0, 8.0, 16.0, 5.0, 0.0, 0.0, 0.0, 3.0, 14.0, 16.0, 10.0, 1.0, 0.0], [0.0, 0.0, 0.0, 4.0, 15.0, 11.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 13.0, 13.0, 11.0, 10.0, 0.0, 0.0, 0.0, 7.0, 14.0, 3.0, 14.0, 12.0, 6.0, 0.0, 0.0, 8.0, 16.0, 16.0, 16.0, 15.0, 8.0, 0.0, 0.0, 1.0, 8.0, 9.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 0.0, 0.0, 0.0], [0.0, 0.0, 5.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 1.0, 15.0, 9.0, 10.0, 12.0, 0.0, 0.0, 0.0, 3.0, 16.0, 1.0, 0.0, 16.0, 4.0, 0.0, 0.0, 6.0, 16.0, 0.0, 0.0, 11.0, 6.0, 0.0, 0.0, 3.0, 16.0, 1.0, 0.0, 11.0, 8.0, 0.0, 0.0, 4.0, 16.0, 4.0, 3.0, 15.0, 4.0, 0.0, 0.0, 1.0, 13.0, 13.0, 13.0, 14.0, 1.0, 0.0, 0.0, 0.0, 4.0, 13.0, 14.0, 1.0, 0.0, 0.0], [0.0, 0.0, 4.0, 12.0, 10.0, 1.0, 0.0, 0.0, 0.0, 3.0, 16.0, 13.0, 15.0, 10.0, 0.0, 0.0, 0.0, 5.0, 16.0, 2.0, 1.0, 14.0, 3.0, 0.0, 0.0, 8.0, 13.0, 0.0, 0.0, 10.0, 8.0, 0.0, 0.0, 8.0, 12.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 8.0, 14.0, 0.0, 0.0, 11.0, 8.0, 0.0, 0.0, 3.0, 16.0, 14.0, 13.0, 16.0, 2.0, 0.0, 0.0, 0.0, 8.0, 16.0, 13.0, 5.0, 0.0, 0.0], [0.0, 0.0, 10.0, 16.0, 9.0, 1.0, 0.0, 0.0, 0.0, 7.0, 16.0, 9.0, 14.0, 11.0, 0.0, 0.0, 0.0, 8.0, 14.0, 1.0, 7.0, 14.0, 2.0, 0.0, 0.0, 2.0, 14.0, 14.0, 14.0, 15.0, 3.0, 0.0, 0.0, 0.0, 2.0, 4.0, 4.0, 16.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 13.0, 9.0, 0.0, 0.0, 2.0, 15.0, 8.0, 8.0, 14.0, 8.0, 0.0, 0.0, 0.0, 8.0, 15.0, 13.0, 10.0, 0.0, 0.0], [0.0, 0.0, 8.0, 15.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 7.0, 10.0, 10.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 12.0, 1.0, 0.0, 0.0, 3.0, 16.0, 16.0, 16.0, 12.0, 4.0, 0.0, 0.0, 1.0, 10.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 7.0, 14.0, 16.0, 6.0, 0.0, 0.0, 0.0, 10.0, 16.0, 12.0, 15.0, 9.0, 0.0, 0.0, 0.0, 8.0, 3.0, 2.0, 16.0, 7.0, 0.0, 0.0, 0.0, 1.0, 8.0, 13.0, 16.0, 14.0, 0.0, 0.0, 2.0, 13.0, 16.0, 16.0, 12.0, 1.0, 0.0, 0.0, 6.0, 12.0, 6.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 6.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 2.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 5.0, 2.0, 0.0, 0.0, 0.0, 0.0, 15.0, 12.0, 1.0, 16.0, 4.0, 0.0, 0.0, 4.0, 16.0, 2.0, 9.0, 16.0, 8.0, 0.0, 0.0, 0.0, 10.0, 14.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 6.0, 0.0, 0.0], [0.0, 0.0, 0.0, 8.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 14.0, 15.0, 6.0, 8.0, 11.0, 0.0, 0.0, 3.0, 12.0, 14.0, 5.0, 10.0, 13.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 13.0, 5.0, 0.0], [0.0, 0.0, 0.0, 11.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 9.0, 4.0, 0.0, 0.0, 0.0, 0.0, 3.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 8.0, 2.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 14.0, 2.0, 0.0, 0.0, 2.0, 16.0, 9.0, 3.0, 13.0, 7.0, 0.0, 0.0, 0.0, 11.0, 14.0, 7.0, 16.0, 9.0, 0.0, 0.0, 0.0, 1.0, 10.0, 14.0, 10.0, 2.0, 0.0], [0.0, 0.0, 13.0, 15.0, 11.0, 12.0, 11.0, 0.0, 0.0, 4.0, 16.0, 15.0, 16.0, 13.0, 9.0, 1.0, 0.0, 3.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 14.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 7.0, 16.0, 2.0, 0.0, 0.0, 0.0, 8.0, 12.0, 11.0, 16.0, 3.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 12.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 11.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 16.0, 10.0, 13.0, 16.0, 7.0, 0.0, 0.0, 0.0, 14.0, 10.0, 0.0, 10.0, 11.0, 0.0, 0.0, 0.0, 4.0, 14.0, 16.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 11.0, 13.0, 12.0, 1.0, 0.0, 0.0, 0.0, 8.0, 10.0, 2.0, 14.0, 3.0, 0.0, 0.0, 0.0, 10.0, 10.0, 3.0, 15.0, 1.0, 0.0, 0.0, 0.0, 2.0, 15.0, 16.0, 6.0, 0.0, 0.0], [0.0, 0.0, 0.0, 10.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 8.0, 8.0, 0.0, 0.0, 0.0, 5.0, 4.0, 10.0, 0.0, 12.0, 0.0, 0.0, 0.0, 7.0, 8.0, 10.0, 0.0, 7.0, 5.0, 0.0, 0.0, 6.0, 10.0, 0.0, 0.0, 2.0, 9.0, 0.0, 0.0, 1.0, 13.0, 0.0, 0.0, 2.0, 11.0, 0.0, 0.0, 0.0, 6.0, 11.0, 4.0, 10.0, 11.0, 0.0, 0.0, 0.0, 0.0, 9.0, 15.0, 14.0, 5.0, 0.0], [0.0, 0.0, 5.0, 12.0, 12.0, 8.0, 1.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 10.0, 12.0, 0.0, 0.0, 0.0], [0.0, 0.0, 12.0, 13.0, 12.0, 12.0, 12.0, 0.0, 0.0, 0.0, 16.0, 13.0, 12.0, 11.0, 11.0, 0.0, 0.0, 0.0, 16.0, 13.0, 11.0, 2.0, 0.0, 0.0, 0.0, 3.0, 16.0, 14.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 11.0, 0.0, 0.0, 0.0, 1.0, 14.0, 11.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 10.0, 14.0, 12.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 7.0, 15.0, 13.0, 1.0, 0.0, 0.0, 0.0, 14.0, 12.0, 9.0, 14.0, 8.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 12.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 6.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 16.0, 5.0, 0.0, 0.0, 2.0, 5.0, 7.0, 13.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 9.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 10.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 16.0, 15.0, 1.0, 0.0, 0.0, 4.0, 16.0, 7.0, 3.0, 16.0, 7.0, 0.0, 0.0, 5.0, 16.0, 10.0, 7.0, 16.0, 4.0, 0.0, 0.0, 0.0, 5.0, 14.0, 14.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 2.0, 0.0, 0.0, 0.0, 4.0, 7.0, 7.0, 16.0, 2.0, 0.0, 0.0, 0.0, 5.0, 12.0, 16.0, 12.0, 0.0, 0.0], [0.0, 0.0, 5.0, 15.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 13.0, 15.0, 9.0, 15.0, 2.0, 0.0, 0.0, 4.0, 16.0, 12.0, 0.0, 10.0, 6.0, 0.0, 0.0, 8.0, 16.0, 9.0, 0.0, 8.0, 10.0, 0.0, 0.0, 7.0, 15.0, 5.0, 0.0, 12.0, 11.0, 0.0, 0.0, 7.0, 13.0, 0.0, 5.0, 16.0, 6.0, 0.0, 0.0, 0.0, 16.0, 12.0, 15.0, 13.0, 1.0, 0.0, 0.0, 0.0, 6.0, 16.0, 12.0, 2.0, 0.0, 0.0], [0.0, 0.0, 12.0, 16.0, 16.0, 16.0, 15.0, 0.0, 0.0, 0.0, 3.0, 4.0, 2.0, 8.0, 15.0, 0.0, 0.0, 0.0, 0.0, 2.0, 4.0, 11.0, 11.0, 0.0, 0.0, 0.0, 6.0, 15.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 5.0, 6.0, 15.0, 7.0, 2.0, 0.0, 0.0, 0.0, 0.0, 10.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.0, 13.0, 16.0, 15.0, 6.0, 0.0, 0.0, 0.0, 13.0, 13.0, 9.0, 16.0, 10.0, 0.0, 0.0, 0.0, 16.0, 9.0, 0.0, 14.0, 6.0, 0.0, 0.0, 0.0, 14.0, 2.0, 6.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 16.0, 15.0, 10.0, 2.0, 0.0, 0.0, 0.0, 0.0, 13.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 7.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 9.0, 8.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 4.0, 16.0, 15.0, 1.0, 15.0, 0.0, 0.0, 0.0, 6.0, 16.0, 12.0, 0.0, 12.0, 1.0, 0.0, 0.0, 5.0, 16.0, 11.0, 0.0, 11.0, 6.0, 0.0, 0.0, 1.0, 15.0, 8.0, 4.0, 15.0, 6.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 9.0, 0.0, 0.0], [0.0, 0.0, 6.0, 15.0, 11.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 13.0, 16.0, 0.0, 0.0, 0.0, 0.0, 2.0, 7.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 11.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 7.0, 0.0, 0.0, 0.0, 16.0, 11.0, 10.0, 16.0, 7.0, 0.0, 0.0, 0.0, 5.0, 16.0, 15.0, 9.0, 0.0, 0.0], [0.0, 0.0, 4.0, 15.0, 13.0, 3.0, 0.0, 0.0, 0.0, 4.0, 13.0, 14.0, 10.0, 13.0, 0.0, 0.0, 0.0, 8.0, 16.0, 8.0, 8.0, 13.0, 0.0, 0.0, 0.0, 0.0, 15.0, 12.0, 15.0, 11.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 12.0, 8.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 16.0, 5.0, 0.0, 0.0], [0.0, 0.0, 9.0, 16.0, 16.0, 16.0, 5.0, 0.0, 0.0, 1.0, 14.0, 10.0, 8.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 3.0, 0.0, 0.0, 3.0, 8.0, 11.0, 15.0, 16.0, 11.0, 0.0, 0.0, 8.0, 16.0, 16.0, 15.0, 11.0, 3.0, 0.0, 0.0, 0.0, 2.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 10.0, 0.0, 0.0, 0.0, 0.0], [0.0, 3.0, 15.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 14.0, 14.0, 14.0, 15.0, 0.0, 0.0, 0.0, 0.0, 8.0, 1.0, 6.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 2.0, 0.0, 1.0, 0.0, 0.0, 4.0, 16.0, 15.0, 8.0, 9.0, 15.0, 0.0, 0.0, 3.0, 16.0, 16.0, 16.0, 15.0, 5.0, 0.0], [0.0, 0.0, 6.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 3.0, 16.0, 5.0, 9.0, 13.0, 0.0, 0.0, 0.0, 5.0, 12.0, 0.0, 0.0, 12.0, 6.0, 0.0, 0.0, 8.0, 14.0, 2.0, 0.0, 7.0, 8.0, 0.0, 0.0, 7.0, 12.0, 2.0, 0.0, 4.0, 8.0, 0.0, 0.0, 4.0, 12.0, 0.0, 0.0, 9.0, 7.0, 0.0, 0.0, 3.0, 16.0, 5.0, 7.0, 14.0, 2.0, 0.0, 0.0, 0.0, 7.0, 16.0, 13.0, 3.0, 0.0, 0.0], [0.0, 0.0, 0.0, 5.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 9.0, 0.0, 0.0, 3.0, 0.0, 0.0, 4.0, 14.0, 1.0, 0.0, 8.0, 15.0, 0.0, 0.0, 10.0, 14.0, 12.0, 13.0, 16.0, 6.0, 0.0, 0.0, 5.0, 12.0, 9.0, 11.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 12.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 7.0, 8.0, 0.0, 0.0, 0.0, 0.0, 13.0, 9.0, 0.0, 16.0, 6.0, 0.0, 0.0, 6.0, 16.0, 10.0, 11.0, 16.0, 0.0, 0.0, 0.0, 0.0, 5.0, 10.0, 13.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 8.0, 0.0, 0.0], [0.0, 0.0, 11.0, 8.0, 12.0, 5.0, 0.0, 0.0, 0.0, 1.0, 15.0, 11.0, 6.0, 14.0, 2.0, 0.0, 0.0, 4.0, 11.0, 0.0, 0.0, 9.0, 4.0, 0.0, 0.0, 4.0, 8.0, 0.0, 0.0, 8.0, 6.0, 0.0, 0.0, 6.0, 7.0, 0.0, 0.0, 11.0, 3.0, 0.0, 0.0, 5.0, 8.0, 0.0, 5.0, 13.0, 0.0, 0.0, 0.0, 3.0, 13.0, 5.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 9.0, 14.0, 4.0, 0.0, 0.0, 0.0], [0.0, 0.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 10.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 14.0, 16.0, 11.0, 6.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 16.0, 16.0, 9.0], [0.0, 0.0, 7.0, 14.0, 16.0, 13.0, 1.0, 0.0, 0.0, 0.0, 15.0, 13.0, 3.0, 13.0, 8.0, 0.0, 0.0, 6.0, 15.0, 2.0, 0.0, 8.0, 8.0, 0.0, 0.0, 6.0, 16.0, 0.0, 0.0, 6.0, 11.0, 0.0, 0.0, 9.0, 13.0, 0.0, 0.0, 13.0, 9.0, 0.0, 0.0, 5.0, 15.0, 0.0, 5.0, 16.0, 5.0, 0.0, 0.0, 4.0, 16.0, 9.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 12.0, 3.0, 0.0, 0.0], [0.0, 0.0, 7.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 15.0, 16.0, 5.0, 0.0, 0.0, 0.0, 12.0, 16.0, 5.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 7.0, 2.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 16.0, 12.0, 15.0, 7.0, 0.0, 0.0, 5.0, 16.0, 14.0, 12.0, 12.0, 11.0], [0.0, 0.0, 9.0, 12.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 12.0, 16.0, 9.0, 1.0, 0.0, 0.0, 1.0, 8.0, 6.0, 2.0, 12.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 2.0, 0.0, 0.0, 0.0, 12.0, 14.0, 9.0, 2.0, 0.0, 0.0], [0.0, 0.0, 12.0, 12.0, 14.0, 15.0, 1.0, 0.0, 0.0, 1.0, 15.0, 11.0, 6.0, 5.0, 0.0, 0.0, 0.0, 6.0, 15.0, 12.0, 4.0, 0.0, 0.0, 0.0, 0.0, 6.0, 11.0, 8.0, 13.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 3.0, 0.0, 0.0, 2.0, 6.0, 1.0, 6.0, 14.0, 3.0, 0.0, 0.0, 1.0, 11.0, 16.0, 13.0, 8.0, 0.0, 0.0], [0.0, 0.0, 1.0, 10.0, 11.0, 2.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 8.0, 15.0, 4.0, 0.0, 0.0, 6.0, 11.0, 0.0, 1.0, 12.0, 7.0, 0.0, 0.0, 4.0, 16.0, 7.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 7.0, 14.0, 0.0, 0.0, 0.0, 0.0, 4.0, 12.0, 7.0, 10.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 3.0, 0.0, 0.0], [0.0, 0.0, 5.0, 12.0, 16.0, 12.0, 4.0, 0.0, 0.0, 1.0, 12.0, 7.0, 5.0, 16.0, 5.0, 0.0, 0.0, 2.0, 9.0, 0.0, 8.0, 9.0, 0.0, 0.0, 0.0, 0.0, 2.0, 3.0, 12.0, 1.0, 0.0, 0.0, 0.0, 4.0, 12.0, 14.0, 15.0, 12.0, 4.0, 0.0, 0.0, 5.0, 4.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 12.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 15.0, 16.0, 16.0, 12.0, 2.0, 0.0, 0.0, 2.0, 16.0, 15.0, 12.0, 12.0, 3.0, 0.0, 0.0, 4.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 12.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 8.0, 0.0, 0.0, 0.0, 2.0, 6.0, 9.0, 16.0, 8.0, 0.0, 0.0, 0.0, 1.0, 15.0, 16.0, 15.0, 3.0, 0.0, 0.0], [0.0, 1.0, 9.0, 15.0, 16.0, 6.0, 0.0, 0.0, 0.0, 13.0, 15.0, 10.0, 16.0, 11.0, 0.0, 0.0, 0.0, 5.0, 3.0, 4.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 11.0, 0.0, 0.0, 0.0, 12.0, 5.0, 3.0, 13.0, 14.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 14.0, 5.0, 0.0], [0.0, 0.0, 0.0, 12.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 15.0, 9.0, 2.0, 0.0, 0.0, 0.0, 1.0, 16.0, 15.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 15.0, 7.0, 1.0, 12.0, 10.0, 0.0, 0.0, 0.0, 10.0, 14.0, 4.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 15.0, 5.0, 0.0], [0.0, 0.0, 4.0, 15.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 6.0, 9.0, 11.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 16.0, 10.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 11.0, 3.0, 0.0, 0.0, 0.0, 8.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 2.0, 0.0, 0.0, 0.0], [0.0, 0.0, 6.0, 14.0, 16.0, 11.0, 0.0, 0.0, 0.0, 6.0, 14.0, 7.0, 4.0, 16.0, 4.0, 0.0, 0.0, 7.0, 7.0, 0.0, 5.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 13.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 8.0, 0.0, 0.0, 0.0, 0.0, 2.0, 4.0, 10.0, 12.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 11.0, 3.0, 0.0], [0.0, 0.0, 0.0, 1.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 12.0, 0.0, 3.0, 9.0, 0.0, 0.0, 0.0, 14.0, 2.0, 0.0, 10.0, 7.0, 0.0, 0.0, 6.0, 13.0, 5.0, 11.0, 14.0, 1.0, 0.0, 0.0, 11.0, 16.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 1.0, 2.0, 1.0, 14.0, 1.0, 0.0, 0.0], [0.0, 0.0, 3.0, 12.0, 11.0, 4.0, 0.0, 0.0, 0.0, 4.0, 15.0, 13.0, 12.0, 16.0, 0.0, 0.0, 0.0, 9.0, 14.0, 0.0, 0.0, 12.0, 2.0, 0.0, 0.0, 0.0, 13.0, 11.0, 7.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 10.0, 14.0, 2.0, 0.0, 0.0, 0.0, 11.0, 13.0, 0.0, 8.0, 8.0, 0.0, 0.0, 0.0, 2.0, 12.0, 16.0, 16.0, 7.0, 0.0], [0.0, 0.0, 2.0, 14.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 15.0, 1.0, 0.0, 0.0, 3.0, 16.0, 10.0, 10.0, 16.0, 4.0, 0.0, 0.0, 5.0, 16.0, 0.0, 0.0, 14.0, 6.0, 0.0, 0.0, 5.0, 16.0, 6.0, 0.0, 12.0, 7.0, 0.0, 0.0, 1.0, 15.0, 13.0, 4.0, 13.0, 6.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 2.0, 11.0, 13.0, 4.0, 0.0, 0.0], [0.0, 0.0, 0.0, 6.0, 12.0, 6.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 11.0, 4.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 6.0, 11.0, 12.0, 5.0, 0.0], [0.0, 0.0, 0.0, 12.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 13.0, 15.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 14.0, 1.0, 0.0, 0.0, 0.0, 7.0, 16.0, 15.0, 10.0, 0.0, 0.0, 0.0, 1.0, 16.0, 8.0, 2.0, 14.0, 5.0, 0.0, 0.0, 0.0, 12.0, 10.0, 4.0, 12.0, 7.0, 0.0, 0.0, 0.0, 2.0, 11.0, 16.0, 13.0, 3.0, 0.0], [0.0, 0.0, 7.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 13.0, 2.0, 2.0, 1.0, 0.0, 0.0, 3.0, 16.0, 9.0, 4.0, 13.0, 4.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 16.0, 9.0, 10.0, 15.0, 0.0, 0.0, 0.0, 1.0, 16.0, 2.0, 5.0, 16.0, 4.0, 0.0, 0.0, 0.0, 7.0, 15.0, 16.0, 16.0, 3.0, 0.0], [0.0, 0.0, 5.0, 10.0, 11.0, 13.0, 12.0, 0.0, 0.0, 2.0, 14.0, 8.0, 8.0, 13.0, 10.0, 0.0, 0.0, 1.0, 6.0, 0.0, 4.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 8.0, 1.0, 0.0, 0.0, 2.0, 15.0, 15.0, 8.0, 7.0, 0.0, 0.0, 0.0, 1.0, 9.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 11.0, 0.0, 0.0, 0.0, 0.0], [0.0, 4.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 11.0, 14.0, 11.0, 11.0, 0.0, 0.0, 0.0, 0.0, 11.0, 7.0, 2.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 8.0, 5.0, 3.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 16.0, 16.0, 0.0, 0.0, 3.0, 16.0, 15.0, 8.0, 7.0, 4.0, 0.0], [0.0, 0.0, 4.0, 14.0, 14.0, 4.0, 0.0, 0.0, 0.0, 0.0, 15.0, 10.0, 10.0, 13.0, 0.0, 0.0, 0.0, 5.0, 15.0, 0.0, 2.0, 15.0, 6.0, 0.0, 0.0, 4.0, 13.0, 0.0, 0.0, 14.0, 8.0, 0.0, 0.0, 6.0, 9.0, 0.0, 0.0, 12.0, 7.0, 0.0, 0.0, 3.0, 14.0, 1.0, 0.0, 12.0, 5.0, 0.0, 0.0, 0.0, 12.0, 9.0, 6.0, 15.0, 2.0, 0.0, 0.0, 0.0, 3.0, 14.0, 14.0, 6.0, 0.0, 0.0], [0.0, 0.0, 8.0, 12.0, 13.0, 1.0, 0.0, 0.0, 0.0, 5.0, 12.0, 2.0, 6.0, 13.0, 0.0, 0.0, 0.0, 11.0, 5.0, 0.0, 6.0, 12.0, 0.0, 0.0, 0.0, 7.0, 10.0, 4.0, 13.0, 15.0, 0.0, 0.0, 0.0, 1.0, 11.0, 12.0, 7.0, 12.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 9.0, 0.0, 0.0, 0.0, 10.0, 5.0, 0.0, 3.0, 13.0, 0.0, 0.0, 0.0, 6.0, 12.0, 16.0, 13.0, 10.0, 0.0], [0.0, 0.0, 9.0, 10.0, 2.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 7.0, 7.0, 4.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 7.0, 0.0, 0.0, 0.0, 11.0, 10.0, 4.0, 11.0, 12.0, 0.0, 0.0, 0.0, 8.0, 14.0, 16.0, 15.0, 6.0, 0.0], [0.0, 0.0, 0.0, 15.0, 16.0, 16.0, 12.0, 4.0, 0.0, 0.0, 4.0, 14.0, 0.0, 10.0, 12.0, 0.0, 0.0, 0.0, 8.0, 7.0, 1.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 12.0, 0.0, 0.0, 0.0, 0.0, 1.0, 8.0, 14.0, 12.0, 3.0, 0.0, 0.0, 0.0, 6.0, 13.0, 16.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 2.0, 0.0, 0.0, 0.0], [0.0, 0.0, 5.0, 12.0, 16.0, 15.0, 0.0, 0.0, 0.0, 3.0, 16.0, 8.0, 9.0, 16.0, 0.0, 0.0, 0.0, 4.0, 16.0, 14.0, 16.0, 7.0, 0.0, 0.0, 0.0, 5.0, 16.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 8.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 10.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 2.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 13.0, 8.0, 5.0, 0.0, 0.0, 0.0, 2.0, 15.0, 15.0, 12.0, 15.0, 5.0, 0.0, 0.0, 7.0, 16.0, 4.0, 0.0, 12.0, 8.0, 0.0, 0.0, 2.0, 15.0, 7.0, 0.0, 12.0, 6.0, 0.0, 0.0, 0.0, 5.0, 15.0, 5.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 9.0, 0.0, 0.0], [0.0, 3.0, 16.0, 16.0, 14.0, 7.0, 1.0, 0.0, 0.0, 1.0, 9.0, 9.0, 15.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 15.0, 0.0, 0.0, 0.0, 1.0, 10.0, 10.0, 16.0, 16.0, 3.0, 0.0, 0.0, 2.0, 13.0, 16.0, 12.0, 5.0, 0.0, 0.0], [0.0, 0.0, 2.0, 10.0, 10.0, 12.0, 15.0, 10.0, 0.0, 0.0, 9.0, 16.0, 12.0, 8.0, 15.0, 6.0, 0.0, 0.0, 13.0, 9.0, 0.0, 4.0, 12.0, 1.0, 0.0, 1.0, 16.0, 3.0, 1.0, 13.0, 2.0, 0.0, 0.0, 0.0, 5.0, 0.0, 9.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 2.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 9.0, 15.0, 12.0, 1.0, 0.0, 0.0, 1.0, 11.0, 12.0, 5.0, 15.0, 4.0, 0.0, 0.0, 6.0, 14.0, 0.0, 0.0, 13.0, 7.0, 0.0, 0.0, 5.0, 16.0, 12.0, 12.0, 16.0, 4.0, 0.0, 0.0, 0.0, 3.0, 8.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 9.0, 0.0, 0.0, 0.0], [0.0, 0.0, 15.0, 13.0, 0.0, 3.0, 3.0, 0.0, 0.0, 0.0, 15.0, 15.0, 8.0, 15.0, 5.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 15.0, 10.0, 0.0, 0.0, 0.0, 3.0, 16.0, 0.0, 10.0, 15.0, 1.0, 0.0, 0.0, 2.0, 16.0, 5.0, 7.0, 15.0, 3.0, 0.0, 0.0, 1.0, 12.0, 16.0, 15.0, 7.0, 0.0, 0.0], [0.0, 0.0, 8.0, 14.0, 11.0, 2.0, 0.0, 0.0, 0.0, 6.0, 16.0, 7.0, 6.0, 13.0, 1.0, 0.0, 0.0, 8.0, 11.0, 0.0, 0.0, 10.0, 4.0, 0.0, 0.0, 7.0, 8.0, 0.0, 0.0, 5.0, 7.0, 0.0, 0.0, 8.0, 4.0, 0.0, 0.0, 7.0, 8.0, 0.0, 0.0, 2.0, 10.0, 0.0, 0.0, 7.0, 10.0, 0.0, 0.0, 0.0, 14.0, 3.0, 4.0, 15.0, 3.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 7.0, 0.0, 0.0], [0.0, 0.0, 8.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 14.0, 12.0, 10.0, 14.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 7.0, 14.0, 0.0, 0.0, 0.0, 2.0, 16.0, 5.0, 10.0, 16.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 7.0, 0.0, 0.0], [0.0, 0.0, 12.0, 16.0, 14.0, 4.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 10.0, 0.0, 0.0, 0.0, 2.0, 15.0, 16.0, 16.0, 13.0, 3.0, 0.0, 0.0, 1.0, 8.0, 12.0, 15.0, 12.0, 4.0, 0.0, 0.0, 0.0, 2.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 12.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 13.0, 1.0, 0.0, 0.0, 0.0, 1.0, 14.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 15.0, 10.0, 7.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 12.0, 16.0, 6.0, 0.0, 0.0, 3.0, 16.0, 13.0, 0.0, 16.0, 12.0, 0.0, 0.0, 0.0, 11.0, 16.0, 13.0, 16.0, 12.0, 0.0, 0.0, 0.0, 1.0, 9.0, 13.0, 12.0, 4.0, 0.0], [0.0, 0.0, 0.0, 9.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 5.0, 12.0, 2.0, 0.0, 0.0, 0.0, 15.0, 15.0, 3.0, 13.0, 3.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 13.0, 0.0, 0.0, 0.0, 4.0, 16.0, 10.0, 15.0, 0.0, 0.0, 0.0, 0.0, 3.0, 12.0, 0.0, 8.0, 7.0, 0.0, 0.0, 0.0, 0.0, 12.0, 8.0, 10.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 13.0, 4.0, 0.0, 0.0], [0.0, 0.0, 5.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 10.0, 14.0, 12.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 8.0, 11.0, 2.0, 0.0, 0.0, 3.0, 16.0, 11.0, 0.0, 9.0, 3.0, 0.0, 0.0, 5.0, 12.0, 2.0, 0.0, 12.0, 4.0, 0.0, 0.0, 1.0, 12.0, 0.0, 0.0, 13.0, 3.0, 0.0, 0.0, 0.0, 13.0, 6.0, 8.0, 13.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 12.0, 3.0, 0.0, 0.0], [0.0, 0.0, 7.0, 8.0, 12.0, 6.0, 0.0, 0.0, 0.0, 1.0, 14.0, 11.0, 12.0, 15.0, 0.0, 0.0, 0.0, 3.0, 15.0, 0.0, 0.0, 10.0, 5.0, 0.0, 0.0, 4.0, 9.0, 0.0, 0.0, 8.0, 4.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 13.0, 0.0, 0.0, 0.0, 7.0, 9.0, 0.0, 9.0, 11.0, 0.0, 0.0, 0.0, 2.0, 14.0, 10.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 9.0, 15.0, 6.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 9.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 5.0, 16.0, 3.0, 0.0, 0.0, 0.0, 13.0, 4.0, 0.0, 15.0, 5.0, 0.0, 0.0, 0.0, 15.0, 11.0, 14.0, 16.0, 2.0, 0.0, 0.0, 8.0, 16.0, 16.0, 13.0, 5.0, 0.0, 0.0, 0.0, 3.0, 15.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 0.0, 0.0, 0.0], [0.0, 0.0, 7.0, 8.0, 6.0, 8.0, 4.0, 0.0, 0.0, 0.0, 12.0, 13.0, 12.0, 12.0, 5.0, 0.0, 0.0, 0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 12.0, 9.0, 2.0, 0.0, 0.0, 0.0, 3.0, 8.0, 7.0, 8.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 8.0, 0.0, 0.0, 0.0, 6.0, 0.0, 1.0, 13.0, 4.0, 0.0, 0.0, 0.0, 7.0, 15.0, 15.0, 5.0, 0.0, 0.0], [0.0, 2.0, 9.0, 11.0, 12.0, 15.0, 6.0, 0.0, 0.0, 6.0, 16.0, 9.0, 8.0, 8.0, 1.0, 0.0, 0.0, 4.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 12.0, 11.0, 5.0, 0.0, 0.0, 0.0, 1.0, 5.0, 4.0, 8.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 1.0, 6.0, 4.0, 5.0, 15.0, 3.0, 0.0, 0.0, 3.0, 16.0, 16.0, 16.0, 8.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 9.0, 16.0, 7.0, 0.0, 0.0, 0.0, 4.0, 13.0, 16.0, 16.0, 2.0, 0.0, 0.0, 8.0, 16.0, 15.0, 13.0, 16.0, 3.0, 0.0, 0.0, 3.0, 7.0, 0.0, 8.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 7.0, 0.0], [0.0, 0.0, 3.0, 15.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 12.0, 13.0, 0.0, 0.0, 0.0, 0.0, 13.0, 13.0, 9.0, 14.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 15.0, 11.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 11.0, 14.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 9.0, 10.0, 14.0, 9.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 15.0, 2.0, 0.0, 0.0], [0.0, 0.0, 7.0, 12.0, 12.0, 2.0, 0.0, 0.0, 0.0, 5.0, 15.0, 6.0, 10.0, 9.0, 0.0, 0.0, 0.0, 11.0, 4.0, 0.0, 11.0, 6.0, 0.0, 0.0, 0.0, 3.0, 0.0, 2.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 7.0, 4.0, 4.0, 2.0, 0.0, 0.0, 0.0, 11.0, 12.0, 13.0, 14.0, 11.0, 0.0], [0.0, 0.0, 0.0, 4.0, 13.0, 16.0, 15.0, 2.0, 0.0, 0.0, 2.0, 15.0, 13.0, 13.0, 16.0, 6.0, 0.0, 0.0, 7.0, 7.0, 0.0, 3.0, 16.0, 4.0, 0.0, 0.0, 0.0, 4.0, 4.0, 8.0, 14.0, 0.0, 0.0, 0.0, 14.0, 16.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 11.0, 9.0, 10.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 10.0, 0.0, 0.0, 0.0], [0.0, 1.0, 12.0, 12.0, 13.0, 8.0, 1.0, 0.0, 0.0, 0.0, 8.0, 9.0, 15.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 7.0, 0.0, 0.0, 0.0, 3.0, 8.0, 15.0, 13.0, 2.0, 0.0, 0.0, 2.0, 14.0, 16.0, 10.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 8.0, 12.0, 9.0, 2.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 11.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 1.0, 15.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 12.0, 6.0, 0.0, 0.0], [0.0, 0.0, 8.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 1.0, 16.0, 15.0, 11.0, 8.0, 0.0, 0.0, 0.0, 1.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 10.0, 6.0, 0.0, 0.0, 0.0, 0.0, 2.0, 12.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 15.0, 0.0, 0.0, 0.0, 0.0, 11.0, 5.0, 13.0, 13.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 6.0, 0.0, 0.0], [0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 2.0, 0.0, 0.0, 1.0, 4.0, 12.0, 16.0, 12.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 3.0, 10.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 12.0, 15.0, 4.0, 0.0], [0.0, 0.0, 0.0, 8.0, 15.0, 9.0, 1.0, 0.0, 0.0, 0.0, 11.0, 14.0, 12.0, 15.0, 8.0, 0.0, 0.0, 0.0, 15.0, 5.0, 6.0, 14.0, 2.0, 0.0, 0.0, 0.0, 14.0, 14.0, 15.0, 1.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 9.0, 13.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 15.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 9.0, 15.0, 2.0, 0.0, 0.0], [0.0, 0.0, 4.0, 16.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 2.0, 12.0, 16.0, 3.0, 0.0, 0.0], [0.0, 0.0, 1.0, 9.0, 14.0, 11.0, 1.0, 0.0, 0.0, 0.0, 10.0, 15.0, 9.0, 13.0, 5.0, 0.0, 0.0, 3.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 7.0, 16.0, 11.0, 10.0, 16.0, 5.0, 0.0, 0.0, 2.0, 16.0, 5.0, 0.0, 12.0, 8.0, 0.0, 0.0, 0.0, 10.0, 15.0, 13.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 9.0, 12.0, 7.0, 0.0, 0.0], [0.0, 0.0, 5.0, 15.0, 16.0, 15.0, 1.0, 0.0, 0.0, 10.0, 16.0, 11.0, 8.0, 16.0, 5.0, 0.0, 0.0, 12.0, 10.0, 1.0, 10.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 10.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 7.0, 16.0, 0.0, 0.0, 0.0, 8.0, 13.0, 5.0, 15.0, 12.0, 0.0, 0.0, 0.0, 5.0, 15.0, 16.0, 14.0, 3.0, 0.0], [0.0, 0.0, 6.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 15.0, 15.0, 3.0, 0.0, 0.0, 0.0, 2.0, 16.0, 6.0, 11.0, 14.0, 0.0, 0.0, 0.0, 0.0, 15.0, 14.0, 15.0, 16.0, 1.0, 0.0, 0.0, 0.0, 3.0, 8.0, 10.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 4.0, 10.0, 16.0, 8.0, 0.0, 0.0, 0.0, 7.0, 12.0, 13.0, 12.0, 1.0, 0.0], [0.0, 0.0, 0.0, 5.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 10.0, 5.0, 0.0, 0.0, 0.0, 0.0, 9.0, 11.0, 10.0, 10.0, 0.0, 0.0, 0.0, 2.0, 15.0, 2.0, 14.0, 6.0, 0.0, 0.0, 0.0, 8.0, 13.0, 5.0, 14.0, 13.0, 4.0, 0.0, 0.0, 11.0, 16.0, 16.0, 16.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 10.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 8.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 10.0, 0.0, 7.0, 7.0, 0.0, 0.0, 5.0, 16.0, 3.0, 1.0, 16.0, 7.0, 0.0, 0.0, 8.0, 16.0, 11.0, 13.0, 16.0, 3.0, 0.0, 0.0, 1.0, 11.0, 15.0, 16.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 10.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 16.0, 8.0, 11.0, 7.0, 0.0, 0.0, 0.0, 10.0, 16.0, 15.0, 16.0, 6.0, 0.0, 0.0, 3.0, 16.0, 4.0, 6.0, 15.0, 0.0, 0.0, 0.0, 3.0, 8.0, 0.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 13.0, 6.0, 0.0, 0.0, 0.0, 3.0, 14.0, 13.0, 9.0, 3.0, 0.0, 0.0, 0.0, 0.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 2.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 13.0, 16.0, 14.0, 0.0, 0.0, 0.0, 2.0, 14.0, 16.0, 12.0, 4.0, 0.0, 0.0, 0.0, 13.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 10.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 5.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 10.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 11.0, 5.0, 10.0, 11.0, 1.0, 0.0, 0.0, 5.0, 16.0, 13.0, 6.0, 10.0, 8.0, 0.0, 0.0, 0.0, 10.0, 9.0, 0.0, 7.0, 11.0, 0.0, 0.0, 0.0, 1.0, 12.0, 16.0, 14.0, 2.0, 0.0], [0.0, 0.0, 6.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 8.0, 16.0, 13.0, 10.0, 16.0, 0.0, 0.0, 0.0, 6.0, 9.0, 0.0, 6.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 8.0, 3.0, 5.0, 0.0, 0.0, 0.0, 8.0, 16.0, 11.0, 16.0, 9.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 7.0, 0.0, 0.0], [0.0, 1.0, 12.0, 16.0, 16.0, 16.0, 12.0, 0.0, 0.0, 9.0, 16.0, 13.0, 6.0, 8.0, 5.0, 0.0, 0.0, 8.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 2.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 13.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 15.0, 11.0, 2.0, 14.0, 2.0, 0.0, 0.0, 3.0, 14.0, 1.0, 0.0, 12.0, 4.0, 0.0, 0.0, 5.0, 12.0, 0.0, 0.0, 9.0, 5.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 12.0, 2.0, 0.0, 0.0, 4.0, 9.0, 0.0, 2.0, 13.0, 2.0, 0.0, 0.0, 0.0, 13.0, 2.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 7.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 10.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 9.0, 16.0, 13.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 12.0, 12.0, 5.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 10.0, 4.0, 0.0, 0.0, 0.0, 3.0, 11.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 14.0, 12.0, 12.0, 13.0, 3.0, 0.0, 0.0, 0.0, 16.0, 8.0, 8.0, 6.0, 1.0, 0.0, 0.0, 0.0, 14.0, 7.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 15.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 13.0, 3.0, 6.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 13.0, 0.0, 0.0, 0.0, 0.0, 5.0, 4.0, 8.0, 12.0, 1.0, 0.0, 0.0, 1.0, 15.0, 15.0, 11.0, 3.0, 0.0, 0.0], [0.0, 0.0, 0.0, 7.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 5.0, 4.0, 1.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 12.0, 3.0, 0.0, 0.0, 1.0, 15.0, 4.0, 1.0, 8.0, 12.0, 0.0, 0.0, 0.0, 8.0, 14.0, 5.0, 5.0, 15.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 11.0, 0.0], [0.0, 0.0, 3.0, 12.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 12.0, 12.0, 4.0, 0.0, 0.0, 0.0, 2.0, 14.0, 0.0, 1.0, 13.0, 0.0, 0.0, 0.0, 1.0, 12.0, 0.0, 0.0, 7.0, 5.0, 0.0, 0.0, 2.0, 13.0, 0.0, 0.0, 2.0, 10.0, 0.0, 0.0, 0.0, 15.0, 3.0, 0.0, 3.0, 14.0, 0.0, 0.0, 0.0, 7.0, 12.0, 8.0, 11.0, 12.0, 0.0, 0.0, 0.0, 2.0, 11.0, 16.0, 11.0, 2.0, 0.0], [0.0, 0.0, 3.0, 10.0, 14.0, 3.0, 0.0, 0.0, 0.0, 4.0, 16.0, 13.0, 15.0, 11.0, 0.0, 0.0, 0.0, 8.0, 13.0, 1.0, 13.0, 16.0, 2.0, 0.0, 0.0, 6.0, 16.0, 14.0, 14.0, 14.0, 6.0, 0.0, 0.0, 0.0, 5.0, 7.0, 1.0, 11.0, 8.0, 0.0, 0.0, 1.0, 8.0, 1.0, 0.0, 8.0, 8.0, 0.0, 0.0, 2.0, 16.0, 11.0, 8.0, 14.0, 7.0, 0.0, 0.0, 0.0, 5.0, 12.0, 14.0, 9.0, 1.0, 0.0], [0.0, 2.0, 13.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 8.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 16.0, 9.0, 2.0, 0.0, 0.0, 1.0, 16.0, 14.0, 13.0, 16.0, 9.0, 0.0], [0.0, 0.0, 5.0, 12.0, 16.0, 10.0, 0.0, 0.0, 0.0, 8.0, 15.0, 5.0, 12.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 11.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 11.0, 3.0, 0.0], [0.0, 0.0, 5.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 13.0, 11.0, 0.0, 0.0, 0.0, 0.0, 13.0, 8.0, 6.0, 16.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 7.0, 8.0, 14.0, 7.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 8.0, 12.0, 0.0, 0.0, 1.0, 15.0, 11.0, 8.0, 13.0, 11.0, 0.0, 0.0, 0.0, 5.0, 11.0, 12.0, 14.0, 3.0, 0.0], [0.0, 0.0, 9.0, 13.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 6.0, 4.0, 9.0, 14.0, 1.0, 0.0, 0.0, 0.0, 1.0, 5.0, 0.0, 11.0, 4.0, 0.0, 0.0, 0.0, 13.0, 14.0, 0.0, 7.0, 5.0, 0.0, 0.0, 3.0, 14.0, 1.0, 0.0, 10.0, 4.0, 0.0, 0.0, 3.0, 14.0, 0.0, 2.0, 15.0, 1.0, 0.0, 0.0, 2.0, 13.0, 8.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 10.0, 1.0, 0.0, 0.0], [0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 16.0, 11.0, 0.0, 0.0, 6.0, 9.0, 5.0, 5.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 11.0, 1.0, 0.0, 0.0, 0.0, 3.0, 6.0, 16.0, 3.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 4.0, 8.0, 16.0, 4.0, 1.0, 0.0, 0.0, 0.0, 1.0, 13.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 2.0, 0.0, 0.0, 0.0], [0.0, 0.0, 5.0, 15.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 12.0, 14.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 14.0, 14.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 9.0, 14.0, 15.0, 3.0, 0.0, 0.0, 0.0, 1.0, 15.0, 5.0, 8.0, 12.0, 1.0, 0.0, 0.0, 0.0, 16.0, 4.0, 4.0, 16.0, 4.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 12.0, 2.0, 0.0], [0.0, 0.0, 1.0, 11.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 16.0, 9.0, 0.0, 0.0], [0.0, 0.0, 1.0, 13.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 11.0, 14.0, 8.0, 15.0, 0.0, 0.0, 0.0, 2.0, 16.0, 3.0, 0.0, 13.0, 2.0, 0.0, 0.0, 5.0, 15.0, 0.0, 0.0, 10.0, 5.0, 0.0, 0.0, 3.0, 10.0, 0.0, 0.0, 10.0, 5.0, 0.0, 0.0, 3.0, 13.0, 0.0, 1.0, 15.0, 3.0, 0.0, 0.0, 0.0, 12.0, 10.0, 11.0, 11.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 11.0, 4.0, 0.0, 0.0], [0.0, 0.0, 4.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 4.0, 12.0, 14.0, 16.0, 13.0, 13.0, 0.0, 0.0, 2.0, 13.0, 16.0, 16.0, 15.0, 8.0], [0.0, 0.0, 0.0, 12.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 3.0, 9.0, 3.0, 0.0, 0.0, 0.0, 14.0, 7.0, 6.0, 16.0, 2.0, 0.0, 0.0, 3.0, 15.0, 2.0, 10.0, 10.0, 0.0, 0.0, 0.0, 10.0, 9.0, 1.0, 16.0, 12.0, 10.0, 0.0, 0.0, 14.0, 11.0, 14.0, 16.0, 11.0, 1.0, 0.0, 0.0, 9.0, 16.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 8.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 7.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 15.0, 2.0, 0.0], [0.0, 1.0, 13.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 9.0, 15.0, 6.0, 13.0, 8.0, 0.0, 0.0, 0.0, 5.0, 10.0, 0.0, 12.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 13.0, 8.0, 8.0, 5.0, 0.0, 0.0, 1.0, 10.0, 14.0, 16.0, 16.0, 16.0, 0.0], [0.0, 0.0, 0.0, 7.0, 12.0, 13.0, 1.0, 0.0, 0.0, 0.0, 8.0, 11.0, 1.0, 10.0, 8.0, 0.0, 0.0, 0.0, 12.0, 2.0, 1.0, 11.0, 7.0, 0.0, 0.0, 0.0, 10.0, 10.0, 14.0, 8.0, 0.0, 0.0, 0.0, 1.0, 7.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 7.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 7.0, 13.0, 5.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 14.0, 2.0, 0.0], [0.0, 0.0, 3.0, 13.0, 11.0, 7.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 16.0, 2.0, 0.0, 0.0, 4.0, 16.0, 9.0, 1.0, 14.0, 2.0, 0.0, 0.0, 4.0, 16.0, 0.0, 0.0, 16.0, 2.0, 0.0, 0.0, 0.0, 16.0, 1.0, 0.0, 12.0, 8.0, 0.0, 0.0, 0.0, 15.0, 9.0, 0.0, 13.0, 6.0, 0.0, 0.0, 0.0, 9.0, 14.0, 9.0, 14.0, 1.0, 0.0, 0.0, 0.0, 2.0, 12.0, 13.0, 4.0, 0.0, 0.0], [0.0, 0.0, 2.0, 13.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 6.0, 14.0, 5.0, 2.0, 0.0, 0.0, 0.0, 2.0, 14.0, 12.0, 14.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 11.0, 13.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 13.0, 8.0, 10.0, 4.0, 0.0, 0.0, 0.0, 0.0, 2.0, 11.0, 16.0, 7.0, 0.0, 0.0], [0.0, 0.0, 3.0, 10.0, 16.0, 4.0, 0.0, 0.0, 0.0, 1.0, 15.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 6.0, 10.0, 2.0, 3.0, 14.0, 1.0, 0.0, 0.0, 8.0, 6.0, 0.0, 0.0, 10.0, 4.0, 0.0, 0.0, 4.0, 8.0, 0.0, 0.0, 5.0, 8.0, 0.0, 0.0, 0.0, 15.0, 0.0, 0.0, 9.0, 8.0, 0.0, 0.0, 0.0, 12.0, 14.0, 10.0, 16.0, 3.0, 0.0, 0.0, 0.0, 4.0, 14.0, 13.0, 5.0, 0.0, 0.0], [0.0, 0.0, 1.0, 9.0, 15.0, 11.0, 3.0, 0.0, 0.0, 0.0, 12.0, 9.0, 1.0, 11.0, 6.0, 0.0, 0.0, 0.0, 13.0, 7.0, 6.0, 16.0, 8.0, 0.0, 0.0, 0.0, 4.0, 10.0, 12.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 6.0, 0.0, 0.0, 8.0, 7.0, 0.0, 0.0, 15.0, 5.0, 0.0, 0.0, 1.0, 12.0, 10.0, 4.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 8.0, 0.0, 0.0], [0.0, 0.0, 2.0, 14.0, 9.0, 1.0, 0.0, 0.0, 0.0, 1.0, 12.0, 12.0, 11.0, 8.0, 0.0, 0.0, 0.0, 4.0, 14.0, 1.0, 0.0, 13.0, 3.0, 0.0, 0.0, 8.0, 13.0, 0.0, 0.0, 10.0, 6.0, 0.0, 0.0, 5.0, 16.0, 1.0, 0.0, 8.0, 9.0, 0.0, 0.0, 0.0, 16.0, 0.0, 0.0, 11.0, 9.0, 0.0, 0.0, 0.0, 13.0, 11.0, 10.0, 15.0, 4.0, 0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 5.0, 0.0, 0.0], [0.0, 0.0, 0.0, 13.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 9.0, 0.0, 0.0, 0.0], [0.0, 7.0, 16.0, 16.0, 16.0, 11.0, 2.0, 0.0, 0.0, 5.0, 16.0, 12.0, 8.0, 6.0, 1.0, 0.0, 0.0, 9.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 10.0, 0.0, 0.0, 0.0, 1.0, 7.0, 13.0, 16.0, 3.0, 0.0, 0.0, 0.0, 4.0, 15.0, 16.0, 6.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 10.0, 16.0, 4.0, 0.0, 0.0, 1.0, 10.0, 16.0, 16.0, 15.0, 4.0, 0.0, 0.0, 0.0, 16.0, 16.0, 10.0, 1.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 5.0, 11.0, 5.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 9.0, 0.0, 0.0, 0.0, 0.0, 3.0, 10.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 15.0, 2.0, 0.0, 0.0], [0.0, 1.0, 7.0, 13.0, 16.0, 11.0, 0.0, 0.0, 0.0, 11.0, 16.0, 13.0, 15.0, 16.0, 0.0, 0.0, 0.0, 3.0, 8.0, 2.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 11.0, 0.0, 0.0, 0.0, 3.0, 8.0, 14.0, 16.0, 8.0, 0.0, 0.0, 0.0, 7.0, 16.0, 12.0, 7.0, 0.0, 0.0], [0.0, 0.0, 0.0, 9.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 8.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 12.0, 14.0, 5.0, 0.0, 0.0, 0.0, 12.0, 10.0, 0.0, 4.0, 13.0, 0.0, 0.0, 0.0, 9.0, 11.0, 0.0, 6.0, 16.0, 1.0, 0.0, 0.0, 0.0, 8.0, 14.0, 15.0, 8.0, 0.0], [0.0, 0.0, 11.0, 11.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 6.0, 12.0, 8.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 6.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 6.0, 3.0, 5.0, 0.0, 0.0, 0.0, 13.0, 16.0, 13.0, 15.0, 9.0, 0.0, 0.0, 1.0, 12.0, 12.0, 12.0, 12.0, 1.0, 0.0], [0.0, 0.0, 4.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 0.0, 1.0, 1.0, 0.0, 0.0, 3.0, 16.0, 8.0, 5.0, 16.0, 6.0, 0.0, 0.0, 9.0, 16.0, 6.0, 14.0, 16.0, 2.0, 0.0, 0.0, 11.0, 16.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 3.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 15.0, 7.0, 0.0, 0.0, 0.0, 3.0, 16.0, 9.0, 6.0, 15.0, 6.0, 0.0, 0.0, 8.0, 14.0, 0.0, 0.0, 4.0, 8.0, 0.0, 0.0, 8.0, 12.0, 0.0, 0.0, 4.0, 8.0, 0.0, 0.0, 4.0, 12.0, 0.0, 0.0, 11.0, 6.0, 0.0, 0.0, 0.0, 14.0, 10.0, 12.0, 14.0, 1.0, 0.0, 0.0, 0.0, 7.0, 15.0, 11.0, 2.0, 0.0, 0.0], [0.0, 0.0, 5.0, 16.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 14.0, 3.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 16.0, 3.0, 0.0], [0.0, 0.0, 4.0, 12.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 14.0, 15.0, 7.0, 0.0, 0.0, 0.0, 2.0, 14.0, 1.0, 2.0, 16.0, 0.0, 0.0, 0.0, 4.0, 8.0, 0.0, 0.0, 10.0, 4.0, 0.0, 0.0, 7.0, 8.0, 0.0, 0.0, 6.0, 8.0, 0.0, 0.0, 4.0, 11.0, 0.0, 0.0, 5.0, 8.0, 0.0, 0.0, 0.0, 14.0, 11.0, 3.0, 13.0, 5.0, 0.0, 0.0, 0.0, 2.0, 11.0, 16.0, 11.0, 0.0, 0.0], [0.0, 0.0, 2.0, 12.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 12.0, 5.0, 0.0, 0.0, 0.0, 0.0, 15.0, 5.0, 0.0, 14.0, 0.0, 0.0, 0.0, 2.0, 15.0, 1.0, 0.0, 9.0, 7.0, 0.0, 0.0, 4.0, 10.0, 0.0, 0.0, 7.0, 8.0, 0.0, 0.0, 0.0, 12.0, 0.0, 0.0, 8.0, 10.0, 0.0, 0.0, 2.0, 15.0, 5.0, 10.0, 16.0, 1.0, 0.0, 0.0, 0.0, 5.0, 14.0, 12.0, 4.0, 0.0, 0.0], [0.0, 1.0, 11.0, 14.0, 9.0, 1.0, 0.0, 0.0, 0.0, 3.0, 16.0, 8.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 4.0, 0.0, 0.0, 1.0, 3.0, 1.0, 8.0, 16.0, 4.0, 0.0, 0.0, 3.0, 10.0, 16.0, 16.0, 8.0, 0.0, 0.0], [0.0, 0.0, 4.0, 12.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 11.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 8.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 14.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 16.0, 11.0, 0.0, 0.0, 0.0, 5.0, 16.0, 14.0, 8.0, 0.0, 0.0], [0.0, 1.0, 13.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 8.0, 15.0, 5.0, 0.0, 0.0, 0.0, 11.0, 9.0, 0.0, 12.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 10.0, 1.0, 0.0, 1.0, 16.0, 14.0, 10.0, 8.0, 11.0, 1.0], [0.0, 0.0, 3.0, 14.0, 16.0, 14.0, 0.0, 0.0, 0.0, 3.0, 12.0, 16.0, 8.0, 1.0, 0.0, 0.0, 0.0, 15.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 11.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 9.0, 0.0, 0.0, 0.0], [0.0, 0.0, 6.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 14.0, 16.0, 0.0, 0.0, 0.0, 0.0, 5.0, 10.0, 11.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 7.0, 0.0, 0.0, 4.0, 6.0, 0.0, 3.0, 16.0, 8.0, 0.0, 0.0, 5.0, 15.0, 9.0, 16.0, 13.0, 1.0, 0.0, 0.0, 0.0, 9.0, 15.0, 8.0, 0.0, 0.0, 0.0], [0.0, 0.0, 7.0, 15.0, 15.0, 6.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 11.0, 15.0, 0.0, 0.0, 0.0, 7.0, 16.0, 10.0, 10.0, 16.0, 1.0, 0.0, 0.0, 3.0, 11.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 1.0, 15.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 5.0, 14.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 3.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 15.0, 2.0, 0.0, 0.0], [0.0, 0.0, 2.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 10.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 15.0, 9.0, 16.0, 5.0, 0.0, 0.0, 3.0, 15.0, 16.0, 15.0, 7.0, 1.0], [0.0, 0.0, 8.0, 14.0, 16.0, 13.0, 1.0, 0.0, 0.0, 10.0, 9.0, 4.0, 6.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 10.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 4.0, 0.0, 0.0, 0.0, 14.0, 16.0, 13.0, 9.0, 0.0, 0.0], [0.0, 0.0, 8.0, 14.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 11.0, 6.0, 7.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 12.0, 0.0, 0.0, 0.0, 1.0, 11.0, 16.0, 16.0, 16.0, 12.0, 0.0, 0.0, 2.0, 12.0, 13.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 7.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 14.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 4.0, 16.0, 4.0, 5.0, 16.0, 6.0, 0.0, 0.0, 8.0, 14.0, 0.0, 1.0, 15.0, 5.0, 0.0, 0.0, 6.0, 16.0, 0.0, 0.0, 13.0, 4.0, 0.0, 0.0, 4.0, 15.0, 1.0, 7.0, 16.0, 1.0, 0.0, 0.0, 2.0, 15.0, 14.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 4.0, 13.0, 9.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 3.0, 12.0, 12.0, 2.0, 0.0, 0.0, 0.0, 7.0, 15.0, 16.0, 16.0, 0.0, 0.0, 0.0, 4.0, 15.0, 9.0, 14.0, 16.0, 3.0, 0.0, 0.0, 2.0, 0.0, 0.0, 14.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 13.0, 2.0, 0.0], [0.0, 0.0, 6.0, 13.0, 10.0, 3.0, 0.0, 0.0, 0.0, 5.0, 15.0, 11.0, 16.0, 11.0, 0.0, 0.0, 0.0, 2.0, 6.0, 0.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 8.0, 4.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 13.0, 12.0, 4.0, 0.0], [0.0, 5.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 9.0, 8.0, 8.0, 3.0, 0.0, 0.0, 8.0, 16.0, 16.0, 16.0, 16.0, 4.0, 0.0], [0.0, 0.0, 7.0, 13.0, 4.0, 1.0, 0.0, 0.0, 0.0, 1.0, 15.0, 13.0, 15.0, 11.0, 0.0, 0.0, 0.0, 7.0, 16.0, 1.0, 13.0, 16.0, 4.0, 0.0, 0.0, 3.0, 16.0, 12.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 4.0, 11.0, 5.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 5.0, 0.0, 0.0, 0.0, 12.0, 6.0, 9.0, 14.0, 1.0, 0.0, 0.0, 0.0, 6.0, 13.0, 16.0, 5.0, 0.0, 0.0], [0.0, 0.0, 8.0, 14.0, 14.0, 4.0, 0.0, 0.0, 0.0, 5.0, 12.0, 4.0, 7.0, 12.0, 0.0, 0.0, 0.0, 4.0, 2.0, 3.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 9.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 8.0, 0.0, 0.0, 0.0, 3.0, 4.0, 4.0, 13.0, 7.0, 0.0, 0.0, 0.0, 11.0, 16.0, 15.0, 5.0, 0.0, 0.0], [0.0, 1.0, 8.0, 15.0, 16.0, 16.0, 9.0, 0.0, 0.0, 8.0, 16.0, 12.0, 8.0, 8.0, 5.0, 0.0, 0.0, 8.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 8.0, 13.0, 8.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 5.0, 0.0, 0.0, 0.0], [0.0, 0.0, 6.0, 15.0, 16.0, 10.0, 0.0, 0.0, 0.0, 3.0, 16.0, 11.0, 15.0, 10.0, 0.0, 0.0, 0.0, 4.0, 10.0, 10.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 9.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 6.0, 0.0, 0.0, 0.0, 3.0, 9.0, 12.0, 16.0, 5.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 8.0, 0.0, 0.0], [0.0, 1.0, 9.0, 16.0, 15.0, 10.0, 0.0, 0.0, 0.0, 6.0, 16.0, 8.0, 7.0, 16.0, 3.0, 0.0, 0.0, 0.0, 11.0, 14.0, 16.0, 11.0, 1.0, 0.0, 0.0, 1.0, 13.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 10.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 7.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 8.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 6.0, 15.0, 16.0, 16.0, 6.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 16.0, 2.0, 0.0, 0.0, 4.0, 8.0, 8.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 0.0, 0.0], [0.0, 0.0, 15.0, 14.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 12.0, 10.0, 7.0, 6.0, 4.0, 0.0, 0.0, 1.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 12.0, 10.0, 4.0, 0.0, 0.0, 0.0, 4.0, 11.0, 8.0, 11.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 7.0, 0.0, 0.0, 4.0, 16.0, 10.0, 11.0, 14.0, 1.0, 0.0, 0.0, 1.0, 10.0, 16.0, 15.0, 4.0, 0.0, 0.0], [0.0, 0.0, 0.0, 6.0, 14.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 13.0, 2.0, 12.0, 0.0, 0.0, 0.0, 0.0, 8.0, 6.0, 0.0, 12.0, 0.0, 0.0, 0.0, 0.0, 2.0, 12.0, 6.0, 14.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 13.0, 11.0, 6.0, 11.0, 0.0, 0.0, 0.0, 0.0, 9.0, 11.0, 2.0, 7.0, 8.0, 0.0, 0.0, 0.0, 0.0, 5.0, 10.0, 15.0, 13.0, 0.0], [0.0, 0.0, 11.0, 15.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 13.0, 2.0, 0.0, 0.0, 0.0, 14.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 13.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 11.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 14.0, 8.0, 1.0, 0.0, 0.0, 0.0, 5.0, 16.0, 10.0, 10.0, 14.0, 1.0, 0.0, 0.0, 2.0, 15.0, 3.0, 0.0, 12.0, 7.0, 0.0, 0.0, 0.0, 10.0, 13.0, 1.0, 10.0, 11.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 15.0, 5.0, 0.0], [0.0, 0.0, 0.0, 7.0, 12.0, 16.0, 16.0, 15.0, 0.0, 0.0, 9.0, 16.0, 10.0, 4.0, 16.0, 10.0, 0.0, 0.0, 14.0, 16.0, 13.0, 14.0, 12.0, 1.0, 0.0, 0.0, 4.0, 16.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 6.0, 0.0, 0.0, 0.0], [0.0, 0.0, 10.0, 16.0, 15.0, 7.0, 0.0, 0.0, 0.0, 6.0, 16.0, 6.0, 10.0, 16.0, 2.0, 0.0, 0.0, 8.0, 6.0, 5.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 16.0, 1.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 8.0, 1.0, 0.0], [0.0, 0.0, 0.0, 11.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 13.0, 12.0, 1.0, 0.0, 0.0, 2.0, 15.0, 3.0, 0.0, 8.0, 7.0, 0.0, 0.0, 0.0, 8.0, 8.0, 0.0, 10.0, 7.0, 0.0, 0.0, 0.0, 1.0, 11.0, 12.0, 15.0, 4.0, 0.0], [0.0, 0.0, 3.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 2.0, 5.0, 2.0, 0.0, 0.0, 0.0, 4.0, 16.0, 2.0, 12.0, 15.0, 2.0, 0.0, 0.0, 1.0, 14.0, 13.0, 2.0, 13.0, 11.0, 0.0, 0.0, 0.0, 3.0, 11.0, 16.0, 13.0, 4.0, 0.0], [0.0, 0.0, 0.0, 12.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 6.0, 11.0, 4.0, 15.0, 0.0, 0.0, 0.0, 0.0, 8.0, 9.0, 8.0, 16.0, 3.0, 0.0, 0.0, 0.0, 3.0, 14.0, 13.0, 13.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 8.0, 0.0, 0.0, 0.0, 13.0, 6.0, 1.0, 7.0, 9.0, 0.0, 0.0, 0.0, 1.0, 10.0, 14.0, 15.0, 2.0, 0.0], [0.0, 0.0, 5.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 5.0, 12.0, 6.0, 0.0, 0.0, 0.0, 4.0, 14.0, 0.0, 2.0, 13.0, 0.0, 0.0, 0.0, 4.0, 10.0, 0.0, 0.0, 9.0, 8.0, 0.0, 0.0, 5.0, 8.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 2.0, 11.0, 0.0, 0.0, 9.0, 6.0, 0.0, 0.0, 0.0, 15.0, 6.0, 8.0, 15.0, 1.0, 0.0, 0.0, 0.0, 4.0, 13.0, 12.0, 3.0, 0.0, 0.0], [0.0, 0.0, 1.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 9.0, 3.0, 2.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 15.0, 4.0, 0.0, 0.0, 0.0, 13.0, 13.0, 6.0, 4.0, 12.0, 0.0, 0.0, 0.0, 9.0, 11.0, 5.0, 9.0, 15.0, 2.0, 0.0, 0.0, 2.0, 12.0, 16.0, 12.0, 6.0, 0.0], [0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 5.0, 10.0, 16.0, 6.0, 0.0, 0.0, 1.0, 7.0, 11.0, 16.0, 13.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 14.0, 1.0, 0.0, 0.0, 0.0, 3.0, 8.0, 14.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 16.0, 12.0, 0.0], [0.0, 0.0, 6.0, 16.0, 16.0, 16.0, 16.0, 10.0, 0.0, 0.0, 6.0, 10.0, 8.0, 14.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 8.0, 0.0, 0.0, 0.0, 2.0, 10.0, 14.0, 15.0, 6.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 13.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 6.0, 16.0, 13.0, 11.0, 1.0, 0.0, 0.0, 0.0, 16.0, 15.0, 12.0, 16.0, 1.0, 0.0, 0.0, 3.0, 16.0, 7.0, 0.0, 13.0, 6.0, 0.0, 0.0, 4.0, 16.0, 0.0, 0.0, 10.0, 8.0, 0.0, 0.0, 8.0, 16.0, 0.0, 0.0, 14.0, 6.0, 0.0, 0.0, 5.0, 16.0, 7.0, 9.0, 16.0, 5.0, 0.0, 0.0, 1.0, 15.0, 16.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 6.0, 16.0, 14.0, 6.0, 0.0, 0.0], [0.0, 0.0, 6.0, 13.0, 10.0, 4.0, 0.0, 0.0, 0.0, 4.0, 16.0, 15.0, 13.0, 13.0, 0.0, 0.0, 0.0, 4.0, 16.0, 14.0, 16.0, 16.0, 1.0, 0.0, 0.0, 2.0, 10.0, 16.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 2.0, 8.0, 16.0, 8.0, 0.0, 0.0, 0.0, 7.0, 16.0, 14.0, 9.0, 1.0, 0.0], [0.0, 0.0, 5.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 9.0, 1.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 1.0, 16.0, 10.0, 8.0, 16.0, 6.0, 0.0, 0.0, 0.0, 12.0, 14.0, 5.0, 9.0, 13.0, 0.0, 0.0, 0.0, 4.0, 15.0, 15.0, 12.0, 3.0, 0.0], [0.0, 0.0, 2.0, 11.0, 14.0, 12.0, 3.0, 0.0, 0.0, 0.0, 14.0, 14.0, 9.0, 15.0, 8.0, 0.0, 0.0, 5.0, 12.0, 0.0, 5.0, 15.0, 2.0, 0.0, 0.0, 3.0, 16.0, 11.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 5.0, 15.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 10.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 13.0, 2.0, 0.0, 0.0], [0.0, 0.0, 3.0, 10.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 13.0, 12.0, 0.0, 2.0, 0.0, 0.0, 0.0, 4.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 14.0, 16.0, 14.0, 2.0, 0.0, 0.0, 6.0, 16.0, 12.0, 5.0, 16.0, 5.0, 0.0, 0.0, 1.0, 12.0, 1.0, 0.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 3.0, 13.0, 13.0, 1.0, 0.0, 0.0, 0.0, 1.0, 13.0, 10.0, 1.0, 0.0, 0.0], [0.0, 0.0, 7.0, 13.0, 11.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 11.0, 16.0, 0.0, 0.0, 0.0, 0.0, 3.0, 8.0, 5.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 8.0, 16.0, 5.0, 0.0, 0.0, 6.0, 8.0, 0.0, 0.0, 11.0, 9.0, 0.0, 0.0, 0.0, 16.0, 6.0, 6.0, 14.0, 6.0, 0.0, 0.0, 0.0, 6.0, 15.0, 16.0, 10.0, 0.0, 0.0], [0.0, 0.0, 1.0, 15.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 9.0, 12.0, 8.0, 12.0, 0.0, 0.0, 0.0, 1.0, 15.0, 1.0, 6.0, 16.0, 2.0, 0.0, 0.0, 2.0, 12.0, 0.0, 1.0, 11.0, 6.0, 0.0, 0.0, 5.0, 10.0, 0.0, 0.0, 11.0, 4.0, 0.0, 0.0, 2.0, 13.0, 0.0, 0.0, 10.0, 3.0, 0.0, 0.0, 0.0, 13.0, 2.0, 3.0, 13.0, 3.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 15.0, 1.0, 0.0], [0.0, 0.0, 10.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 8.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 16.0, 13.0, 11.0, 0.0, 0.0, 0.0, 8.0, 14.0, 8.0, 11.0, 14.0, 1.0], [0.0, 8.0, 16.0, 12.0, 15.0, 16.0, 7.0, 0.0, 0.0, 13.0, 16.0, 14.0, 6.0, 4.0, 1.0, 0.0, 0.0, 12.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 2.0, 0.0, 0.0, 0.0, 1.0, 4.0, 6.0, 16.0, 5.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0], [0.0, 1.0, 12.0, 12.0, 15.0, 16.0, 7.0, 0.0, 0.0, 7.0, 16.0, 16.0, 13.0, 6.0, 1.0, 0.0, 0.0, 12.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 5.0, 7.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 5.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.0, 16.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 9.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 14.0, 16.0, 12.0, 2.0, 0.0, 0.0, 4.0, 16.0, 14.0, 13.0, 11.0, 14.0, 0.0, 0.0, 0.0, 16.0, 5.0, 3.0, 7.0, 16.0, 3.0, 0.0, 0.0, 11.0, 12.0, 8.0, 16.0, 10.0, 0.0, 0.0, 0.0, 2.0, 14.0, 16.0, 12.0, 0.0, 0.0], [0.0, 0.0, 0.0, 14.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 8.0, 3.0, 0.0, 0.0, 0.0, 3.0, 16.0, 15.0, 8.0, 14.0, 2.0, 0.0, 0.0, 0.0, 16.0, 11.0, 0.0, 11.0, 10.0, 0.0, 0.0, 0.0, 9.0, 14.0, 7.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 14.0, 1.0, 0.0], [0.0, 0.0, 4.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 12.0, 14.0, 0.0, 0.0, 0.0, 2.0, 11.0, 0.0, 0.0, 9.0, 6.0, 0.0, 0.0, 5.0, 6.0, 0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 4.0, 9.0, 0.0, 0.0, 7.0, 4.0, 0.0, 0.0, 4.0, 10.0, 0.0, 2.0, 14.0, 0.0, 0.0, 0.0, 0.0, 14.0, 15.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 4.0, 13.0, 10.0, 0.0, 0.0, 0.0], [0.0, 0.0, 10.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 5.0, 16.0, 6.0, 7.0, 14.0, 0.0, 0.0, 0.0, 3.0, 16.0, 3.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 13.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 10.0, 0.0, 0.0, 0.0], [0.0, 2.0, 13.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 7.0, 13.0, 10.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 1.0, 4.0, 10.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 9.0, 0.0, 0.0, 2.0, 12.0, 6.0, 6.0, 16.0, 6.0, 0.0, 0.0, 1.0, 15.0, 16.0, 16.0, 9.0, 1.0, 0.0], [0.0, 0.0, 3.0, 12.0, 12.0, 1.0, 0.0, 0.0, 0.0, 3.0, 13.0, 6.0, 9.0, 12.0, 0.0, 0.0, 0.0, 9.0, 5.0, 0.0, 2.0, 15.0, 0.0, 0.0, 0.0, 7.0, 9.0, 4.0, 12.0, 16.0, 1.0, 0.0, 0.0, 0.0, 9.0, 11.0, 3.0, 10.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 3.0, 0.0, 0.0, 0.0, 10.0, 2.0, 1.0, 13.0, 0.0, 0.0, 0.0, 0.0, 3.0, 13.0, 16.0, 4.0, 0.0, 0.0], [0.0, 1.0, 10.0, 15.0, 15.0, 5.0, 0.0, 0.0, 0.0, 11.0, 16.0, 9.0, 12.0, 10.0, 0.0, 0.0, 0.0, 15.0, 6.0, 0.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 11.0, 8.0, 12.0, 14.0, 1.0, 0.0, 1.0, 15.0, 16.0, 16.0, 12.0, 5.0, 0.0], [0.0, 0.0, 3.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 2.0, 12.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 15.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 3.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 6.0, 8.0, 12.0, 15.0, 12.0, 10.0, 0.0, 0.0, 2.0, 13.0, 16.0, 16.0, 15.0, 11.0], [0.0, 0.0, 1.0, 14.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 11.0, 0.0, 0.0], [0.0, 2.0, 15.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 9.0, 15.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 14.0, 14.0, 16.0, 5.0, 0.0, 0.0, 2.0, 14.0, 16.0, 13.0, 9.0, 1.0, 0.0], [0.0, 1.0, 15.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 10.0, 16.0, 0.0, 0.0, 0.0, 0.0, 4.0, 12.0, 1.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 2.0, 3.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 14.0, 14.0, 15.0, 3.0, 0.0, 1.0, 15.0, 16.0, 16.0, 16.0, 16.0, 5.0], [0.0, 0.0, 0.0, 0.0, 5.0, 11.0, 14.0, 1.0, 0.0, 0.0, 0.0, 10.0, 13.0, 8.0, 15.0, 2.0, 0.0, 0.0, 11.0, 9.0, 4.0, 9.0, 12.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 15.0, 16.0, 13.0, 16.0, 3.0, 0.0, 0.0, 0.0, 2.0, 3.0, 1.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0], [0.0, 0.0, 8.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 4.0, 14.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 14.0, 7.0, 6.0, 3.0, 0.0, 0.0, 5.0, 16.0, 15.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 10.0, 13.0, 8.0, 2.0, 0.0, 0.0], [0.0, 0.0, 1.0, 7.0, 10.0, 3.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 9.0, 12.0, 9.0, 0.0, 0.0], [0.0, 0.0, 1.0, 14.0, 16.0, 16.0, 15.0, 1.0, 0.0, 0.0, 11.0, 14.0, 8.0, 13.0, 11.0, 0.0, 0.0, 0.0, 15.0, 6.0, 0.0, 14.0, 3.0, 0.0, 0.0, 0.0, 5.0, 1.0, 5.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 2.0, 16.0, 15.0, 9.0, 1.0, 0.0, 0.0, 0.0, 0.0, 11.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 2.0, 0.0, 0.0, 0.0], [0.0, 0.0, 6.0, 15.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 14.0, 16.0, 6.0, 6.0, 1.0, 0.0, 0.0, 3.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 8.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 10.0, 12.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 14.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 14.0, 3.0, 0.0, 0.0], [0.0, 2.0, 11.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 12.0, 16.0, 15.0, 16.0, 4.0, 0.0, 0.0, 0.0, 2.0, 3.0, 2.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 3.0, 11.0, 9.0, 0.0, 0.0, 0.0, 16.0, 16.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 14.0, 15.0, 12.0, 5.0, 0.0, 0.0], [0.0, 3.0, 16.0, 13.0, 15.0, 16.0, 11.0, 0.0, 0.0, 5.0, 16.0, 14.0, 12.0, 8.0, 10.0, 0.0, 0.0, 2.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 11.0, 0.0, 0.0, 0.0, 3.0, 6.0, 8.0, 16.0, 7.0, 0.0, 0.0, 0.0, 2.0, 15.0, 15.0, 9.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 12.0, 14.0, 3.0, 0.0, 0.0, 0.0, 1.0, 13.0, 11.0, 9.0, 13.0, 0.0, 0.0, 0.0, 7.0, 11.0, 0.0, 1.0, 16.0, 4.0, 0.0, 0.0, 8.0, 6.0, 0.0, 2.0, 15.0, 0.0, 0.0, 0.0, 4.0, 12.0, 0.0, 0.0, 15.0, 0.0, 0.0, 0.0, 0.0, 15.0, 1.0, 1.0, 15.0, 0.0, 0.0, 0.0, 0.0, 7.0, 10.0, 7.0, 13.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 7.0, 0.0, 0.0], [0.0, 2.0, 11.0, 16.0, 12.0, 1.0, 0.0, 0.0, 0.0, 9.0, 16.0, 9.0, 16.0, 4.0, 0.0, 0.0, 0.0, 14.0, 7.0, 4.0, 16.0, 1.0, 0.0, 0.0, 0.0, 6.0, 5.0, 9.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 11.0, 8.0, 12.0, 9.0, 0.0, 0.0, 2.0, 15.0, 16.0, 16.0, 13.0, 16.0, 1.0], [0.0, 0.0, 10.0, 14.0, 10.0, 1.0, 0.0, 0.0, 0.0, 4.0, 14.0, 6.0, 13.0, 7.0, 0.0, 0.0, 0.0, 6.0, 12.0, 0.0, 7.0, 7.0, 0.0, 0.0, 0.0, 1.0, 16.0, 10.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 13.0, 6.0, 6.0, 15.0, 5.0, 0.0, 0.0, 3.0, 15.0, 0.0, 4.0, 12.0, 7.0, 0.0, 0.0, 0.0, 12.0, 16.0, 15.0, 8.0, 0.0, 0.0], [0.0, 2.0, 14.0, 16.0, 16.0, 13.0, 5.0, 0.0, 0.0, 7.0, 16.0, 13.0, 8.0, 8.0, 1.0, 0.0, 0.0, 10.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 1.0, 8.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 11.0, 0.0, 0.0], [0.0, 0.0, 2.0, 13.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 13.0, 7.0, 7.0, 13.0, 0.0, 0.0, 0.0, 1.0, 7.0, 0.0, 7.0, 13.0, 2.0, 0.0, 0.0, 0.0, 1.0, 10.0, 16.0, 16.0, 13.0, 0.0, 0.0, 0.0, 6.0, 13.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 5.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 12.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 15.0, 5.0, 0.0, 0.0, 0.0, 8.0, 16.0, 0.0, 6.0, 15.0, 1.0, 0.0, 0.0, 7.0, 16.0, 4.0, 10.0, 16.0, 3.0, 0.0, 0.0, 1.0, 11.0, 16.0, 16.0, 12.0, 0.0], [0.0, 1.0, 14.0, 16.0, 16.0, 11.0, 2.0, 0.0, 0.0, 0.0, 14.0, 9.0, 2.0, 10.0, 11.0, 0.0, 0.0, 0.0, 5.0, 16.0, 5.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 5.0, 0.0, 0.0, 0.0, 7.0, 14.0, 14.0, 8.0, 0.0, 0.0, 0.0, 1.0, 14.0, 3.0, 6.0, 11.0, 0.0, 0.0, 0.0, 4.0, 8.0, 0.0, 8.0, 11.0, 0.0, 0.0, 0.0, 2.0, 13.0, 12.0, 15.0, 2.0, 0.0, 0.0], [0.0, 0.0, 9.0, 15.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 9.0, 3.0, 9.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 10.0, 11.0, 15.0, 2.0, 0.0, 0.0, 3.0, 1.0, 0.0, 0.0, 14.0, 4.0, 0.0, 0.0, 10.0, 13.0, 7.0, 2.0, 12.0, 4.0, 0.0, 0.0, 0.0, 7.0, 14.0, 16.0, 10.0, 0.0, 0.0], [0.0, 0.0, 3.0, 15.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 12.0, 12.0, 7.0, 16.0, 6.0, 0.0, 0.0, 4.0, 12.0, 0.0, 9.0, 13.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 13.0, 7.0, 0.0, 0.0, 0.0, 0.0, 8.0, 13.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 12.0, 15.0, 12.0, 6.0, 1.0, 0.0, 0.0, 0.0, 0.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 2.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 14.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 12.0, 0.0, 0.0, 0.0], [0.0, 0.0, 10.0, 13.0, 8.0, 1.0, 0.0, 0.0, 0.0, 0.0, 16.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 6.0, 1.0, 11.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 16.0, 15.0, 4.0, 0.0, 0.0, 2.0, 10.0, 11.0, 7.0, 2.0, 0.0, 0.0], [0.0, 0.0, 8.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 6.0, 12.0, 2.0, 14.0, 13.0, 0.0, 0.0, 0.0, 9.0, 6.0, 1.0, 14.0, 14.0, 0.0, 0.0, 0.0, 2.0, 11.0, 12.0, 8.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 9.0, 0.0, 0.0, 0.0, 2.0, 0.0, 1.0, 12.0, 6.0, 0.0, 0.0, 0.0, 8.0, 9.0, 13.0, 7.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 7.0, 0.0, 0.0, 1.0, 5.0, 11.0, 16.0, 16.0, 8.0, 0.0, 0.0, 11.0, 16.0, 16.0, 13.0, 16.0, 8.0, 0.0, 0.0, 3.0, 7.0, 1.0, 4.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 8.0, 0.0], [0.0, 0.0, 0.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 9.0, 1.0, 0.0, 0.0, 0.0, 14.0, 11.0, 0.0, 8.0, 9.0, 0.0, 0.0, 0.0, 11.0, 14.0, 3.0, 2.0, 14.0, 0.0, 0.0, 0.0, 8.0, 11.0, 4.0, 14.0, 7.0, 0.0, 0.0, 0.0, 1.0, 12.0, 14.0, 6.0, 0.0, 0.0], [0.0, 0.0, 12.0, 8.0, 8.0, 7.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 11.0, 7.0, 0.0, 0.0, 0.0, 2.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 8.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 11.0, 14.0, 9.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 10.0, 8.0, 3.0, 0.0, 0.0, 0.0, 0.0, 16.0, 13.0, 12.0, 14.0, 7.0, 0.0, 0.0, 0.0, 14.0, 9.0, 4.0, 11.0, 13.0, 0.0, 0.0, 0.0, 2.0, 12.0, 16.0, 12.0, 4.0, 0.0], [0.0, 0.0, 10.0, 16.0, 13.0, 12.0, 15.0, 5.0, 0.0, 4.0, 16.0, 8.0, 12.0, 16.0, 6.0, 0.0, 0.0, 6.0, 12.0, 2.0, 16.0, 7.0, 0.0, 0.0, 0.0, 1.0, 5.0, 9.0, 14.0, 1.0, 0.0, 0.0, 0.0, 1.0, 7.0, 16.0, 12.0, 2.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 12.0, 5.0, 0.0, 0.0, 0.0, 1.0, 11.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0], [0.0, 5.0, 12.0, 13.0, 16.0, 16.0, 2.0, 0.0, 0.0, 11.0, 16.0, 15.0, 8.0, 4.0, 0.0, 0.0, 0.0, 8.0, 14.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 1.0, 6.0, 6.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 3.0, 0.0, 0.0, 0.0, 1.0, 5.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 16.0, 2.0, 0.0, 0.0, 0.0], [0.0, 2.0, 15.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 13.0, 12.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 8.0, 6.0, 6.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 13.0, 14.0, 12.0, 0.0, 0.0, 3.0, 16.0, 14.0, 8.0, 8.0, 7.0, 0.0], [0.0, 0.0, 0.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 11.0, 5.0, 0.0, 0.0, 0.0, 0.0, 14.0, 15.0, 12.0, 15.0, 11.0, 0.0, 0.0, 0.0, 12.0, 13.0, 0.0, 0.0, 16.0, 5.0, 0.0, 0.0, 6.0, 15.0, 4.0, 11.0, 16.0, 4.0, 0.0, 0.0, 0.0, 13.0, 16.0, 14.0, 9.0, 0.0], [0.0, 0.0, 1.0, 8.0, 10.0, 8.0, 3.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 8.0, 8.0, 1.0, 0.0, 0.0], [0.0, 0.0, 2.0, 11.0, 13.0, 4.0, 0.0, 0.0, 0.0, 1.0, 13.0, 7.0, 8.0, 15.0, 0.0, 0.0, 0.0, 6.0, 11.0, 0.0, 5.0, 13.0, 0.0, 0.0, 0.0, 9.0, 7.0, 2.0, 14.0, 14.0, 0.0, 0.0, 0.0, 3.0, 14.0, 15.0, 8.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 5.0, 0.0, 0.0, 0.0, 11.0, 7.0, 0.0, 10.0, 7.0, 0.0, 0.0, 0.0, 4.0, 10.0, 15.0, 15.0, 3.0, 0.0], [0.0, 0.0, 8.0, 16.0, 6.0, 3.0, 0.0, 0.0, 0.0, 2.0, 13.0, 5.0, 10.0, 14.0, 0.0, 0.0, 0.0, 4.0, 14.0, 1.0, 9.0, 16.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 8.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 11.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 2.0, 0.0, 0.0, 0.0, 4.0, 1.0, 0.0, 14.0, 1.0, 0.0, 0.0, 0.0, 6.0, 15.0, 16.0, 10.0, 0.0, 0.0], [0.0, 0.0, 3.0, 13.0, 10.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 16.0, 5.0, 0.0, 0.0], [0.0, 0.0, 0.0, 2.0, 9.0, 13.0, 6.0, 0.0, 0.0, 0.0, 11.0, 15.0, 8.0, 9.0, 10.0, 0.0, 0.0, 3.0, 16.0, 10.0, 4.0, 13.0, 5.0, 0.0, 0.0, 1.0, 15.0, 16.0, 15.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0], [0.0, 1.0, 13.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 7.0, 14.0, 9.0, 5.0, 0.0, 0.0, 0.0, 0.0, 6.0, 13.0, 3.0, 12.0, 6.0, 4.0, 0.0, 0.0, 1.0, 14.0, 12.0, 14.0, 16.0, 4.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 11.0, 14.0, 8.0, 13.0, 0.0, 0.0, 0.0, 4.0, 16.0, 4.0, 2.0, 14.0, 2.0, 0.0, 0.0, 1.0, 12.0, 14.0, 13.0, 6.0, 0.0, 0.0], [0.0, 0.0, 0.0, 2.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 8.0, 11.0, 0.0, 0.0, 0.0, 0.0, 12.0, 7.0, 12.0, 8.0, 0.0, 0.0, 0.0, 5.0, 16.0, 12.0, 15.0, 14.0, 5.0, 0.0, 0.0, 8.0, 13.0, 9.0, 16.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 6.0, 0.0, 0.0], [0.0, 0.0, 7.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 12.0, 4.0, 1.0, 0.0, 0.0, 0.0, 6.0, 16.0, 16.0, 16.0, 16.0, 5.0, 0.0, 0.0, 8.0, 16.0, 7.0, 1.0, 15.0, 8.0, 0.0, 0.0, 7.0, 16.0, 0.0, 0.0, 16.0, 4.0, 0.0, 0.0, 2.0, 16.0, 7.0, 10.0, 12.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 13.0, 3.0, 0.0, 0.0], [0.0, 0.0, 1.0, 14.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 2.0, 10.0, 5.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 7.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 4.0, 0.0, 0.0, 0.0, 13.0, 0.0, 0.0, 4.0, 12.0, 0.0, 0.0, 0.0, 13.0, 6.0, 4.0, 8.0, 13.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 15.0, 6.0, 0.0], [0.0, 0.0, 8.0, 11.0, 8.0, 10.0, 0.0, 0.0, 0.0, 3.0, 15.0, 8.0, 12.0, 16.0, 4.0, 0.0, 0.0, 3.0, 12.0, 0.0, 3.0, 16.0, 2.0, 0.0, 0.0, 0.0, 11.0, 10.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 7.0, 9.0, 4.0, 16.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 5.0, 16.0, 0.0, 0.0, 0.0, 0.0, 3.0, 10.0, 9.0, 3.0, 0.0, 0.0], [0.0, 0.0, 0.0, 8.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 11.0, 2.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 11.0, 14.0, 7.0, 0.0, 0.0, 0.0, 11.0, 16.0, 6.0, 6.0, 15.0, 0.0, 0.0, 0.0, 0.0, 10.0, 14.0, 12.0, 8.0, 0.0], [0.0, 0.0, 0.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 0.0, 0.0, 0.0], [0.0, 1.0, 12.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 2.0, 14.0, 5.0, 9.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 8.0, 15.0, 5.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 11.0, 9.0, 0.0, 0.0, 4.0, 14.0, 4.0, 4.0, 14.0, 6.0, 0.0, 0.0, 0.0, 15.0, 16.0, 16.0, 11.0, 1.0, 0.0], [0.0, 0.0, 0.0, 5.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 12.0, 0.0, 2.0, 12.0, 0.0, 0.0, 3.0, 16.0, 1.0, 0.0, 11.0, 10.0, 0.0, 0.0, 9.0, 13.0, 0.0, 3.0, 16.0, 5.0, 0.0, 0.0, 13.0, 15.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 5.0, 12.0, 14.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 2.0, 0.0, 0.0], [0.0, 0.0, 3.0, 15.0, 7.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 11.0, 1.0, 1.0, 7.0, 0.0, 0.0, 8.0, 16.0, 2.0, 0.0, 13.0, 15.0, 0.0, 0.0, 8.0, 16.0, 13.0, 14.0, 16.0, 5.0, 0.0, 0.0, 0.0, 8.0, 9.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 4.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 15.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 13.0, 15.0, 8.0, 3.0, 0.0, 0.0, 0.0, 5.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 5.0, 4.0, 2.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 16.0, 14.0, 2.0, 0.0, 0.0, 4.0, 16.0, 7.0, 1.0, 13.0, 8.0, 0.0, 0.0, 0.0, 11.0, 12.0, 1.0, 11.0, 13.0, 0.0, 0.0, 0.0, 1.0, 12.0, 16.0, 16.0, 10.0, 0.0], [0.0, 0.0, 0.0, 2.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 14.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 16.0, 7.0, 0.0], [0.0, 0.0, 10.0, 10.0, 14.0, 16.0, 14.0, 0.0, 0.0, 0.0, 14.0, 8.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 0.0, 6.0, 11.0, 5.0, 0.0, 0.0, 3.0, 16.0, 14.0, 10.0, 10.0, 9.0, 0.0, 0.0, 3.0, 14.0, 5.0, 0.0, 9.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 13.0, 0.0, 0.0, 0.0, 0.0, 3.0, 9.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 8.0, 13.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 6.0, 15.0, 14.0, 2.0, 0.0, 0.0, 0.0, 5.0, 16.0, 11.0, 14.0, 12.0, 0.0, 0.0, 0.0, 5.0, 11.0, 3.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 10.0, 16.0, 3.0, 0.0, 0.0, 1.0, 4.0, 0.0, 0.0, 12.0, 7.0, 0.0, 0.0, 7.0, 16.0, 5.0, 6.0, 16.0, 5.0, 0.0, 0.0, 1.0, 8.0, 15.0, 16.0, 12.0, 1.0, 0.0], [0.0, 0.0, 0.0, 10.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 5.0, 3.0, 3.0, 0.0, 0.0, 0.0, 15.0, 7.0, 0.0, 13.0, 11.0, 0.0, 0.0, 7.0, 14.0, 1.0, 7.0, 16.0, 8.0, 0.0, 0.0, 9.0, 13.0, 5.0, 15.0, 13.0, 1.0, 0.0, 0.0, 11.0, 16.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 9.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 14.0, 15.0, 5.0, 0.0, 0.0, 0.0, 8.0, 16.0, 5.0, 3.0, 14.0, 0.0, 0.0, 0.0, 5.0, 11.0, 0.0, 0.0, 10.0, 5.0, 0.0, 0.0, 5.0, 9.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 0.0, 14.0, 0.0, 0.0, 10.0, 8.0, 0.0, 0.0, 0.0, 14.0, 13.0, 13.0, 16.0, 1.0, 0.0, 0.0, 0.0, 2.0, 14.0, 14.0, 7.0, 0.0, 0.0], [0.0, 0.0, 1.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 3.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 16.0, 6.0, 15.0, 15.0, 5.0, 0.0, 0.0, 1.0, 16.0, 14.0, 4.0, 3.0, 12.0, 0.0, 0.0, 0.0, 7.0, 7.0, 0.0, 9.0, 12.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 9.0, 2.0, 0.0], [0.0, 0.0, 0.0, 1.0, 13.0, 13.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 15.0, 16.0, 0.0, 0.0, 0.0, 8.0, 16.0, 10.0, 6.0, 14.0, 0.0, 0.0, 0.0, 10.0, 6.0, 0.0, 8.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 7.0, 0.0, 0.0], [0.0, 0.0, 0.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 1.0, 9.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 4.0, 16.0, 8.0, 11.0, 11.0, 0.0, 0.0, 0.0, 3.0, 16.0, 7.0, 4.0, 16.0, 4.0, 0.0, 0.0, 8.0, 16.0, 4.0, 0.0, 16.0, 8.0, 0.0, 0.0, 5.0, 16.0, 10.0, 0.0, 13.0, 11.0, 0.0, 0.0, 0.0, 13.0, 16.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 2.0, 10.0, 13.0, 6.0, 0.0, 0.0], [0.0, 0.0, 7.0, 14.0, 16.0, 11.0, 0.0, 0.0, 0.0, 2.0, 16.0, 11.0, 11.0, 16.0, 2.0, 0.0, 0.0, 0.0, 3.0, 3.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 6.0, 0.0, 0.0, 0.0, 3.0, 12.0, 13.0, 15.0, 2.0, 0.0, 0.0, 0.0, 6.0, 16.0, 12.0, 5.0, 0.0, 0.0], [0.0, 0.0, 5.0, 13.0, 13.0, 8.0, 0.0, 0.0, 0.0, 0.0, 16.0, 11.0, 13.0, 16.0, 6.0, 0.0, 0.0, 1.0, 16.0, 5.0, 2.0, 14.0, 9.0, 0.0, 0.0, 0.0, 9.0, 16.0, 16.0, 15.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 14.0, 14.0, 0.0, 0.0, 0.0, 5.0, 15.0, 4.0, 0.0, 16.0, 6.0, 0.0, 0.0, 6.0, 14.0, 7.0, 6.0, 16.0, 4.0, 0.0, 0.0, 0.0, 7.0, 15.0, 16.0, 10.0, 0.0, 0.0], [0.0, 0.0, 10.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 4.0, 16.0, 6.0, 1.0, 16.0, 2.0, 0.0, 0.0, 0.0, 10.0, 0.0, 1.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 16.0, 12.0, 8.0, 0.0, 0.0, 0.0, 13.0, 15.0, 11.0, 8.0, 14.0, 7.0], [0.0, 0.0, 3.0, 11.0, 13.0, 15.0, 3.0, 0.0, 0.0, 4.0, 16.0, 14.0, 11.0, 16.0, 8.0, 0.0, 0.0, 2.0, 5.0, 0.0, 14.0, 15.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 12.0, 0.0, 0.0, 0.0, 0.0, 8.0, 11.0, 15.0, 8.0, 0.0, 0.0, 0.0, 0.0, 2.0, 12.0, 14.0, 3.0, 0.0, 0.0], [0.0, 0.0, 0.0, 9.0, 13.0, 10.0, 1.0, 0.0, 0.0, 0.0, 9.0, 12.0, 4.0, 15.0, 5.0, 0.0, 0.0, 0.0, 16.0, 4.0, 0.0, 12.0, 4.0, 0.0, 0.0, 3.0, 15.0, 9.0, 3.0, 14.0, 1.0, 0.0, 0.0, 0.0, 2.0, 9.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 10.0, 8.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 12.0, 0.0, 0.0], [0.0, 1.0, 10.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0, 7.0, 14.0, 9.0, 12.0, 12.0, 0.0, 0.0, 0.0, 1.0, 1.0, 5.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 14.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 14.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 0.0, 0.0, 0.0, 7.0, 8.0, 13.0, 16.0, 5.0, 0.0, 0.0, 0.0, 15.0, 16.0, 12.0, 5.0, 0.0, 0.0], [0.0, 1.0, 15.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 11.0, 14.0, 0.0, 0.0, 0.0, 0.0, 12.0, 10.0, 5.0, 16.0, 0.0, 0.0, 0.0, 0.0, 4.0, 7.0, 8.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 14.0, 10.0, 11.0, 12.0, 1.0, 0.0, 0.0, 13.0, 16.0, 16.0, 15.0, 11.0, 1.0], [0.0, 1.0, 10.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 10.0, 11.0, 8.0, 12.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 8.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 5.0, 1.0, 2.0, 0.0, 0.0, 0.0, 10.0, 16.0, 14.0, 14.0, 12.0, 0.0, 0.0, 0.0, 14.0, 16.0, 16.0, 13.0, 7.0, 0.0], [0.0, 0.0, 0.0, 0.0, 7.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 8.0, 0.0, 0.0, 0.0, 1.0, 13.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 16.0, 0.0, 0.0, 0.0, 8.0, 16.0, 3.0, 16.0, 13.0, 0.0, 0.0, 0.0, 2.0, 3.0, 0.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 12.0, 0.0, 0.0], [0.0, 1.0, 10.0, 14.0, 13.0, 4.0, 0.0, 0.0, 0.0, 12.0, 11.0, 5.0, 8.0, 14.0, 0.0, 0.0, 0.0, 8.0, 3.0, 2.0, 12.0, 8.0, 0.0, 0.0, 0.0, 0.0, 3.0, 15.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 4.0, 7.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 11.0, 11.0, 0.0, 0.0, 0.0, 12.0, 16.0, 16.0, 9.0, 1.0, 0.0], [0.0, 0.0, 0.0, 7.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 10.0, 2.0, 6.0, 0.0, 0.0, 0.0, 3.0, 16.0, 3.0, 9.0, 13.0, 2.0, 0.0, 0.0, 11.0, 12.0, 6.0, 14.0, 16.0, 10.0, 0.0, 0.0, 11.0, 16.0, 16.0, 16.0, 10.0, 3.0, 0.0, 0.0, 2.0, 8.0, 10.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 13.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 12.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 7.0, 16.0, 11.0, 12.0, 9.0, 0.0, 0.0, 0.0, 6.0, 7.0, 0.0, 14.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 8.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 5.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 12.0, 12.0, 8.0, 1.0, 0.0, 0.0, 0.0, 2.0, 15.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 14.0, 3.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 3.0, 15.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 9.0, 5.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 13.0, 8.0, 1.0, 16.0, 3.0, 0.0, 0.0, 5.0, 15.0, 2.0, 5.0, 15.0, 0.0, 0.0, 5.0, 15.0, 16.0, 16.0, 16.0, 8.0, 0.0, 0.0, 14.0, 12.0, 12.0, 14.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 5.0, 0.0, 0.0], [0.0, 0.0, 1.0, 9.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 7.0, 1.0, 0.0, 0.0, 0.0, 0.0, 14.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 6.0, 8.0, 4.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 13.0, 14.0, 7.0, 0.0, 0.0, 0.0, 14.0, 3.0, 0.0, 3.0, 12.0, 0.0, 0.0, 0.0, 6.0, 9.0, 7.0, 9.0, 15.0, 0.0, 0.0, 0.0, 0.0, 10.0, 14.0, 14.0, 2.0, 0.0], [0.0, 0.0, 0.0, 12.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 7.0, 2.0, 14.0, 1.0, 0.0, 0.0, 6.0, 16.0, 2.0, 9.0, 16.0, 11.0, 0.0, 0.0, 9.0, 14.0, 9.0, 16.0, 15.0, 6.0, 0.0, 0.0, 5.0, 16.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 11.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 13.0, 0.0, 0.0, 0.0], [0.0, 0.0, 11.0, 13.0, 12.0, 12.0, 3.0, 0.0, 0.0, 5.0, 14.0, 4.0, 4.0, 7.0, 2.0, 0.0, 0.0, 7.0, 10.0, 1.0, 4.0, 1.0, 0.0, 0.0, 0.0, 8.0, 15.0, 14.0, 12.0, 15.0, 2.0, 0.0, 0.0, 2.0, 7.0, 0.0, 0.0, 12.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 3.0, 0.0, 0.0, 1.0, 8.0, 3.0, 10.0, 12.0, 0.0, 0.0, 0.0, 1.0, 12.0, 16.0, 12.0, 2.0, 0.0, 0.0], [0.0, 0.0, 7.0, 13.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 5.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 6.0, 2.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 4.0, 4.0, 0.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 16.0, 3.0, 0.0, 0.0, 6.0, 15.0, 6.0, 9.0, 9.0, 1.0], [0.0, 0.0, 8.0, 15.0, 12.0, 1.0, 0.0, 0.0, 0.0, 8.0, 13.0, 8.0, 12.0, 6.0, 0.0, 0.0, 0.0, 4.0, 2.0, 0.0, 8.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 0.0, 4.0, 4.0, 0.0, 0.0, 0.0, 10.0, 12.0, 9.0, 15.0, 11.0, 0.0, 0.0, 0.0, 9.0, 16.0, 9.0, 7.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 6.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 16.0, 4.0, 0.0, 0.0, 3.0, 11.0, 16.0, 10.0, 16.0, 4.0, 0.0, 0.0, 4.0, 11.0, 3.0, 0.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 4.0, 0.0], [0.0, 0.0, 2.0, 14.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 15.0, 15.0, 8.0, 0.0, 0.0, 0.0, 2.0, 16.0, 7.0, 4.0, 15.0, 0.0, 0.0, 0.0, 4.0, 16.0, 4.0, 0.0, 13.0, 7.0, 0.0, 0.0, 4.0, 16.0, 1.0, 0.0, 10.0, 8.0, 0.0, 0.0, 4.0, 16.0, 5.0, 1.0, 12.0, 11.0, 0.0, 0.0, 1.0, 15.0, 14.0, 13.0, 16.0, 3.0, 0.0, 0.0, 0.0, 3.0, 12.0, 13.0, 5.0, 0.0, 0.0], [0.0, 1.0, 15.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 9.0, 16.0, 4.0, 0.0, 0.0, 0.0, 2.0, 12.0, 5.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 13.0, 4.0, 4.0, 3.0, 0.0, 0.0, 2.0, 13.0, 16.0, 16.0, 16.0, 16.0, 2.0], [0.0, 0.0, 0.0, 9.0, 14.0, 4.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 12.0, 4.0, 0.0, 0.0, 0.0, 1.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 7.0, 3.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 14.0, 16.0, 10.0, 0.0, 0.0, 0.0, 3.0, 16.0, 15.0, 10.0, 16.0, 6.0, 0.0, 0.0, 0.0, 12.0, 16.0, 7.0, 13.0, 9.0, 0.0, 0.0, 0.0, 1.0, 11.0, 16.0, 16.0, 9.0, 0.0], [0.0, 0.0, 7.0, 14.0, 9.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 5.0, 10.0, 7.0, 0.0, 0.0, 0.0, 0.0, 13.0, 2.0, 3.0, 13.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 16.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 10.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 14.0, 0.0, 0.0, 0.0, 4.0, 2.0, 0.0, 0.0, 14.0, 3.0, 0.0, 0.0, 5.0, 15.0, 16.0, 16.0, 12.0, 1.0], [0.0, 0.0, 3.0, 14.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 16.0, 14.0, 9.0, 0.0, 0.0, 0.0, 4.0, 16.0, 16.0, 11.0, 15.0, 3.0, 0.0, 0.0, 5.0, 15.0, 6.0, 0.0, 4.0, 8.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 4.0, 8.0, 0.0, 0.0, 5.0, 11.0, 0.0, 0.0, 6.0, 6.0, 0.0, 0.0, 0.0, 13.0, 10.0, 5.0, 15.0, 5.0, 0.0, 0.0, 0.0, 2.0, 12.0, 14.0, 8.0, 0.0, 0.0], [0.0, 0.0, 0.0, 3.0, 14.0, 6.0, 0.0, 0.0, 0.0, 0.0, 3.0, 14.0, 10.0, 1.0, 0.0, 0.0, 0.0, 1.0, 13.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 14.0, 1.0, 0.0, 7.0, 5.0, 0.0, 0.0, 11.0, 15.0, 8.0, 9.0, 16.0, 10.0, 0.0, 0.0, 7.0, 16.0, 16.0, 15.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 2.0, 0.0, 0.0], [0.0, 0.0, 9.0, 16.0, 16.0, 12.0, 2.0, 0.0, 0.0, 3.0, 13.0, 5.0, 4.0, 14.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 11.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 8.0, 0.0, 0.0, 0.0, 4.0, 5.0, 5.0, 15.0, 4.0, 0.0, 0.0, 0.0, 12.0, 16.0, 15.0, 5.0, 0.0, 0.0], [0.0, 2.0, 10.0, 15.0, 16.0, 16.0, 14.0, 0.0, 0.0, 7.0, 11.0, 4.0, 6.0, 15.0, 9.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 12.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 6.0, 16.0, 2.0, 0.0, 0.0, 0.0, 1.0, 16.0, 16.0, 6.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 16.0, 14.0, 3.0, 0.0, 0.0, 2.0, 14.0, 16.0, 12.0, 10.0, 11.0, 0.0, 0.0, 0.0, 13.0, 12.0, 9.0, 15.0, 10.0, 0.0, 0.0, 0.0, 3.0, 14.0, 14.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 15.0, 16.0, 3.0, 0.0, 0.0], [0.0, 0.0, 10.0, 9.0, 14.0, 10.0, 0.0, 0.0, 0.0, 2.0, 15.0, 15.0, 4.0, 14.0, 2.0, 0.0, 0.0, 0.0, 13.0, 5.0, 9.0, 12.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 11.0, 12.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 1.0, 12.0, 0.0, 0.0, 0.0, 0.0, 3.0, 12.0, 2.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 7.0, 0.0, 0.0, 0.0], [0.0, 0.0, 6.0, 10.0, 9.0, 4.0, 0.0, 0.0, 0.0, 0.0, 14.0, 10.0, 16.0, 16.0, 1.0, 0.0, 0.0, 4.0, 15.0, 1.0, 9.0, 16.0, 0.0, 0.0, 0.0, 3.0, 16.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 12.0, 14.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 12.0, 1.0, 9.0, 12.0, 0.0, 0.0, 0.0, 0.0, 16.0, 6.0, 14.0, 9.0, 0.0, 0.0, 0.0, 0.0, 8.0, 12.0, 6.0, 1.0, 0.0, 0.0], [0.0, 1.0, 10.0, 16.0, 16.0, 8.0, 0.0, 0.0, 0.0, 10.0, 16.0, 13.0, 16.0, 12.0, 0.0, 0.0, 0.0, 1.0, 3.0, 3.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 16.0, 12.0, 3.0, 0.0, 0.0, 0.0, 0.0, 5.0, 11.0, 16.0, 11.0, 0.0, 0.0, 0.0, 2.0, 7.0, 14.0, 16.0, 6.0, 0.0, 0.0, 0.0, 11.0, 16.0, 13.0, 5.0, 0.0, 0.0], [0.0, 0.0, 5.0, 16.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 15.0, 8.0, 0.0, 6.0, 5.0, 0.0, 0.0, 9.0, 16.0, 6.0, 12.0, 16.0, 9.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 15.0, 1.0, 0.0, 0.0, 0.0, 3.0, 10.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 1.0, 14.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 2.0, 0.0, 0.0, 0.0], [0.0, 0.0, 7.0, 13.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 7.0, 16.0, 15.0, 3.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 10.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 5.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 15.0, 13.0, 0.0, 0.0, 0.0, 0.0, 8.0, 16.0, 16.0, 16.0, 15.0, 9.0, 0.0, 0.0, 0.0, 4.0, 10.0, 14.0, 8.0, 5.0, 0.0, 0.0, 0.0, 0.0, 8.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 10.0, 0.0, 0.0, 0.0], [0.0, 0.0, 2.0, 12.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 10.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 9.0, 5.0, 11.0, 8.0, 0.0, 0.0, 0.0, 4.0, 16.0, 14.0, 6.0, 12.0, 5.0, 0.0, 0.0, 0.0, 13.0, 7.0, 0.0, 10.0, 8.0, 0.0, 0.0, 0.0, 3.0, 14.0, 16.0, 16.0, 5.0, 0.0], [0.0, 0.0, 7.0, 13.0, 8.0, 6.0, 0.0, 0.0, 0.0, 0.0, 16.0, 15.0, 16.0, 14.0, 10.0, 0.0, 0.0, 4.0, 16.0, 13.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 10.0, 16.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 11.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 3.0, 0.0, 0.0, 0.0], [0.0, 1.0, 12.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 7.0, 6.0, 5.0, 14.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 16.0, 10.0, 8.0, 4.0, 0.0, 0.0, 0.0, 13.0, 15.0, 16.0, 12.0, 7.0, 0.0], [0.0, 0.0, 0.0, 1.0, 7.0, 12.0, 14.0, 1.0, 0.0, 0.0, 1.0, 13.0, 8.0, 4.0, 13.0, 0.0, 0.0, 0.0, 10.0, 16.0, 9.0, 15.0, 11.0, 0.0, 0.0, 1.0, 16.0, 15.0, 15.0, 16.0, 3.0, 0.0, 0.0, 0.0, 11.0, 9.0, 3.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 6.0, 0.0, 0.0], [0.0, 0.0, 2.0, 8.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 15.0, 16.0, 2.0, 0.0, 0.0, 0.0, 6.0, 15.0, 11.0, 16.0, 4.0, 0.0, 0.0, 0.0, 5.0, 16.0, 10.0, 16.0, 1.0, 0.0, 0.0, 0.0, 2.0, 15.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 16.0, 12.0, 9.0, 3.0, 0.0, 0.0, 0.0, 4.0, 14.0, 0.0, 12.0, 14.0, 1.0, 0.0, 0.0, 1.0, 12.0, 10.0, 7.0, 0.0, 0.0], [0.0, 0.0, 0.0, 3.0, 15.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 3.0, 4.0, 3.0, 0.0, 0.0, 7.0, 16.0, 5.0, 3.0, 15.0, 8.0, 0.0, 0.0, 13.0, 16.0, 13.0, 15.0, 16.0, 2.0, 0.0, 0.0, 12.0, 16.0, 16.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 16.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 4.0, 0.0, 0.0], [0.0, 0.0, 7.0, 12.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 12.0, 10.0, 5.0, 14.0, 0.0, 0.0, 0.0, 6.0, 13.0, 13.0, 3.0, 15.0, 0.0, 0.0, 0.0, 8.0, 9.0, 11.0, 16.0, 8.0, 0.0, 0.0, 0.0, 1.0, 11.0, 10.0, 9.0, 11.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 12.0, 0.0, 0.0, 0.0, 8.0, 12.0, 16.0, 13.0, 2.0, 0.0], [0.0, 0.0, 0.0, 0.0, 10.0, 13.0, 7.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 11.0, 0.0, 0.0, 0.0, 4.0, 14.0, 16.0, 16.0, 7.0, 0.0, 0.0, 3.0, 14.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 7.0, 16.0, 16.0, 16.0, 16.0, 4.0, 0.0, 0.0, 0.0, 2.0, 13.0, 16.0, 16.0, 3.0, 0.0, 0.0, 0.0, 0.0, 11.0, 16.0, 16.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 13.0, 16.0, 1.0, 0.0], [0.0, 0.0, 8.0, 15.0, 16.0, 16.0, 6.0, 0.0, 0.0, 2.0, 16.0, 11.0, 5.0, 0.0, 0.0, 0.0, 0.0, 3.0, 16.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 13.0, 0.0, 0.0, 0.0, 0.0, 10.0, 13.0, 6.0, 15.0, 5.0, 0.0, 0.0, 0.0, 3.0, 1.0, 0.0, 11.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 16.0, 4.0, 0.0, 0.0, 0.0, 0.0, 9.0, 16.0, 8.0, 0.0, 0.0, 0.0]]

y = [False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, True, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, False, False, False, True, False, False, False, False, True, False, False, False, True, False, False, True, False, False, False, False, False, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, True, True, False, True, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, True, False, False, False, False, False, False, False, False, False, True, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False]

In [4]:
# Instantiate an RBF SVM
svm = SVC()

# Instantiate the GridSearchCV object and run the search
parameters = {'gamma':[0.00001, 0.0001, 0.001, 0.01, 0.1]}
searcher = GridSearchCV(svm, parameters)
searcher.fit(X, y)

# Report the best parameters
print("Best CV params", searcher.best_params_)

# Larger values of gamma are better for training accuracy, but cross-validation helped us find something different (and better!).

Best CV params {'gamma': 0.001}


**Jointly tuning gamma and C with GridSearchCV**   
In the previous exercise the best value of gamma was 0.001 using the default value of C, which is 1. In this exercise you'll search for the best combination of C and gamma using GridSearchCV.

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split



# Split into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# Instantiate an RBF SVM
svm = SVC()

# Instantiate the GridSearchCV object and run the search
parameters = {'C':[0.1, 1, 10], 'gamma':[0.00001, 0.0001, 0.001, 0.01, 0.1]}
searcher = GridSearchCV(svm, parameters)
searcher.fit(X_train, y_train)

# Report the best parameters and the corresponding score
print("Best CV params", searcher.best_params_)
print("Best CV accuracy", searcher.best_score_)

# Report the test accuracy using these best parameters
print("Test accuracy of best grid search hypers:", searcher.score(X_test, y_test))

# Note that the best value of gamma, 0.0001, is different from the value of 0.001
# that we got in the previous exercise, when we fixed C=1. Hyperparameters can affect each other!

Best CV params {'C': 10, 'gamma': 0.0001}
Best CV accuracy 0.9986013986013986
Test accuracy of best grid search hypers: 1.0


**Using SGDClassifier**  
In this final coding exercise, you'll do a hyperparameter search over the regularization strength and the loss (logistic regression vs. linear SVM) using SGDClassifier().

In [16]:


# We set random_state=0 for reproducibility
linear_classifier = SGDClassifier(random_state=0)

# Instantiate the GridSearchCV object and run the search
parameters = {'alpha':[0.00001, 0.0001, 0.001, 0.01, 0.1, 1],
             'loss':['log_loss', 'hinge']}
searcher = GridSearchCV(linear_classifier, parameters, cv=10)
searcher.fit(X_train, y_train)

# Report the best parameters and the corresponding score
print("Best CV params", searcher.best_params_)
print("Best CV accuracy", searcher.best_score_)
print("Test accuracy of best grid search hypers:", searcher.score(X_test, y_test))

# you finished the last exercise in the course! One advantage of SGDClassifier is
# that it's very fast - this would have taken a lot longer with LogisticRegression or LinearSVC

Best CV params {'alpha': 1e-05, 'loss': 'log_loss'}
Best CV accuracy 0.9986111111111111
Test accuracy of best grid search hypers: 1.0
